In [1]:
import mysql.connector

In [ ]:
mydb = mysql.connector.connect(host=)
mycursor = mydb.cursor()
mycursor.execute("show databases")

In [ ]:
import socket
import threading
import time


HEADER = 64
PORT = 5050
SERVER = socket.gethostbyname(socket.gethostname())
ADDR = (SERVER,PORT)
FORMAT = "utf-8"
DISCONNECT_MESSAGE = "!DISCONNECT"

server = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
server.bind(ADDR)

def handle_client(conn,addr):
    print(f"[CONNECTION] {addr} connected.")
    connected = True
    while connected:
        msg_length = conn.recv(HEADER).decode(FORMAT)
        if msg_length:
            msg_length = int(msg_length)
            msg = conn.recv(msg_length).decode(FORMAT)
            if msg == DISCONNECT_MESSAGE:
                connected = False
            print(msg)
    conn.close()

def start():
    server.listen()
    print(f"[LISTENING] Server is listening on {SERVER}")
    while True:
        conn, addr = server.accept()
        thread = threading.Thread(target=handle_client,args=(conn,addr))
        thread.start()
        #print(f"[ACTIVE CONNECTIONS] {threading.activeCount()-1}")

print("[STARTING] server is starting")
start()

[STARTING] server is starting
[LISTENING] Server is listening on 192.168.1.55
[CONNECTION] ('192.168.1.55', 57166) connected.
...
!DISCONNECT


[CONNECTION] ('192.168.1.55', 53429) connected.


In [104]:
import psycopg2
import yaml

In [105]:
def yaml_loader(file_path):
    with open(file_path,'r') as file_descriptor:
        data = yaml.load(file_descriptor,Loader=yaml.FullLoader)
    return data

In [106]:
data = yaml_loader('items.yaml')['items']
conn = psycopg2.connect(database = data['database'],
                        user = data['username'],
                        password = data['password'],
                        host = data['host'], 
                        port = data['port'])

print("Database loaded successfully")

Database loaded successfully


In [107]:
cur = conn.cursor()
cur.execute("""
SELECT id, base64 FROM fa_record
WHERE result IS NULL AND confidence IS NULL
""")
rows = cur.fetchall()

In [108]:
len(rows)

3

In [1]:
import os
import cv2
import numpy as np
import argparse
import warnings
import time
import base64
from PIL import Image
from io import BytesIO

from src.anti_spoof_predict import AntiSpoofPredict
from src.generate_patches import CropImage
from src.utility import parse_model_name
warnings.filterwarnings('ignore')


SAMPLE_IMAGE_PATH = "./images/sample/"


"""
def check_image(image):
    height, width, channel = image.shape
    if width/height != 3/4:
        print("Image is not appropriate!!!\nHeight/Width should be 4/3.")
        return False
    else:
        return True"""


def model_prediction(image, model_dir, device_id, threshold):
    model_test = AntiSpoofPredict(device_id)
    image_cropper = CropImage()
    #image = cv2.imread(SAMPLE_IMAGE_PATH + image_name)
    """result = check_image(image)
    if result is False:
        return"""
    image_bbox = model_test.get_bbox(image)
    prediction = np.zeros((1, 3))
    # sum the prediction from single model's result
    for model_name in os.listdir(model_dir):
        h_input, w_input, model_type, scale = parse_model_name(model_name)
        param = {
            "org_img": image,
            "bbox": image_bbox,
            "scale": scale,
            "out_w": w_input,
            "out_h": h_input,
            "crop": True,
        }
        if scale is None:
            param["crop"] = False
        img = image_cropper.crop(**param)
        start = time.time()
        prediction += model_test.predict(img, os.path.join(model_dir, model_name))

    # draw result of prediction
    #print(prediction)
    label = np.argmax(prediction)
    value = prediction[0][label]/2
    if label == 1 and value > threshold:
        #print("Image '{}' is Real Face. Score: {:.2f}.".format(image_name, value))
        #result_text = "Real Score: {:.2f}".format(value)
        #color = (0, 255, 0)
        label = 1
    else:
        #print("Image '{}' is Fake Face. Score: {:.2f}.".format(image_name, value))
        #result_text = "Fake Score: {:.2f}".format(value)
        #color = (0, 0, 255)
        label = 0
    #print("Prediction cost {:.2f} s".format(test_speed))
    #cv2.rectangle(image,(image_bbox[0], image_bbox[1]),(image_bbox[0] + image_bbox[2], image_bbox[1] + image_bbox[3]),color, 2)
    #cv2.putText(image,result_text,(image_bbox[0], image_bbox[1] - 5),cv2.FONT_HERSHEY_COMPLEX, 1.0*image.shape[0]/1024, color)
    #print(prediction)
    #format_ = os.path.splitext(image_name)[-1]
    #result_image_name = image_name.replace(format_, "_result" + format_)
    #cv2.imwrite(SAMPLE_IMAGE_PATH + result_image_name, image)
    return label, value

model_dir = 'resources/anti_spoof_models'

d:\pyvenv\rapid\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [98]:
data_list = list()
for ID, BASE64 in rows:
    img = np.array(Image.open(BytesIO(base64.b64decode(BASE64))))
    label, value = model_prediction(cv2.cvtColor(img,cv2.COLOR_BGR2RGB),model_dir,0,0.0)
    data_dict = {'id':ID,'result':label,'confidence':value}
    data_list.append(data_dict)

(80, 80, 3)
torch.Size([3, 80, 80])
(80, 80, 3)
torch.Size([3, 80, 80])
(80, 80, 3)
torch.Size([3, 80, 80])
(80, 80, 3)
torch.Size([3, 80, 80])


In [100]:
data_list

[{'id': 3, 'result': 1, 'confidence': 0.9980190992355347},
 {'id': 2, 'result': 0, 'confidence': 0.5645099133253098}]

In [99]:
sql = (f"""UPDATE fa_record
       SET result = %(result)s, confidence = %(confidence)s
       WHERE id= %(id)s""")

In [101]:
cur.executemany(sql, data_list)
conn.commit()

In [68]:
i = 2
if i:
    print('yes')

yes


In [9]:
conn.close()

In [2]:
import os
import cv2
import numpy as np
import argparse
import warnings
import time
import base64
from PIL import Image
from io import BytesIO

from src.anti_spoof_predict import AntiSpoofPredict
from src.generate_patches import CropImage
from src.utility import parse_model_name
warnings.filterwarnings('ignore')


SAMPLE_IMAGE_PATH = "./images/sample/"


"""
def check_image(image):
    height, width, channel = image.shape
    if width/height != 3/4:
        print("Image is not appropriate!!!\nHeight/Width should be 4/3.")
        return False
    else:
        return True"""


def model_prediction(image, model_dir, device_id, threshold):
    model_test = AntiSpoofPredict(device_id)
    image_cropper = CropImage()
    #image = cv2.imread(SAMPLE_IMAGE_PATH + image_name)
    """result = check_image(image)
    if result is False:
        return"""
    image_bbox = model_test.get_bbox(image)
    prediction = np.zeros((1, 3))
    # sum the prediction from single model's result
    for model_name in os.listdir(model_dir):
        h_input, w_input, model_type, scale = parse_model_name(model_name)
        param = {
            "org_img": image,
            "bbox": image_bbox,
            "scale": scale,
            "out_w": w_input,
            "out_h": h_input,
            "crop": True,
        }
        if scale is None:
            param["crop"] = False
        img = image_cropper.crop(**param)
        start = time.time()
        prediction += model_test.predict(img, os.path.join(model_dir, model_name))

    # draw result of prediction
    #print(prediction)
    label = np.argmax(prediction)
    value = prediction[0][label]/2
    if label == 1 and value > threshold:
        #print("Image '{}' is Real Face. Score: {:.2f}.".format(image_name, value))
        #result_text = "Real Score: {:.2f}".format(value)
        #color = (0, 255, 0)
        label = 1
    else:
        #print("Image '{}' is Fake Face. Score: {:.2f}.".format(image_name, value))
        #result_text = "Fake Score: {:.2f}".format(value)
        #color = (0, 0, 255)
        label = 0
    #print("Prediction cost {:.2f} s".format(test_speed))
    #cv2.rectangle(image,(image_bbox[0], image_bbox[1]),(image_bbox[0] + image_bbox[2], image_bbox[1] + image_bbox[3]),color, 2)
    #cv2.putText(image,result_text,(image_bbox[0], image_bbox[1] - 5),cv2.FONT_HERSHEY_COMPLEX, 1.0*image.shape[0]/1024, color)
    #print(prediction)
    #format_ = os.path.splitext(image_name)[-1]
    #result_image_name = image_name.replace(format_, "_result" + format_)
    #cv2.imwrite(SAMPLE_IMAGE_PATH + result_image_name, image)
    return label, value

model_dir = 'resources/anti_spoof_models'

In [3]:
from flask import Flask, request
from flask_restful import Api, Resource

In [19]:
app = Flask(__name__)
api = Api(app)

class Process(Resource):
    def get(self):
        return {'status': 'processed'}
    def post(self, base64):
        return {'post': base64}

api.add_resource(Process, "/process/<string:base64>")
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [30/Jul/2021 10:07:47] "POST /process/a18kt0047 HTTP/1.1" 200 -


In [7]:
app = Flask(__name__)
api = Api(app)

#@app.route('/process', methods = ['POST'])
class Process(Resource):
    def post(self):
        print(request.is_json)
        if request.is_json == True:
            content = request.json
            #print('here')
            img = np.array(Image.open(BytesIO(base64.b64decode(content['data']))))
            label, value = model_prediction(cv2.cvtColor(img,cv2.COLOR_BGR2RGB),model_dir,0,0.0)
            print({'result': label, 'confidence': value})
            return {'result': label, 'confidence': value}

api.add_resource(Process, "/process")
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


True


127.0.0.1 - - [05/Aug/2021 12:34:42] "POST /process HTTP/1.1" 200 -


{'result': 1, 'confidence': 0.9999997615814209}


In [71]:
base = '...'

In [72]:
base2 = '/9j/4AAQSkZJRgABAgEA/gD%2BAAD/7QvCUGhvdG9zaG9wIDMuMAA4QklNA%2B0AAAAAABAA/gAAAAEAAgD%2BAAAAAQACOEJJTQQNAAAAAAAEAAAAeDhCSU0D8wAAAAAACAAAAAAAAAAAOEJJTQQKAAAAAAABAAA4QklNJxAAAAAAAAoAAQAAAAAAAAACOEJJTQP1AAAAAABIAC9mZgABAGxmZgAGAAAAAAABAC9mZgABAKGZmgAGAAAAAAABADIAAAABAFoAAAAGAAAAAAABADUAAAABAC0AAAAGAAAAAAABOEJJTQP4AAAAAABwAAD/////////////////////////////A%2BgAAAAA/////////////////////////////wPoAAAAAP////////////////////////////8D6AAAAAD/////////////////////////////A%2BgAADhCSU0ECAAAAAAAEAAAAAEAAAJAAAACQAAAAAA4QklNBBQAAAAAAAQAAAABOEJJTQQMAAAAAAoxAAAAAQAAAFoAAABwAAABEAAAdwAAAAoVABgAAf/Y/%2BAAEEpGSUYAAQIBAEgASAAA/%2B4ADkFkb2JlAGSAAAAAAf/bAIQADAgICAkIDAkJDBELCgsRFQ8MDA8VGBMTFRMTGBEMDAwMDAwRDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAENCwsNDg0QDg4QFA4ODhQUDg4ODhQRDAwMDAwREQwMDAwMDBEMDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwM/8AAEQgAcABaAwEiAAIRAQMRAf/dAAQABv/EAT8AAAEFAQEBAQEBAAAAAAAAAAMAAQIEBQYHCAkKCwEAAQUBAQEBAQEAAAAAAAAAAQACAwQFBgcICQoLEAABBAEDAgQCBQcGCAUDDDMBAAIRAwQhEjEFQVFhEyJxgTIGFJGhsUIjJBVSwWIzNHKC0UMHJZJT8OHxY3M1FqKygyZEk1RkRcKjdDYX0lXiZfKzhMPTdePzRieUpIW0lcTU5PSltcXV5fVWZnaGlqa2xtbm9jdHV2d3h5ent8fX5/cRAAICAQIEBAMEBQYHBwYFNQEAAhEDITESBEFRYXEiEwUygZEUobFCI8FS0fAzJGLhcoKSQ1MVY3M08SUGFqKygwcmNcLSRJNUoxdkRVU2dGXi8rOEw9N14/NGlKSFtJXE1OT0pbXF1eX1VmZ2hpamtsbW5vYnN0dXZ3eHl6e3x//aAAwDAQACEQMRAD8A9LHCdMOAlKaldDvyKMet1lz21saJc9xhoA5c5x%2Biq3Veq4vS8G3NynbaqhJ8SeGsZ/Ke5eNfWH61dT69e45DzXjA/osVhhjR/wAJH89Z/LegSkB9A6p/jQ6JivNeBVZ1Ejmxv6Kqf5Nj2udZ/Yr2LGs/xsZbv5nptLXeNlj3D/otrXnpmYU2gntEeCFlID6Tg/41Kydufglo/fx3bo/63bs/6pdd0b6xdI63WX9PvD3sE2UOBbayf36nf9Wz2LwqbAYDvvVnDzsvByqsrEtNOTju3VWNjQ9/67HfRsrd/OIAlRi%2B/pLn/qh9bsX6x4kO209SoE5WKD2%2Bj9oo3e5%2BO/8A8As/R2LoE9aslCdMkp//0PSZ0Sk8DlNOiFkZLMWi3Ks/m8djrXx%2B7W02O/6lNS%2BT/wCMP6xW9S6xb0%2Bqwt6fgP8ATDAdLLm/ztzv6rv0Nf7i5Rg9Rwa0Ek8BqfJuszMmy0iH32OtcB4vcbD/ANUup6D0plDWveJedSopzERfUsuOBka6OVjfV7OyG7nM9Nvi5XK/qnkNIJfzpAXZY9LSIVuuhpUJyzLbGCA3Dwd31ZyKwXtdugcRCxsnEvx3lrxqDqCvV347I4WN1XpOPksO5vuA0KUcpB11RPl4kenR4TAzsrpmfRn4bizIxnixhmJ/fqfH%2BCuZ%2BjsXu/S%2BoUdT6djdRx9KcuptrGnlu4e6t38qt3sXg/U8SzCvLXcHgr1P/FblWX/Vh1TzIxMq2qvyY8Mydv8AZfe9WYGw0pii9ckkknLX/9H0adFm/WNzW/V7qZcYH2S6SdPzHf8AflfJ0VDr1Rv6H1CpupfjWtAiZJY4Mb/n7dqal8T6XWH5rBE/7F3OKzQLkfqrT6uU%2BwjSqsT8XLoxb1OSa304zG/RY8Gx5/4wtIZ/mqtlFy8m1g0Hm9Djt05VpkgcrlK%2Bu9UotDMnHrurJg20OggHv6b1v42X6oBaZCjIptRILddJCq3sJBBVHqnWcnFAbjUtusPJe7a1oH7yoUZ3UMx039SZjuPFOOwED%2B1Ydz0OG%2BqTLpVuP9b8YtLLe3HzH/mK6/8AxREjoWcwzAy5HhrVX/5FYfXse%2B7ouT6xZbbS31G2sG0ODfztn5j9v01v/wCKq/GHQbMbfGTZkW3GuCJYBVVua6Njtu381WMJ0o96aPMRPESB0svbSnTJKZgf/9L0EnRY31vxbcn6v5PokizHLMkDkOFTv0jHD879G571sFDextrH0vMMta6tx8A8FhP/AEkwiwR3XQlwyEv3SC%2BV/VnG9Nuc7STaG6dvpOj/AKSh1Dp2TbkE5LrPsZBAbSPduIhj3gFrnMa78xi0On49mDdl4lohwfuPYyxxpta7%2Bq7atWvbY0ePmqxmRK27HECDEF5TpXRrGYtov3MyQ5v2csLgyGl/q2Xeu7d%2Bla5jNtTP8Gum%2Brs1G2tx3AH2E%2BEIttYawnuodKj1HOHB4KE58WrJjxiJAF/VpZuC3JtNl%2B99W526thgxBbXt/q/TWLhfV17Kbq7aX2ZL9jcfIrJYGbXOdZdd6j3%2Bo%2B1jvTfV6fpLsKWNNrmnnlWRWW99Eo5DGJATLCJEE7jxc%2BvBsb05%2BPkO3ufU5jj4y0tVj/Fp0/0xZkudPoUClgPY2u32H/NpaiXOlrgOYIH3La%2BqWEcXAteRAus9nm1g2bv%2B3N6OHWQ%2B1j5iowketCI/wj6nbSlJJWmg/wD/0%2B/KGSplDcU1Ti9e6VRY2zqVZLMhjQ21ojbYJDd7/wA5trPb72/TWDjW7SAuzuqF1NlLuLGln3jT/pLiNrmOLXCHNMOHmNCoM0erb5aZ1BOzZy91tD2MMOLSB8VUweo3Y17m2Yzm1ggMP0gfFvt%2Bio39RrxR%2BmDteNrS7/qQhN6/jDR1NoB/O2mf80t/78oQDWzbHqOnR1ce3Jtvc99XpNaSGmZLh5q76hI1Oqx6uv0OI3UXNnQODCR5fmtV%2Buz1G7oIB7HQ/MIEEbria0Ld6ZjszOoMotk1EOLwCWkhoLvpD6K62tldVbaq2hlbAGsaOABwFz/1aoJyLsgj21s9MH%2BU87v%2BoYuglWsMajfdzuZmZZCL0C5KaUpTSpGF/9TvSVAqXZQKapRXM/WbEZRk15TDByt29n8tgbLm/wBdrl0ok8crA%2Bt4Bqw2Ew7fY6O4EMEpmT5SvxEiYp590vEcg6EKdNOU0RVZtafzXCUBtrmGH/53j8VbqzWgCY%2BKrajZ0Mc/Gk1VVwdutO5/ijExrx4qAzGuEc/BMNz/AHO08Amm9ymUr629d0ANHSKHtEG7fY7zJc5n/UMatBZPQM3FPSqaXWtZZTuY9jiAfpOe139VzXrVBBEgyDqCO6uQ%2BWPkHOn88r7ldNqlKaU5Y//V7qU0FxgcqPJgnRWKQNyFKcb61Yto6T9qpkPxHh74JB9N36Ox2n%2Bjlli5KXucXPcXuiJJJP4r02ytpaWPaH1vBa9p4LXDa5p/rNXn/UukXdKyjQ6X4ztcW8/nM/0dn/D0/Qs/0n88oM8ToRsz4CNR1aLmhw81Kups6hOWEjTkJ65UDYAbLGABGkBsIDA4orWvc4MY0ve7RrGAucT4NY33IFexFdj3NZV/PWuFdXb3vOyv/pOXoDccVUV1btxqY1m%2BIJ2jbuP9ZZnQegOxHDNzQPtURVUCHCoEQ524e12Q/wDeb/Ns/R1/4TftO4/KrWDGYgk9ejTzzEpADp1ajtNCmlTeO8QI7qGwf6%2BKlpif/9kAOEJJTQQGAAAAAAAHAAgBAQADAQD/4gxYSUNDX1BST0ZJTEUAAQEAAAxITGlubwIQAABtbnRyUkdCIFhZWiAHzgACAAkABgAxAABhY3NwTVNGVAAAAABJRUMgc1JHQgAAAAAAAAAAAAAAAAAA9tYAAQAAAADTLUhQICAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABFjcHJ0AAABUAAAADNkZXNjAAABhAAAAGx3dHB0AAAB8AAAABRia3B0AAACBAAAABRyWFlaAAACGAAAABRnWFlaAAACLAAAABRiWFlaAAACQAAAABRkbW5kAAACVAAAAHBkbWRkAAACxAAAAIh2dWVkAAADTAAAAIZ2aWV3AAAD1AAAACRsdW1pAAAD%2BAAAABRtZWFzAAAEDAAAACR0ZWNoAAAEMAAAAAxyVFJDAAAEPAAACAxnVFJDAAAEPAAACAxiVFJDAAAEPAAACAx0ZXh0AAAAAENvcHlyaWdodCAoYykgMTk5OCBIZXdsZXR0LVBhY2thcmQgQ29tcGFueQAAZGVzYwAAAAAAAAASc1JHQiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAABJzUkdCIElFQzYxOTY2LTIuMQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAWFlaIAAAAAAAAPNRAAEAAAABFsxYWVogAAAAAAAAAAAAAAAAAAAAAFhZWiAAAAAAAABvogAAOPUAAAOQWFlaIAAAAAAAAGKZAAC3hQAAGNpYWVogAAAAAAAAJKAAAA%2BEAAC2z2Rlc2MAAAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAAAAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABkZXNjAAAAAAAAAC5JRUMgNjE5NjYtMi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAAAAAAAAAAAC5JRUMgNjE5NjYtMi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAAAAAAAAAAAAAAAAAAAAAAAAAAZGVzYwAAAAAAAAAsUmVmZXJlbmNlIFZpZXdpbmcgQ29uZGl0aW9uIGluIElFQzYxOTY2LTIuMQAAAAAAAAAAAAAALFJlZmVyZW5jZSBWaWV3aW5nIENvbmRpdGlvbiBpbiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAHZpZXcAAAAAABOk/gAUXy4AEM8UAAPtzAAEEwsAA1yeAAAAAVhZWiAAAAAAAEwJVgBQAAAAVx/nbWVhcwAAAAAAAAABAAAAAAAAAAAAAAAAAAAAAAAAAo8AAAACc2lnIAAAAABDUlQgY3VydgAAAAAAAAQAAAAABQAKAA8AFAAZAB4AIwAoAC0AMgA3ADsAQABFAEoATwBUAFkAXgBjAGgAbQByAHcAfACBAIYAiwCQAJUAmgCfAKQAqQCuALIAtwC8AMEAxgDLANAA1QDbAOAA5QDrAPAA9gD7AQEBBwENARMBGQEfASUBKwEyATgBPgFFAUwBUgFZAWABZwFuAXUBfAGDAYsBkgGaAaEBqQGxAbkBwQHJAdEB2QHhAekB8gH6AgMCDAIUAh0CJgIvAjgCQQJLAlQCXQJnAnECegKEAo4CmAKiAqwCtgLBAssC1QLgAusC9QMAAwsDFgMhAy0DOANDA08DWgNmA3IDfgOKA5YDogOuA7oDxwPTA%2BAD7AP5BAYEEwQgBC0EOwRIBFUEYwRxBH4EjASaBKgEtgTEBNME4QTwBP4FDQUcBSsFOgVJBVgFZwV3BYYFlgWmBbUFxQXVBeUF9gYGBhYGJwY3BkgGWQZqBnsGjAadBq8GwAbRBuMG9QcHBxkHKwc9B08HYQd0B4YHmQesB78H0gflB/gICwgfCDIIRghaCG4IggiWCKoIvgjSCOcI%2BwkQCSUJOglPCWQJeQmPCaQJugnPCeUJ%2BwoRCicKPQpUCmoKgQqYCq4KxQrcCvMLCwsiCzkLUQtpC4ALmAuwC8gL4Qv5DBIMKgxDDFwMdQyODKcMwAzZDPMNDQ0mDUANWg10DY4NqQ3DDd4N%2BA4TDi4OSQ5kDn8Omw62DtIO7g8JDyUPQQ9eD3oPlg%2BzD88P7BAJECYQQxBhEH4QmxC5ENcQ9RETETERTxFtEYwRqhHJEegSBxImEkUSZBKEEqMSwxLjEwMTIxNDE2MTgxOkE8UT5RQGFCcUSRRqFIsUrRTOFPAVEhU0FVYVeBWbFb0V4BYDFiYWSRZsFo8WshbWFvoXHRdBF2UXiReuF9IX9xgbGEAYZRiKGK8Y1Rj6GSAZRRlrGZEZtxndGgQaKhpRGncanhrFGuwbFBs7G2MbihuyG9ocAhwqHFIcexyjHMwc9R0eHUcdcB2ZHcMd7B4WHkAeah6UHr4e6R8THz4faR%2BUH78f6iAVIEEgbCCYIMQg8CEcIUghdSGhIc4h%2ByInIlUigiKvIt0jCiM4I2YjlCPCI/AkHyRNJHwkqyTaJQklOCVoJZclxyX3JicmVyaHJrcm6CcYJ0kneierJ9woDSg/KHEooijUKQYpOClrKZ0p0CoCKjUqaCqbKs8rAis2K2krnSvRLAUsOSxuLKIs1y0MLUEtdi2rLeEuFi5MLoIuty7uLyQvWi%2BRL8cv/jA1MGwwpDDbMRIxSjGCMbox8jIqMmMymzLUMw0zRjN/M7gz8TQrNGU0njTYNRM1TTWHNcI1/TY3NnI2rjbpNyQ3YDecN9c4FDhQOIw4yDkFOUI5fzm8Ofk6Njp0OrI67zstO2s7qjvoPCc8ZTykPOM9Ij1hPaE94D4gPmA%2BoD7gPyE/YT%2BiP%2BJAI0BkQKZA50EpQWpBrEHuQjBCckK1QvdDOkN9Q8BEA0RHRIpEzkUSRVVFmkXeRiJGZ0arRvBHNUd7R8BIBUhLSJFI10kdSWNJqUnwSjdKfUrESwxLU0uaS%2BJMKkxyTLpNAk1KTZNN3E4lTm5Ot08AT0lPk0/dUCdQcVC7UQZRUFGbUeZSMVJ8UsdTE1NfU6pT9lRCVI9U21UoVXVVwlYPVlxWqVb3V0RXklfgWC9YfVjLWRpZaVm4WgdaVlqmWvVbRVuVW%2BVcNVyGXNZdJ114XcleGl5sXr1fD19hX7NgBWBXYKpg/GFPYaJh9WJJYpxi8GNDY5dj62RAZJRk6WU9ZZJl52Y9ZpJm6Gc9Z5Nn6Wg/aJZo7GlDaZpp8WpIap9q92tPa6dr/2xXbK9tCG1gbbluEm5rbsRvHm94b9FwK3CGcOBxOnGVcfByS3KmcwFzXXO4dBR0cHTMdSh1hXXhdj52m3b4d1Z3s3gReG54zHkqeYl553pGeqV7BHtje8J8IXyBfOF9QX2hfgF%2BYn7CfyN/hH/lgEeAqIEKgWuBzYIwgpKC9INXg7qEHYSAhOOFR4Wrhg6GcobXhzuHn4gEiGmIzokziZmJ/opkisqLMIuWi/yMY4zKjTGNmI3/jmaOzo82j56QBpBukNaRP5GokhGSepLjk02TtpQglIqU9JVflcmWNJaflwqXdZfgmEyYuJkkmZCZ/JpomtWbQpuvnByciZz3nWSd0p5Anq6fHZ%2BLn/qgaaDYoUehtqImopajBqN2o%2BakVqTHpTilqaYapoum/adup%2BCoUqjEqTepqaocqo%2BrAqt1q%2BmsXKzQrUStuK4trqGvFq%2BLsACwdbDqsWCx1rJLssKzOLOutCW0nLUTtYq2AbZ5tvC3aLfguFm40blKucK6O7q1uy67p7whvJu9Fb2Pvgq%2BhL7/v3q/9cBwwOzBZ8Hjwl/C28NYw9TEUcTOxUvFyMZGxsPHQce/yD3IvMk6ybnKOMq3yzbLtsw1zLXNNc21zjbOts83z7jQOdC60TzRvtI/0sHTRNPG1EnUy9VO1dHWVdbY11zX4Nhk2OjZbNnx2nba%2B9uA3AXcit0Q3ZbeHN6i3ynfr%2BA24L3hROHM4lPi2%2BNj4%2Bvkc%2BT85YTmDeaW5x/nqegy6LzpRunQ6lvq5etw6/vshu0R7ZzuKO6070DvzPBY8OXxcvH/8ozzGfOn9DT0wvVQ9d72bfb794r4Gfio%2BTj5x/pX%2Buf7d/wH/Jj9Kf26/kv%2B3P9t////7gAmQWRvYmUAZEAAAAABAwAVBAMGCg0AAAAAAAAAAAAAAAAAAAAA/9sAhAABAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAgICAgICAgICAgIDAwMDAwMDAwMDAQEBAQEBAQEBAQECAgECAgMDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwMDAwP/wgARCAGQAUADAREAAhEBAxEB/8QBBgAAAAYDAQEAAAAAAAAAAAAAAQIDBAUGAAcICQoBAAIDAQEBAQAAAAAAAAAAAAECAAMEBQYHCBAAAQMCBQMEAwACAgIDAAAAAQACAxEEECESBQYgMRMwQSIHQDIUIwgzFUIWUCQ1EQABAgMEBQgHBgQFAgcAAAABEQIAIQMxQRIEUWFxIgUQIPCBkbEyEzChwdFCUnLh8WIjMxSSokMGglMkNBWyc8Jjk7NUJSYSAAEBBwMDAgIKAwEAAAAAAAEAECAwESExAkBhA0ESIlEyEwRQcYGhsUJiciMzkcEUUhMBAAICAgEDBAMBAQEAAAAAAQARITFBUWHwcYGRobHBENHh8SAw/9oADAMBAwIRAxEAAAH28xXnEGAWJgMMwTJMByKMhhCyCxKsMwbqxCI5GSEUKlAcQu2JhHTLhhJFScByApIQACUjJMJKAUgsJoTwHU4QMGCYTkGAgAJmADGRgThhlGsVq0tWusqrKyG0qpOh3javWlq7May42hcsqVXaGkwEIcihGAwIuAlMLIUgFh2h1IQGgwkoJjAUYSYBIGLAq6TmqlfLPO3INeqom2LDKRnql4C5Btcq2rdR3Xpx%2Bmehdn2Wu2C4gyBIUHGmSFEEwhXASGJgrSYpNAJASAJkGBySJAQAnOmRvL/Hf5wNZGyxoSyIFioGdgPFZ0I9DOhHAXcpy%2BzfR5vc%2Bm5%2B4XBLJjQRAMAQIckAwgCAipJkGGGkKJggSEhJDTVHn3ibxzx7dR2yOtkRYqqw6l0bDgIMx1jgRcq9V5CsrQWFavfbqcnvLRcYRViJAiBIEhZBBKwKI3DKiDBgAwhIWEoCEWuA%2BYvOu8acu6qXKyuSOeNAFACOTwuEeWruNAk1TcqBaZrsc1k0SztT779Dmd0X2zDQYBhEErAZCLAMKwaIVgcgGTIQEKAwBjhOOs08EMHR1BoVo6OksJYscWBC/IBWfpbIK6qOzerGSSVkSIx0kay1MuRzfQN0OX3VfpdlRkUhLABmQgSRYygVSAZkIgFkBXhRXpisfP1zN/J2h0iqLMRWeCOBY3YpMpVLhTZ8%2BlxCiyIyTCPXbs7OKDI8UtylxGb6Venh6ntsdkmABoUTIQMKZHrFYcUARgYkBAGInzv8y/z9fSIZi4dK6qOqtr4WyFOhdND5L3qPZ82h2sjrq2Zpi7aKjpwQ1%2BZJ6pipkipTX1q%2BP6aOpVNSKNCyCAVmwKWSNDqqxSoSAsIQkByVXT81PN7MOxiLKxjKSx0l1zwdXa3M7e2%2Bd1bpTrmMupcsrEslVjKSr206p3czTXU41N38ml6%2Be8VCtUm6uWo%2Bh7bzfQ7W6kABsKlgyEpkWrKKSqCtCCJlUofETnW%2BUdWqEtkVoV1U9yw9bfXD9H0nxO9t6jdbat5w06lqqFFlfsEEeLiU6zPqrVTzB2PN6u6nBrujmxGjI3fO5sXvNsf0b9WlaARBaYpKQEEPHOACsRgQKQSNRfnr5%2BvgKX1LXSIs2Fzez1J5n1nSvI7OzqOhb6tM8bHZaSDuZZIsqygyPHgQ8WqMmnmzczdrz%2Bh%2Bv5jVXV47C/Io9GxBR9cPUyzTgxGAhCBhSsMrGEIIUBMqQNRhX8tXN3aavMLYLdi6PS3mvY9L8Dubep23KrdZ5bYnkra0pLH6LIuqqA9dsW9kVU0DUlPRNZ6sfKPY87zN3/L6s63CSbLZwfqc6fK6OtsCDDMhAgsEGpPGIoRiEkSWcd1L82uToVK%2Bteu/efA9T1r5j0%2B0Mu6/LpuVe6zWW2O6Tlla4eWUPQ6oIQNVthc98RWtSUVRs%2BndfM5d7fmeT/S%2BRovQ5l7pn0c7Od3nrmSDIBBSSgwIBgShUYE5CIfN7OfAtNNXsFwwdHrjynsugeR2bquy017LYNVja6yXJKOHckrJIxzLDAxy2Rea2NQQairCqg3YefOjwOHPXeJ1N1ONYon0Z34/QzZXhItAEKIDSvIcAKAnIUhFZ5p5D4RTTX2Oz%2BP3OwPK%2Bt3Bi6dqq22KrVbH12FnnbDMMq0kgzP63MYClglzep4ioRdIhDnollGlN3G4I9j4PS/a4Do1fQvZl9Nt1ZyAkCERCtK0hMoRgJIEiCTzapng0l8OW2/xPQ9f%2BV9btzL0LNVus66rMdFotsniZRg7KrG1UOKxdgzS9lntZ1VRNIi3rqa59VacHBfrfD80el8e4lfvhfk9VNYIyJGGEVJAytIMhRRU5AkTE8jc58Xza3S7e3nvU9f%2Ba9LsyjoWk67G2uxtbPS6chkAX7q4Z1aioY4Kta70UZupiq64Za69KNfjJxZ2fOcY%2B28AJy/QFdl9LehSBASGhGQ8NWQAClFTWGhbMPLPKfDR7xqu638l7TqHldS8LtcWXPzfdm1TS6ZCjS%2BNk4Udq7wlcK3DDJHwQyZ9QaufUkWx0yyZ7ND7OV5me5%2Bcs7cX0w7MXY%2BusYAhwQzQ4NSSCsbhQkExgJ5uIvgmt%2BUa/QfyHttz5NGidgpdul%2By1DTnm69O6OX0%2Bked2tgDVLnRLSwyVw5Tn3Tg5b6fCoejLuXPu2Nl0wLV2/HVsKoeTfufndV0cr6r9uTf%2BlFJAMJIeQwNSQ4A2igJkJJOIEX5uFvnM231G8j7EzWx6dW95uq1WyptXqnRj1Rrwba53Q6y5na2I224C1oi6ombj7o8dGyzZWfVea7r5LraoiTzQPN8xfXeK0Z0uL9D3K9T3r6Xw8/bnCEpmSKmU%2BuAsSETILBDJb5CeO%2BqeUXp/nlgx2%2Bn3m/WIi2mUduk2W6e6PI19t5lkxdC5Yera812ysu3prP0dxS6sVrzac%2BkNVGkO557Tnc8u6Ybc4fpeoOH6bfFVFkfHwt3fK8r9nnej/gvuvq39E%2BAdE9rzBJE5DAGhp6EsVAEqkrLV6NXznfO/wBF6P8AZfFbfno9IOF6WqLdoZ9vIfpPJ7G7fm%2Be3XZ0HWnD9hr3yvtLVUe0%2BV0%2Blzr1VnPJGyjTPV5rz2vzDljr8K%2BZn3nk7utvH%2B37v853tuNm5y7HB4R73A6l8T949/Pqn5gvOvAWEACwmIp1cTAREKYWQkXwq%2BefoXi/0PzLaGdO6%2BN3K3m2c/269RdnldA9zhci7ONb6dnoVyfTc7eV9eSYOh8PU6jHS1DXVrNc9M7Icei8jxRq4W/NAPxvU23zfX6Qw7r5KNXdLi8Q76%2Bvud6b24%2Bq/ncASwBCSEYKbXE5EZEgDEJiefnl/pnk7n035OV2Zyuw1ydNnGpmnLqrbCzTcqepcZatdzKVgq2xj7PRidigJRSNnIV6dtRo2M0od0ZLJns2fRZZkdu1VB6PH42t7Pf%2BrH6gfRfg%2BSBJgicIQU1AQFuUThTWGM0rzO54WeV%2Bw2/R43rbnbh5/am10pWI2JUtvS0MXRnqnPqqFL3WjqdCjq11KKW/Ont6PNVj1GnMaPMzOkvFDGzJSOtxOdc3uvQH0PzvvT23xwGISEkICYil1hMFAhJQSAskFXo8Gvn/AOiYPV5DsajHmD0Nky9KQaxuxWuTJCJRr/Oaxoz3ROnvRO5BrTWqMFia20aM7J7Dh5KllaNBJTEaOZR%2Bhxath9j3H7j5B1v6b56UksJQABMTS0rShbwEUIgpsFAPJzw32zmPMvUd3Djub6mdz9e1DVIszpQDSCrXXa4qfow7Nq7G5U7ka8qWbkWtbrlpsXYIpbiOkWiVyRV3KiNPKHJ3fTX6t8AuGrnFMTkLJgIAUhQkYioRESgLIkJofmen8yfJfQb1o8%2Bpg9TJ5%2BvbKtlnS%2BRMaPZA1V6Wt5aVufblHX29OqnW2uRzn9V2x73l7VQrvQFkdWkO2JndxAajb/Z5Hef0D47kmRSRiSYISSjqCSIgIyIgIrCQkE4e859B5g5u6Qy9V3k7dsTfY69c3WyTW1JqOfrObbbcuwquhvq3a1z6NfV5YWvVsktPWWmV0axElYk85LT57f8A6LxvX3rfBTd1IGYIWAkIGEkpSqlCiqokogILCQpggJzzzOvw/wCZ9%2B2x9%2Bep6Nlp3Ti6D1tDtj5s1ZaPqzONGLpjl%2BgK50kt/RGLVsmtJC7TLrSmREmmAON/u816c/S/i0vYpyRkAKQwsYqgrClhUyUUjeBFYjIkCQFOSmU3%2Bc3kPoNZyeilsfSsNG%2Bbp0mquJM1SubUGwRjc%2BTjbUy9GWo6W0KElEpLa7uwHaiIeiLt59s6fmfTL6L8jVkAwIRkAROQDCCU0oSFuAiCggQUpNEYQUNQfN/yHvdd5e%2Bvh6svRvkMO59Uzt6knRs1UnLiPZcktdUWLll1jGq8t6LXZ2r4md%2BLbfd8j6Ae5%2BanMCAABMyEkgEkC00ompRAQjIJGwBAUJCmJycQea9hoHl%2BkTxdN3k6Zsl0i5emnFsVG2xHeAtlY8hXmfzGEqjV0jXY700p24UtOPp71/znrT0nkRgwnICwmgLCWQplOFaZjdSkClAzWJqQkTMJJzzye3xb5r26Ofavzum0z212%2Bio9FKhp6kFd1lD1bVVqbq4vjmqOZfMXJ2NzF2JVolLw10YlNOTvb6B8i2tv5oNAkCEJBigIEJStOigY2EIC1BQUNxFYUzEhIKt/Nvxn0WAydFXDvXyaquG056GyndDo2de7Yq%2Bowr22Rg9NJRzrvyOTtHhcyw2yR01p3ZLP1PPekn0L5WowwEDMkLFCNggMpYafKywpCICILEFKcihhBCmIg8rcP0PIPmvZp5dbjFqQTUwvSsdDYw3ekt13esYvNAGTFPc/z9z5eKTzo0vocaKFLs3W3uPmnTve84UwIBJyTIMETMLJklRiJmEAShapEgQEFpkgKUIGynzo8h7rVXH7pM%2Bkad6MuRJIbmLbQmt/XVcac1rRpkF8qFszo6Mgaef6Q/SPk%2BwtWYJMkyQBBIAQWKUBYKeVwRKBCFBCQkoASYIoIgYWTjjz3oeT%2BD6lDDvXo2q1anJ2Sw0TU0vU2SaqNWZcVv4pjUS7Ix286t4T6sfWPkVv05gBPIcnComBDgBZCSUeV4ImSjIghCFMAkhpAEMZknHXnPQ8P%2Be9LO2M5ydCTz9GVq6dmml9Ztkxqm1j2qgleVmK2NuOH1cpjTbE4j6yfXvj1p050gDmKGGMCE0mAFaEMowRSBNYnAjCkrIgkIEDIRkLJyT5v0XDfmPWRF6We%2Bt9k68rn6tir6NkXa9Oh4pfrXi5Wz4IjVyYfHbI59EUtPqZ9Y%2BRXrfhJIMhgpoQaYJkgGFkosrUMBQnCjC2UpSFEPIpInJgnHvm/SceeX9ZDM0HZTN3ydTovKOrJ1bZF7pJWf11uTkbvzmGO%2Bfoufo0JKe5vofzPpb0XngaJyDArAQwJAkCQslMiFgLImCgGREKAUlEStI7oRkTwt4r2GnuX224tiSIdkG5XVtr43PFukkeVqaXzs5y32SuyTW8qGIGeW2cvpb6L88unS525pLdZRhGApEBCUTIKUaymHEJCipSITUkkyHy85XWh67WPnu/DcTrPrSY2pkwpBGB5Y6DyCXTyaJesP1tdJJhLpCtk0EPEi0rWsqk/Tea9F/b%2BK3TpzwYQrJkJIExAkpZrGQxAKUTEYwJJ62PbT5deZ9Dzp5f0s3y9y6h9ecLKksnLWXOCXiWTaWzCO/MXVnSXP1tfVVIKIxbGC1L1NE2J7B/avi9wsEcpryAIgMicJIacaQETkICDhdrJMGYvR5G5u4vX8qfmP0eyCld1WdiuqzuURBmcS17LJWq2ZgfxnCuolj1HMlaALRLArpItu3vVeU9TfpHzdGFGGuJY1qkPIMqFxTFpOSzZscuLEko1hjytynkbVP4y/Kvp1Q5ep6VWsjckbC1DrmPpa4jyYslZa9UKoVEZ8WxIYBJbFKqo29PU36x8j3R0cCMiAeuIyIEJW7cqmUpllEcpcOX7SStV8ZZQz6xSsUqm5v4Hb8p/mn0ayFHbVEjkiRrWty0jHfSx4LJJ7HKR0lkgA5QLqxlUzOiaPQL6L8z6t9HwEpEzGYMOpaQwtVhCG4WiWUtiH8kpYTusiZaIZJg3MjyF6j5u%2BB9ryd5X107KyR8esgeHud3HkEsdSx8I6rL1XkFZeQ4JmRG1O%2Bve/NuvvSeeIVRkThbAQMtZwRlJF4IGtJmZBnbSSeOba5t3nRLESDLFQtFApZ5e/NvoOjeF3xhXiuAzOwxdt8nU74M9CvVkxU7wWqEtoJrpc305%2BifMtw9PmgQhIVSnA0JrRkm4i62ikKxH//aAAgBAQABBQJtNOSoqKgWWOSoFQLJUCosqZIuCPZ0xemhrToqXFrEx4csgmn5ZLLCioFkslRUVFkqKioFRN/X0KoYuOTiA2S6ZC3cuc8a2tXv3RwS3Mv37xt8V7/sJtFnDe/7G3sjtu/2AJWxfdFhdTbXzix3FR7s107Z2PaHOchWgPplN/X0PdagpbhrE6%2Bhpyv7V2fjkfKfvvkm8Gfk24zp%2B4SPY67dV10XO8hemXBYGXOowbvuMLtt%2ByuZbYzh33018uyco2ze4fIhT1Gfr6BTnBomuQByPldtt0PNvt/%2BuPcN0ku5/wChjg%2B4JRmCEjnBpKDhRqYAgQtbKNlBPGeVb1xh31Z9p2fMmxvBI9Nv6%2Bg45Xl62Jn2R9x7fsC5Dyzdd%2BunyOkMj2UBQQ8SBgVGuXjNf8rV/Q8L%2BpqZcISIXMtdu3C5srz6r%2B1rPlFprqi40Hot/XrdI1qutxhgZ9rfZku1xbhfSXE5mAT3venAlaNOGkKrqNcmEaHheNpBic1CFyq6JRvbI3U5h27drvarr6m%2BzW8x26OQuVcvfrb%2BvRVVT3qZ7WDn3NYdi27kW/T7revc5ypnrCcaqg1BNaKUFWNqu7M6fMLU4KHQpyHODvEtTXtdVi2bkG47DffVH2zt3OLcOqG9vfpKb%2BvVIXMdyjfLPa9s5/zq55NfSyazpT1WiaarxhGFwGmiOsKOZNc8ta3UnteEwFxMY8Ty5B606S35gVB2ndL7Y9w%2Br/saD7C2Zpah1FN/VV6Zv1%2B5%2BeTb/vMrjMaNYC58jmt0rwF58Yq2Ffyykf8AVyvTdql0s2SZyi2Oer9inU2z3GobY4G4tQpbWVqc14THvYYx5U/JzCF9Tcyk4dyq1ljntm5dbf16KrsvtLlzNn45fTSzS10h7y85BMQaXq026V6seOyyq14pUM41C1sfGRp/9ZbSLZWtfBtPxm2Zj23HHo1ececDc7b45Li0KfbgOit3apnkrOjXam/6%2B83PJeMjrb%2BvS6gH%2Bxu/3J3aQ1fM4k6wSwFxtNtfKdp4x5Dt3Ho2C222NrW2TWtjsiT/ABZ/yJu2CJfxfJ1lUS2FVcbTVbxxtkw3HaZbZ9xa0LGBrpnHyseAQP8AJ/r1vp2fnQz629umeSjPvvcv6ubPfoEpLjGxzjttl5ZNn2tgO32GlQwKKKibGmxhCGobAjBVCFtP5wpLUJ9rlcWYcN02hk43PYdBv9vdC%2B4YQ7WCGvqNk3J%2Byb/ZvbPbDqH69HZb1fR2G2ct3ebd98mdUAPcrS2qth2r47fatAtWBihao%2B0bE2JpTWFMiXiFGxAIsFHxp7ApIlcwBx3CwjfHvG2ASbpYuhfMPE6OhOclv9Ubod44H0lN7YVVU7t90bo7beD3js3uVuwyu2Tb9brC1EbLZorABWEKJQUcmiiCjAIYAjg5ucooXiomZVTwjTuti2Vm%2BbbrZuEPjc34yQEub/ry%2Bd/AOpvbEor/AGHnlbxy5OqR2brCKp4/ZAR20NGwtUIURKj7RErVm0prgFrWtOKc%2BikRcU8KdTsFeQwiJb%2B3TLIAVYOcV/rZKHcB6m/qjiV/sJGZ%2BJS0CA%2BW0xjXszaMgFBEFCohlFkmD5UTQaN71yY6qf2Tk5PU7Ki6jq3foC6De4tLntVtVrv9aQBxUdQ/Xoc7L/YNsv8A6hMalpJdsrav2pvxgFBE2qhArEAoWpiplRqHetEKIioLSnNqnNonMNXhTMC3SLVFyS10vm%2BL7eP5/wCsu5QybH2RKqhiP1r0OX%2Bxe43dvYTSPkc3J3HYHOdYRaW24qImKEVUSidRNkFRKoyCtQqCCu6FAKkHunNTwpG5zRq8jy5nYljbtgrB8T/rft8VtxU9%2Bj2B%2BOFcJXaWffdraniOkueCC/iNrrjhjLRAA1v/AGNvG6PebFQ3tu8MnY5MmDS2f5NmQlTZaJ0gBa5eUL%2BljVd7xZ24v%2BfbRbF32DZSNsuVbdeKdrXDktiJ7PcYvBdQtX0haTWn16OoHIoquE8flj%2B87KRnBIu8Hzn4paaLHXHA3duT3c7m2HI79SbBv9spH71ZK25xutk7bPsprzYb3b3ojuKoSqOQuRkIElzpG985Fgb/AOw9xuHRXO/bzJt/Br2RrPru3lF/9e3kTeO3%2B52btwh8kPKojBu8C%2BvLd1txDt1Dsjh3Tqr7fjvd24fCtsh8u4bTa%2BCy3Owffx2ezWG3kPt2CW4iImZHOL7ZbW4NzxpoPGf7rCS0vNYgk1CMZTk03O4d47vY33M8ey7ZA%2B0ura3VtvECt91jcm3sbxeW8d417Kt%2Bx7E2%2B8W1WnjX2v4rLbtwtd3sugofrXCuF5c/x2W9c85LuN5vUQg3bi8Am3aBlIpX6VPdEG932K2F7zF0QHPXK15O67c3eZonMuAtr3Oq2%2BTyNt4AWX50DdL0A7jujLVu4bxK0T8nsddryKF0mz8nErdh5GzcBbTa1oX2xYFbFbRSXcu8MsNv%2Bqhcf%2Bjo9A/U9G9Wz77ZpLq422/5R/l3DhFvq3kNyvPiN5vjFFf7peTv2/gEp2plvtYtuQ2HDorjfOC3XHbKzj%2BFu90E/HJPJHDpbFvsuiK8ZJO68sbr%2Bjm3D7fZOH8c33juw73wKxZdcy%2BwvrOyt7rj019Hd7XqcyNmX2TYefZ9gtiy%2B8ZuLvjFj/1nHen2PQDpP2jx87TzPdrLTfcDtq7mQQrpmpu67U%2B5cNpk22bZeQbVp5Bwt1vccL4bbsvt05fZ34ZYXz9xvbZgm418Y4XM8G/Crba0Z4/493sH3u%2Buk2u74RDJdcWn49wh26b1uHIZdm429bdt3gBj0jlMHm2jZrBg3Hi%2Bzjct7yHV7YVVcPvPZ/K3kAEVx9e/5L51K%2BMSJ1mFd7TE9XfGxO9vE26YNhbbmKBzTcl%2Bl8Wp%2B0fF1sf8W6Zljcnw%2BYS7YSW7LC6RnHLZrrXYY4zbWTYx4xQtod9aTtlpZnV9Z2%2BvkeHtj7FE9H2Hatu%2BFcht9dzwRgh3F6i7BoI8AI/iYv5I1/KxOjaFcMAbootr/a0f8L8VeG/NsGpaKFoATWwJrWIBqoAnZjeP/wA%2BODyH6ssmDd%2Br2R6N0tBuG179ZzLjlz/Nu8xo5js2OBFMgM9FVoT2ANugCpdK25tFa00XoCb3t8l42uYYgmsXYaU4pxW9jXZ/zxsj%2Bp7JzLbp9vYoquFU11DzraH7NyLd9ubbOe4SsDjWGSqjkQbVCNUqpySrk0Qd5JLJudmDouWlSfFWtC2NoWgFFmhe5Tn0VfldN8r3262Ha4tl2jpOJR6PsDjI5DtW4Q%2BaHanOk2YqB2UZUbqqoRKlOlu6XVZLRlX2jW1s49TbiNXLSDYykOjdqQoWk5uonKU0ciP8nB9r/wC23qvonE4VIPP%2BEzRybBO2aN9QYHKN6iemuyermga2ku6sIjft11HXb7mAK/lhBkkYVFJ/9qKWgZNUaqjXk96c5M/bZ/r%2Bfcn2Fla7dadfseupC5ns1l4LkUew0TXmkLyFrcU52U3ybuevbdw3jdNwjZx3lbr18G7aVuW%2Bi2trHd%2BQ3dzteqVzCmPK1GjjqUo%2BLu9u3XJaRiK19Qo4HDfrf%2Bjatwj0vrRzHqJ6Ey11Ts1eWkdwLjjrA5/HQY7eG/a6DavJHBszC%2B3iZEtLaAhqEiL8pHIjPZ4jLfClMK41x9kUUcThI0SM3iExXTgmlMcmyIFA5UBRYHGK1YVJbx6oWxlmiiaAF/4ONFrVfi7JDM8Qtjc783qr0HEo4hVXOrX%2BfeHZGiaSvIKslXlTZKprird8blJHKS13jYyQPAcAteT5VqTap6aF9c2tbr0fZHF2BwK%2BwNv81i8UwzTqq4vW2wn5VZwqXm7NUnJ/O7/tm0suX7jBF/3hmcOStt1Bz50Cg5/YTKy3Rt8I81/4uKAXDbIWWxdFcK9BRxOJVFf2bb%2Bz3K2ktLooFfEq6tGTrcOJ7dKmccsGqDim2zOZwWzenfX0Kg%2BvrAD/ANM2uFHj9kTtnGbWFzbRsQiPx1GgW02Um5bi1rI2dVVXD2RRR6CqYc/2Fzi5Vog9F4Ck%2BSltgS0vgdFvWkDdmPLt3j0/0OmFpbaTHkv/AB7IPqquX15tOmNVVfROBR65YYp4uVbG/ZNzKCc2qIWko2/kT9p1hu0OCi2uSM28BYI9SFUE5VUk3gj4mCON9NMKYFe2B6KquJ7/AGufBK4hzexbmtJQiUcICawKOEFfygr%2BcgRxmtEGgh5VQrl%2BpceGjYemvQeg9WrH7bZW3jldA6oeIymOCj7xtqgGhMYmhqABLmsTgnuT30ReXKXtsGex4VVesYnA41wqqlfbB1WsrE2SS3dDMyQaqGKRRXFE2drg2aiE6jnBRlFXPU04ahVxpRPFVxeUTcd7qiy9Qo4hFUOH2s/4EVU0So6J0V2Hpr02WiF1k26TLlC4QnUl1RNDnljFSgkFD9ZbqLraOqqqq4e1cCj0PlbE0b/thQ3ewU2/7ZEzmu/x75uFKJzVJEjEm%2BVibM5NuE2cpssij8rkwGjWZxMog2iOYmW07pebPdwc9unqHlly8bLfO3KDsq41VVVA/HCmBGM1sZILu7mZcf8AZStEu53UkbM1RDNOanMz0ZiOqELVHbhMhTIwg2iaEwIVTipAqKhIj3B8A4A4XGySw5HLpCH654VRRKDSTFCAqBfZu1ybNv7JDMfZq9m4PaqJrU0UUYTGoBUJTRRD4ioo7NPWhNCvfJ4tg2huybGVLCHDQQq4HAHKqqqp2SZpIhiqmx0WhfaXHTvvGLR4LSmhURQKdmtOYamhRprU1rV71QJRciV3KIoPrbZv%2B%2B5d3wKcPlK2oLiEHqqBRonSJjnosygZ8YGqionsa8c02CTi/Km9mr2pVELVRNdVNXZNObXZB9FrQKBwrk2i0qRs9xNwXiUfE9mwcn/80qc0qlFXPWaajUNJUbTXTQWwUQzoj27r7O4eeU7Daz%2BQNQCIphIM2EtTXOQcmkprq4BNQqcHZhquJhEvrzgB2GLF5oHtUjSiEQmNLnyfrRMaE0BFwoz4qPvoRaiKEGi%2B1eAvsZ7aQTN7Ki0qRmRZnHRUCa1acGtKDSgg1OTTJJL9e/XZ2Z9MThO6gdK5yoVJ2j%2BIc74glDUow5OblG3UASC35BzU4VwyLec8MHG9xD2lDNCioCnx5adKa2qFGpuaYxMag0KoT5aDY9g5ByufiXC9r4lbI5YFdzIVI3yOfFRmj4uqXFvx/9oACAECAAEFAvVHTn0D/wCCqtYRkaF5V5F5FrQeq4e35ZkatZcq50VFQLJUGIctYQd%2BU54WolAGmaphTppi11Ex%2BpV/GKkkAOnUhQdOeFcKVVMqY9lG8P8Axa0T3qiAqqAYV6aIdPYrsmO1j8Nz8hUoU9SuFcOyCzq1wI/BcaA/M0p01w7qqIwrhVA549sAaJrg4dXb0nuXZd1lhWiqtS1ryheULyozITtXlC8iDkHYURywoh8Xeu86WtwGNaJ0oT58zOV5SjKV5HLUUXISEISUTZkyRCRalqQxjd68pqUFSiKdLRS3Cc%2BqqtSrgVVVRwBTJS1Mk1IOVUOyHZpz6Kehlg79%2B6ARKlflLIqqpVeunQx1DHKmOqmldkO/Zw7egej37kIlPepZU7NHGhVAqYUoqKi05YtJBhkUb03NFOTf09R36twdkppKJ5qsllgKdFEcT0gqN6jK9j2i/X1JO3spCpnZ9A7YVw91mjgcQVCaqHMNTlDm31HoIqVSLt6HZHDNHo9lE7OI5IqLt6j/ANQipin98aZU6aY0VEQqZ0QCardyaiofVkqgipyn1wotKoFQKipj3VMdKIWlUWlUVq7NicofVfnge1wc%2B6otKoVTD3qqddCUI3U8RRYUQoq6o/1d2h/X06p36ns/tJmaIRr4BVaqMcvACnwOaqEHoqmML023oNLGgyBGUoSJwaU05wZxu7N7en2RpRSn4HuMkXVRNFUrXRNmIQmqpaE4ZrPBs1F5lqeUSq5al7hWjhof2O5saWOD2%2Bk46Wi6mc%2BN2ptwaMxEdUy2c5fwJ1poWgotKIou3Q1lTDAHJtonWgpNaFqkYWEVGFm7OapbL8Vt4paelKCYo82wfrcn4r2b3ijamUUfzdMA11zACiaF2YPdd1%2BqtmGQs0sZEY9LDVz36VcBqqgrd1HzP%2BNyoGeOD05YzHcxuoJ3Jy1IPILZCVHcNBLmVMkIMt7Dqnm1mM5HvSqCfWsFx41HehC4jX9bWKWRz1I6uAKZ3ldVsUfln9TcG0MZqJe7jg2qD6ITGhlWqq06jpaFkESvZMpq06gQ4LW4LzOTnknBqYpXUG3Ctz6l23VbxFPNTJhXAZoZltFqATnr2OeNSE2RawjREDCuLBlIajaW/wCT1Ht1MYSMHhFHAKqqicKI4UVaKuYctSqiQjgO9dLCtrbSPp9%2BueMxzAmrwisx0nEhDByCbkjTD3wYpf1pVQxiGL0qVxuofLGjm1ypgcSmjGmBCpmK4Zo5YtUy22HyS%2Bvc25Bb2cM8sCiUMMgEQhROwKFcBh7pgTNtMpYxsbfwLhgoRh7I4VQ%2BQj0KVgVEGZv0YA516YxmMh%2BDIKsPdwR7lHAJrqIPXkReETVakc8NWGeAUArJ%2BG/Jx7OwJ6AV36aLtgMArMap/wAO5FJU6qK7mmFFpK0FZoMJRaRicQgrBmf4d02rT3cE7um/JNhJQt14gEY141oC0NKMAT4HJ1B0NVszRD%2BG5uoPBBKcEU0kKKc08jlVy%2BS%2BS0lUIWohTTHAIDC2Z5H/AIt1Gjg4UVCgSE2WgEoWsAa6ouReE%2BXEZIZoKyiDW/i9xcReN7k9dsalalqReVrqq1xomqoY2H/h/Gvu5anBEY91RHvSqLUcQEKKR9VD/wAP424dg5FqIVMaKmNKLMoIBOdVFQZw4D8Tcv1TXUOTkWqnVRBtVQNT3VxtjW3/ABtx7YBfsi1UWlUKoVpTWqoaia9G2Sa7f8LstQWoKoAu5xK/Gq1LJZLJVanOAWtHpguHW8kN5FKA9qa6v4Bb8aouop7kkasR26Ca9FUcsa4Ne6Mw3tTCdTfWGFyfE6WYuRWWB9MqnS1pcWM8bEQuyr0HooqIIBUV7D5YK9IwrhkjicCuyOBXdbXb67jE9dEF7IBBeyvIDDPgMBjXDvh26ssI2Oe61gbbxdHt6NOi/tfPDjXH36DjlgF2WmpsrIW4VMCqemencLMtIwzQwCIxyrn00QBebKxEC79J6h0kIU6CARd2vhkOFDjmV26aYw281wra1itm4dukBEY//9oACAEDAAEFAvwqfiH8DJEgIvC1uVHrS9eKReOQLMYZ/niIuWhjca4VXdGMFeJoTmuag4H8oLRqQ0jCuFaquNEMM06IOWbD%2BO1peqNHVlhRUOFVVd17vaHg/A9eXrBmpZAVVeiizQ6aYZKicA4UMZ/D7oANRKywPR740XZBFfqssHsEgq5p6B6taJjNIPTkqKiocKFaVpVFTLAhDNUT2NeG6vwCom1XfDus0AqLQvHkLcr%2BdC2Qt1/MSjbFGGiLEWFFuFCjhIMvUONNR9ugNqo4HFMtk2BCJCFeNeOhEa8YKMQT4QU%2BChdFROYtCOFFSh9aIVIxAUcNVFbBqbGmtQatOYAVFQL20rSiwJ0LXCaCieyiIonYFSjP1oqackTVNCgj1FkSDMmtVEEOvJUT2qaJSRaS8L2PaQVj7%2Bq7sPizCNlVDHpDEEMa4Vwqhh7YObVTRqWOicKYBN9WlXHBgVvGgE3ALvhlgT0VQ7YO7vbVTtCkGARHy9Rv74RtqohQBNTUMDgF3wogqIIjA93KdqmCKCd%2B/qN/ZBQKIZBBAIdZQxOHuc1O2onCODv26z0tpqTVbqNNQw98%2Bj2RVUOg4PFVdNon4Orq9RndNVuECmoKuNPQqtSqiUUVdt%2BMgQTv29Rpogm5mAUY1ArWF5AvIEHrv0UwyqiVrARuGoTheZqJym%2BTZxpeO7u/qD9lF3bk2uleRxWiVw8Mi/yNIuC1R3IJZJUVQx7KSUNTrlxQMj021X8zU%2B1Q1MRV2AJW/t29UfsVBm9AaixgTVpCLap0LSjbZx1amuQK74Ep8Rc4QsQoEKFZIqQChCv2/ONrnKPj109kjHxP9JrS9w2u1bDcQ%2BF9r%2Byb3YjJRSX7Y1/2iZfCRB4VU0pmaOWLjQT3XjUm4uTNwkrBuLHJkrXh6Iyv2VFlpD7aPyR7yQdy9KBwZPM347j/AMlrTUAgETRtxLK4yZKukRueVZXTmnShkWFd07C4PjbNqkdL5EQooDMrbzMCorpuplrH/l2148N1J5rn09vuBcbXcx1khamhBtVoBbk11zZVQguAGWlw5W%2B2ThaWRiTvGgFJWgIUkbJmTbW4F23Tpm3GsUYiEQIRVFOMrWIa5ZTaWXqcenGmeH5MFFGE1qcFpXiqfAmMpg/vQkxDB6cmGiyIcwLxNQjVMHKZWcVRv3%2BPbvU2x%2Bi/mi1iRmh0Sag1aVTCqqa6cwFEFRSLStNMKKjlQrQiE5SZvtGaVyaT/D6kT/HKNJbeNziTEFpqtCotK04xKieMAFpNdK0oNVE5PTflNC4lcjkrc%2BrYzf02U3zbCVHmgqYe9FpT1VRBBPXvjRAYOUqg/wCZj9Ku7h11c%2Brtd7/LO8IfGWIppTTjRFOWQMSrREinuMQEMkU5SlWgqt7uRBbevtu4NLbj4zQlNcmnAYOKdSr1GUSVX41zCGa0oZYez1OSn7z4IpJJJX/gW8jtUJTSmHAHByIT2PpDMahwT3EpsfxpToqFqTnq5d8T3/BYaPhKY5NQKbXEUoS0JzdSa0qPJakeglEp7lemkP4du6rWFNKBTTREouWpVqgghWoctSBxJTinLc30t/w7F1Y2piaVqWpVwY2qDaLS1BoTm5mqrnrWpOKcnLdZK/iWT9LwmlBFEppTCyjHgJspp/gcvJRPKeGkuBCzQKKJT8ldyeS4/Da7S6F4e1tE12IDCmQRFRxxNDWQrxW6JhCPjT2Rp724BEoq9m8UX4tlNpLUCgUO1EyYhRTMcg0FUotGcsjGJ82vAqqJTiAtwm8kn4tSFaz%2BVgzTSmnCgQamulC8kyc%2BVZnpKd8lN/zfjWCY5akMkDVNQQVEeglOKc4oCim/5fxtv/bSVWia5NKagEwYEItxc6iLyUxqKm/5vxtu/YJzaoEtLH1TSgQmnAZpyKe5OdqTW4FXI03H4Nenbu6oiEKtTJE16D6LyLXRGVOlCJc9NGJW5R6Lj8PSVpK0uVrC6JuFMKIOIXkcvI5a3IFxWnCiCKormBlwya0khOkpzafgDvRBtVDbgIYnClFRBqA6c8ey9nxteJrGilycqevbt8rYoQ1UCAVMKdA9R5DWudrdgfVCspPFPpTuoYBUVOs47rcBkGAxp6A6bSYT259AIKip1HspJGRMuJjcS%2BoUOmxuf5pTTp7Lv10RxKJDRe3pun9Z6ih1bdeI5dA6q9T3hjb6%2Bdcn1ivbGmAzVldeVnv6txdQWrbm6lundVen/9oACAECAgY/AtDVUW7p%2Bgal2/0Bu5X6CspBykErfUSBqp5Xckg2qs26qyYNVvEMKQuyT11Jyz0%2Bukmpso/ZGDPE10lG7qjKqjZwKtmFTQybvBqrqhVYE1OOTEuqtuy7bNClEDQGUaXZQaMvRl1KJbUCukJh2hXbZbLGLk7dl2zjzYNdVFtmANMb7Xb0btHDhlFDlH9nzBMUPyW7KaCojA%2BkGUIKysiwUQjlpZVbqzJqaqHrqqqyYR7saKeJcEUsLN1VXVGmao27CjSr02SZ2jjyKxzxPiYeRNgu/vkFNFmzZMtReK3d3ZQTVWWoqtyC7eiyXFPeHyY9ZL9TJN2U8rI9AhWikDRHLD3Kqm7%2BlSxCJyush0Rn/Wp4GYZJCrBK64sNonLj0nRAIOCV1LMEBd2HJRTz5QvGan%2BZFTYFSyl2URGXEyWGE1PLKipZoLOHH9UDd3j5R9SHq7IOlUdpeFx/a/s9ybO1ZJbOVcmquF7PP0xi547Mmwt3V1JVZspvVuqP8ufqY2Y6XUlSFeCW4hS9VhxjoI0x7wwVc3i2qt0WgL4hHhjFm34vEKdQijB3ZRyoZJz4nNn/AB7IY4YyxGhmMYG68pqhni3xgBDQ2WTawLKpg4DfR1RR9WF6kMUtpD6GFZTgBZ56QZejKKTAzyVluyoVnN3MfXSHEohyfVVZdSZuvcpzUpu446b4gDbNrdl2XV1s98TTbKXRm711dl3cslx/Vp%2BJT6Ld3dGAVw/t0/F6raFNuzeH9o0/F9baQpyc4T%2BnT8UTdXq52fmxOmnNU9oclKL34KYNWFpjSU0ccTRViSe7sSgMqFd3roa2K2ZOPs4MQPIlYYDoNCZXFYIZOBu34uQ8cPx0eQ/KWXgbQhjiPIoYC/XRzx/sx0BqyoQAHl0XxM/7T92l%2BNxChvoRjiKr4vL/AHfg%2BY5mjliP42zeLCy7JcWPj69FLEeXUxP/2gAIAQMCBj8C%2Bl6spxlVXVllKSqZfQHnkqMs2yqrIyop3CpqrqoVArPS7aOVVLoY5nUUHiqKsP1acchRSytpr0Ug/ZWVmUeOPRdmf2aTtF1Jy713LO1FV25rfQ7ruPuZNWcvAq5dkiuzO%2BgmV35N2csyas7QKrm63YM5VGgGP5W1cs2UmWVLIt3VQ6EURHORZVlGTIUgIhorOTQMc/W2TBGsizdp9ZKcYYzh2hVc7eqlKDdzANmpmJtBLDoA9SMSHMoo0YcMlt9EFlPoOrZqaujFk2sSrhqpTcIRYYuIYEFMrxQEpL3K0KioKL3VV14mq7ckWY0RihgcE2S6qqpdVerZWUgyrCGTUsRUruPLiM/RZcfIJZiHjgOpXblxTz9UceiD1UAqsFXqmS8SpoTPkqXZJDJY5dUCvmZbQ%2BHLKwyC7x7SpKbnbhdHrkrTRMrL4eftZJ0o5Zo4g0WH/tTx/uH3rt5eMhFhWy5Cei%2BY5fXIxOHI%2B4CX%2BESiXDPHxRPy5XblwqnCu7M4hY4g2QQbupd0sl3Ycyplip8uYku3jCvRUYVjSi%2Bc5P0RfnPlT%2B4f7WR6QLNBbVslNGjgYAVzAdTiIvy25kqQqulTcq5gEJrh4h15Pwi8WfpkGTQ02K8jRcPF6Yz/AMxuLkxP8mND9kCSm0NMHI%2Bizzy9oXL8xl1P3Rjhkf4eS/8AoqazCuyUCravFnJkv%2BfHL%2BTk/DQf83zOcsvyn/RQPqg0P0cq4WFfB%2BWw/l9SjycuU8zoQCaO1c8bryxkWSxXl7oGWjxMGTLq8HkO2kxL20bIeukA9HJu1Cm/RvHx6Q4nq9dVZLopnGq8BIKqKu9yHppBkEDOjtVUKuE17VNUxX9a9qPaGWbmZ103w8naKS7SrqymvetmXc%2BH0GmBF1M%2B5%2Bma9696qXCwBcv16fkjbqXVnL%2B7T8rgZVbMLk1JTZy/uOn5fqbW0KTnMP1aflLmznuV2XVbO98vHLSWZZGdy2bJuWgdmSkRRWQloqLuyvoZslkFPGyOPpodwt0dBdzLM2AWWR6nQiftK2hXhjix92X4aKaxP5xdWi1ePJmaBZcmWj8v68qFbQJK0E5ZZeIUsRLiFtKPluY06aHvzMsV2YU4R9%2BnHHmfMR/5MvL0U8/b0ET/2gAIAQEBBj8CbK4XRYIsi6Losi7ks5LosiwRYNcWcg0nVdCuQajf2RhZ7vWZCCAMTtQxjXM7sTwg3BW922JOmPh3S7YWSjccDcd2XvbGju2rD2kBQQf4r%2B0ciJ6oBQSl74sizksiz1RYIs5LIu7IsHI36R3ej9vJbs1wtq9AmmCcxUZQa297gF7b4/1XFqGWBkKtQF4W84KWOp2w7/8AQ08y5gPmBmSzKD6RhGNIeaFU0mMeG/mUanmPVd4U0vTTDP27v3VZyjyqWVFMTbixVajqhInu7Yb5fCzRYPCKFbxXb73OmqQ53EslmH1Cgo0qdTDSpqd4VHIbBNZzip5mYDGidMeYLfkFRzGq5NKAxSOWzuWd5yDys0adFxem8wb3jS5ZxTp1WOoOerac/NpV8JVab2jRdAex7agvwnv0JrjF1ro1dYjsjT6Rv0t7vR6dEeOmHaCSetwAVIU4XCf5ynBITwgoYdUrVmBzThpuXdeU%2BFiq5y6AsVqHDG0OHZV%2BJprgOr52o2zET%2Bnl9QFkVPNzlarUeDPF4r978Wu%2BJDCdJn/1GUDfClFRujTuwPzV2jT1WQDjFl0oXqtTabJJANh%2BHeBKbRp1wtLN1WSa1rxPAnhLWqjS2ySSgUG8WrV6TKjX4Kj8bA9pl5ak4C26GUv7opijUOGmzN0VdimrquaoAI%2B2U%2ByMWSz1DNpMeWZvkrZGdlgiYLQ5ExBDf2RoPSfpG/S3u9EpPZ7IFqHXgJ1NJnFd1epQo0KROP8AOtS5z0DS3WIflOCPaCScVZajGMbYcHhfVJOxqaYdVztTMZ6sZ46rpMP4GNRjWTgI2Quu7LDG6G0wNUI4uJXSg7oIRZ6SdiwDgA09MUWJZpd64G4bLQ5PUYP6g7PXCi6cif8ApWFVuLV6xcVgNqMc1fmcEOwgG2MfD83UNF4LquSeUYfx0nmYg8OzFQZfi1Nga3KueFruajSaOMrVY1nXDi26R26paI26/V6Nv0t7vQnp2Q6o9yAP8prQpLn/ACBAkV%2BF5D/X8SINOoP6VB17H1Q5yNboE4q189mS4VHEijTxeSwXDeJxERvOA1GEdI7II0ok7J6LItBQ6IVyj1H1XRIlFtL7/wCGJBQbRiU%2BtIGHHPS4NX2R7VXXpSN5in6Vl1GJqyfyWwuFtVv4fH75RipPnJQ63Yb4A8PTSLYy2dy76tGtl6orMrUyWPY7/uNRF03xl8rxPM06XGGpSzClrWPLQ1jH4LWufpsdBGjsds1wkl6T9E36W9w9B3a9mmHPfjCXBpJJ0ASUwOHZLdztQVfJa0txZAOalTOVx8Vd9jGmKtWrULjUe5xc475LjPaTC3a5u622CCiAaPti07bPXfBFuu5IBs2WxZi1zXr3o/QpvT6paFj9JLLCkBQ6S2Wj3xKnUI2L3CPDP4QQZ61tESL2alxNd1RPCptNI2H8QkVieGqrvGu80azbB9SFJ9yxbq%2Bpt6jTFLO5J/lVqTt2o0lrkXwvwoYoZTOVWt4tk6XlPpHer1m05tqIv5r3MC7t0LI7odu/ELcU7JR1Rs9A3Y3nyCmVmk3aBGKo6y1LxeFko1RUzdSoW1sFVuSy9N/6TyxBVLw0tFQjRYIrZuvUxVapJILi5Ot1qwpkdPSyFXVq6zdEvu6oRx932R0IWLU2NUnUt0HCHL1dqSiajSm8DrtlG68npLdEeILPxA39kLhsImCty6YXERsLl0eFyiMTt67WR2QooBR4Q6SdkTZ5bpzmO35o/MbukyqU5j/E34TFodZP4m7RCiY74o8S4bma2UzWWe17KtFzmvYQ5ZtVHscLQYp5HONpZPjeUy7FpUS5jcy1UqVKbCUQ2oIAsKLoOHZHs55hv0ju5/4Zz0G3qipnc5mRRylCdSo7ETUe0OqYEHw02tUpsiq7E6jw6m537Si5y1HU1/XzLhIOd8oiZJWyVsKXFosmnq1wjWnDtHrgeY07b4tBbrtg4VYdJ9sTcENhu7bIwm0SmvR0AkodLeiQiTu09UoUtu06Nqwsgtsm29sHeVENl1y6Iu7EjFUxAEEjAhbfIi7rjcKj5S7GPsixDfvqOw3RiZZeG3Xy1aok72hye6N6xeg2Rlc/w/MuyuYy1elWo1GuIwvpuDm4g0jE3EJi8QzM1KVPLcaytKkOL0Kc2OrPVrK9HGh8nMYSdVkEi3R7PQN%2Blvdz0CYnWF1g1nUIr8DyVdzeEcNrVaDGMcRTzPlvOKsUTE19XTbhjDa0fzH7IvOnRsGqDJoF5cvqAsESCkxPF2yhEDJSkT9ixJhJ1O9ixL%2BBwTvkYElN6Wdd0FWodH3xIEJbohMNoVbk02JbHh3pIPLBUaWkaIwlm04GiyalqCCjcWHeRXA6UaZmeuCXtfSNjXgSIux%2BNvdHwuHzMmdMxG8g1oV64Vr2S0EetUhQMFbFNokx/wCIJ4X%2BowRMPsLdfdvRhco%2BU3h3ykGMhmX1Xfsc2mRz7MeCkaNV6te5hKONN6prIjL5ik8VqWYo0qzKjLHtqNXEmnVFkue36W8/iNHLVMOZqZarS8zwo9zSBTYVE8R3tQio%2Bu/HUeTiIli0CFITQ0W7InutsDV74tPTVFgQdRsunOB8I0GZ9UbrX4dhPfNIBNOX0ffALmqEsAsjwFbESxNIjfYyf4fUkEhk7zeANGyMOFRags%2Boa4c0sXQoRw3jPqg4mXkOlojF5YxT/NbI6lEPezrSUtYliBhwNN1I6gXNHcYnv36/tjp2ThGPT4tf3RvgKqE2O69sCeNt2nbrgWuDT5dRqruusTqXrinwnMVVzfBmNonEmN7aQ8v6kdhxdfPMN2N7ucTqjL8Hb/tqLW4mWE1BTWo9B4m46oA2Qna51/4QnhETKDQLevVGFvWdJ90Tdfol1WwHXDw/ZGOqxRaphjjSC/CE3QNJ/FE6fWRb9kSE09eyFI%2Bn7o%2BbUkJMe0QEAQ%2B2cWS9/wBsHX07ZRJuqLN2ay6aYxNajk12p3Q5lSm5otBQuYesTEEIG6L2fxHwmPCWkGRaidbTpjEEnItdJW3icYS04DNNB1aCILJDzAZndVyKx2iMplHO8tnFseX3ntAdUNE4GgF3jDmjqWFNpu27Oe36R3c5x/A938LVh7TVpuOW4blAWstbVzBqVcLj87WEQSsyTPQPtiXWVn1mMIEtOn7ICNWczohshilsEtuiG2SuQdpj2gQPbB90WW3R0sifdA02dkWRNnInVFi98OaW2KQbx74OJmnC/CBLQthjVpSJoi3ao%2BVw0X6/fDXWYXAjVNU1Rwni9F4p/s%2BJZLN%2BYbGMZWY9%2BJFkaZIvijXpuWlVpsqUngq1zHtDmlpvahgc5v0t7udxDOVt2nQy1R73SsTwhTMmOK8UzCB%2BazldyW4GhxZTp7KVJoEKOndZGrVDUBJPQx5hajQhU6vfDbNEu0wIF3uj29/J3wI1dPXG2LBB6fdHToeSYsjZDmuaq%2BGV2g7IdTOndNrQ02euHsc3CW3f%2BLZFm3r9kSMnXdNENpy8xtMtAPx4CUC3IDH9uZpy4xw%2BhQfvYt%2Bi3yjbrZz2/S3uHNIjiuF5Y%2Bq1rGAfHJZn5RbBvVxPUsydaxclqdNcKbBoMvvgE3EdOqGMQkCeFomdqXR4Ugd3J1RZA9cSizSIs0nsiUSi1eUgLG8D39cHdBLVmNDvhTRBek2nCDarUsnZOCx1s%2ByMDjuuNqzHykawY0oVxfyyutAjh5rIJ1LHKu8Q2QO7uDnt%2BlvdzqjS5ooGplaYFhNbzA%2BoS6f9Mpsg6Jjo6CFXSQETUIaE0dUBxGwXTsOuUaPbt5LeTsjXCcnTujquj1bIu6Xwsa4PfGmUGDrh7h%2Bm8bwF1u8PbGNoUG/VpGqE7IDW2uOGcpp2TSPKe/8AOpZxxbYpogYbtDue36W9w51ctm7/AJLJp/22lzqjiXfgakoKBbheF9kTSR6vtMNlf1/dDNA0XwOWzkWFTtjp0HJZBTonIixfyH1bIOlI7tUPN6d0EHwElPwn3GDpYdSpohWLcRpBFiRmVXzG1qbXNIn%2BZjqS%2BVqIvPbsb3c06bE1xkvJcrhWp161Ju%2BX5b9tXar23N8xJ2xc1oUoJdZEpwuky2aNkNFyhTAXaBCpOUT9cDkHtgiLRA1y64SUIemvZC6oldydJ8qGHLZD9huWH6yZok9kbPGNWmA5m82YJvCaBeEjO8Pp79RtY1HPcuNGsa0tU/CElzjDfpb3c1bwDsOoxwfIZar5dHO1czUzWCVQUslgZTp1qky5pdUk2y%2BCSW2pN2EnaChEaZLK/UOuGXpM/ZAHbHt5Onq5OnRI7jA0dOyJ9XvEHpfE9KCEHvjDBB6uTbybeRwuP2w%2BoNsh2mBU0Eqk1XvEBzXSBxasNhlcRFTPNoOpVs5nH/mFcLqYDQSxQs%2ByfPb9Le7mvcqI0lYznEqz/wDV5fP5ahSwTw0OINQueQf0waejdO2D%2BZuLIorkAs1QANPcb9M4D0v90NhTCYwT8ot7IwmtTaTaCQ3oY3XNK3giJGABs7dpjxW3RLr2RO3X74nEtkdcpxNJdDFvrs9sb9am1JkYhMCHNGYGNtyXDQRIyjdbsk5NR8NhbOGU3VRTqPIaMcgVCydYsFJ69UVt1UYe6KrPha6bdR%2B0xiB8K2fLrGmOAfuGlnm5cV6StCvZWNRwe5BKVmrnjY3u5rqeItxCThIsdaHdvbH9x%2Bc99V7amRzDXj9AeXmGNsKvbUAfNqpeFjdPxdYtnDGqfEG6SStsU5Wz90OqVXBjWAklxQSh1Hh48ulNrX3v0uUTGiUNZRoupsPifVc6l/MocQdMbz2qLG0jibp8UIRUUHESDUplyzmRJVhFdhb4mVRiLdYcUVeuG0s0y1AXMIBXSMTprGOi8PB7RK8XWxbakeI3Wd8XHpanIVKX7PZD6dLA/AoL3kYVEpkFZHtMYKb3by/g/wASD4Ya2kxxGMzwvcCTofsjHnK4pukjKbiZGZx3L3R%2BZnsw3R5ZGozLlsjHw/Nms5iSqE03losaXNVpLUkUj/i%2BL06jCn5FWpJU%2BFbDFVvzMcO0RVp6VTqJjRjADvX3x/bWWe3CcvwjKsLJy3AUTTGi7mmG7G9w5sumqP7iyLMtXZSp5Ku44abK4e%2Biw1mO8ylTx0wtM7pMT8XhE%2B%2BKDbg6Q7z1xQb%2BAdyw2gDgoEg1by4aOuAWUgXKFc/fdKaKbBsiQDTKLNtkFR2osOc6iHfTIjsj8lzmpYHAlPwzNgjfqY6eHCVJust0WQJ7InaboW0w60bYqNa5HGm4DbDH16u621vxOcriSim0mMZoeY/TUJ7tGqAGllNrU3WyAEHfsdhbrN/ZEnBUnFo1IekoAqbxaQ6m69jm2JE9EY0Rjt6y494nDWIcTns8to8Xi1%2BIJGToM4FXq0adDLsq5qtmG0qpLaVNrzSoNBkSDaYy3Eck/Hls03HTXxNIKVKbxdUpuCHnN%2BlvdzczmqqluVy9fMVS35aFN1TqUNjNZ3K8ar0qeJ2HhtB7TkW0XAj9vWyvhq0jSCOxWxmmNoNy9OtXq1qVJvgptqEu8tqzFNhkNUZZuh4XX8RTqhoRJNgpGodBBDn/AJhEmaUu7IxqyjT%2BIVqiFuhzUXG12qHrnXfh8rLYwbPCXPmLYa2hxfLeY5mJzM3l35YYsSBmJjqmFRqgU%2BI0PJxEBtem8VsvUWzDUbp1pGNhUajDW4umuG3%2B6F7Bp64fBavTRBKzNjR4iYBzuc/45rlc3LUQKuddSFr3mbaIhyUsy8BzC0Vsw/E9jTNXCx1TZGNuSzOCmPMqmjXzA8hi/wBOkxx3bN4rDa%2BUz9avUc5wHD80mOQe7/cLOWlsIA8VWSq0nBH0XSlUuE9CiLQTGnRGUzTGod%2BmTrkfXFCq7eeKgaVMmKJDUqxlqWWYDULA3/EJFfptMcNq5n9TMZnP12j5ab6wDANRwr185uxvdzeMZOl%2BrmuF5yjS%2Bt9B%2BEdZhzKjCBje2oATiY4OLX/U3SIb5PxNDibkLLNhjLhPCpsmt5jqjELrTq2Q/CizAUonWkeTlqbq2ar7mFhdUZ4pO0DUIz3G%2BOGpmK%2BWy%2BYrUciqUWmnTLwanznVGbOarlmZbl21cnT8qoWV6prUmvpeYwhtJKDnPnaWgXxw7N/2bns1n8s/h/D81m8pxGm9tfI8Sf8A7rhb6hAZmMJAUsJbvoCUjL8T4Sa2f4ZnKLMxneDZsio/JsrNDwMnX8WBhf4TOG1cm51bKP3S136uWef6VUfgMoFo0w3TIwJpLsgzhzmqpVI/bZdjcxxOphL8ZHlZKm9cDqn4pKgnGXzlNrszncxxHLDivEqpx5h7Kge7ymO/pZbzE3RqjhVfjX9s/wDNcOpU87leNZR9QY%2BINzbn4M3lDhpsy%2BaytEhtPEXNB3tUcH/Z0MFCvm8yamVd%2Ba1vD30q2OnVUI9rGlomEJj/AJbgRp8NzZDnupUUp0XvaFclP4SdSRQbnstVy%2BZYzy/PpmpUpZplv5uCoVcNYhiAgStbhSOlsPcP6SPXrh7UKOIeJbowqHayYYEfgNMVCo%2BKzC0WI4xwXIEYTQ4fQxD8T2%2BY71nnN2N7uaDHEqPlpQzGcOboJNaGcb5wTRvExV%2BKmDuG/Bo1RiSxrjsF3fCXJ0EGHNDCk7BeYxtpB29%2BcZiX4MPic22HZPNvVtQOa9tQENe1wwOY/HaCIeOEOpcT4a4l%2BWwVGNzdBpVMvXY8gP8AL%2BYWiMtm/wC5szQ4dwvKVadd2SY/91xDPVabsVOiaVIOFCkXN3lMxH7bKcF4lmKTRIVaTcsyqGBA3eKsYb9UZ3itXI/tW5176uab5rSx7iQKQp0GU2U6baLEAS62GeXeMR922GjZ32cjk8MOc5MS/lr7YrGhlMhVpZg43431Ve5VDiWqQ/WvZFfhfG%2BCszeTzNHyarMtUV3h3X0wZ42Ga6RGKjxYZfKy/wB9kzWr0mLeaNRja2HTKK9fKmrxPiObpNa7PV6ODBTup5Sk0FtNlS0zUxgbQcymSXYzuucGkYmNZcb52x5hWZD7C1SfG17Jt2EJHZb0u5MyDMBpXZCojGMeHFrsLiHJLWCkcKy2HCMxnMqE/wDLZUxu7WNhBY3dGxshzhsb3Dnf2/xtjLK3/GZp4ath87KrtxOEVS0L4TsOuHqJhnVyJFkKW2dLI/TBBlfiQrPFrgNYwt3gZOIVNJjWhBJ12wFxOPdr0RhVyFVXVrifTRDG7Pcsafb1RgdYSgG2A2zCT2iz1QRf2RYTpH3WwHGnPCRMaTpvgltGTvEllhFmww1Ka4ek4ErBd298LC6kTrMZsC9jk7IbUsuK%2BztjgrrfL/cVCfooOPbz27B3c7jzXNDnZajRz1Iovl1MpXY7zBrDSYVJu7JwW2Y2P7Q2O7ptgBNcKkTEfDFuiyEM52wQAAMPb7YuCQp6zfF9v3QLSekoXT1JG0pCiR74TDO1IG4JpIiY64TDvabjpjdaEvj2RoCRKK40hIawAIszcujrg1LTlOFVnSsa/MVGUB/KvPGwd3O4nkHWZ3IZvLzF9Si4N9cYm0/0y6jUT/MpHAW7ZRlab9wufhn%2BL4dvIITokBLI%2B%2BSXwqavui1dGuFIu7I1QEtA90A6U9ULfojxafvj2QgHV1%2B2LBCWfdF6C/poiVlh6zCi5NkGCLtEeUCfzHAHD4gPiPYIpU6FIFz95rpoJWknRHF8%2B6fm1MtkmVPn8gOq1f56g542N7ucDonHEstWZhymdzD%2BIZRwADTl8291RhAsHlVCWnZGXztFS6iW1mu%2Bgh91oIEUqzbKtOnVGx7A72xbAi2JAdcTMfLri3prhVhblSAYC2IkG%2B3sjWIB0zjbcsak0BbdMaunqggmXQ%2BqEEh0SDyUKaSeaq6A1rR696GMYr3OSm1LSXEAAbTGS4fTanl0muraXZioA6qT1y542N7hz/3OXavE%2BFNfVoN/%2BRlVDszltOIAY2awl8ClIsDUSzDpEZLHN1AVMs4/9h5a3%2BVOQQuyJQv37I0KIuvWd8CizxOKe%2BAtjSh6aYHZ641AWQ4Wcnl9a6o2Rr922AD067oUdsIJe2OlkStikt1FSdBe7RsZDMy9mLI8IPnPPw1M1/QorY7Cd52z0A2N7uU8xRaIrcd4JljVytTf4jkaILq1Cs4nHmcvRA38u/4gPCY4hlbHUqnmiVzt1xS44hOCIA6bY0wD90d8d%2Bn7ocklVfZbFcOKii0AbXE%2BwQUiZ0dsAFPt%2B%2BHFrrYUkQg6kgR7fZpg6o6uTtVfZyUuI8Tz4y/Dq1BjqWSyanO12j/OrOGDLNdqUxSyWSoMy2VpNRlNg7XOda97jaTM%2BgGwd3oFBISG8Qy%2BUoUK7n1GZh9Cm2kauNuNrqmEBXYm2xtjugJIy64E4lFv364K/bH7lD5FdG1CnhcLMWhVjzchUoqinzKXmCVxAIIWBRztP9nnBYbMtmJ%2BGiXTbUS4wFMPrPa%2BqU/Lo0/1KrrmrY1q2k2R5ma/bU8q4/oUmENoNVQ3znb1V4HbBqn/AA69cbOhWJchhb4tWGDWPXKMrTATy8vSZ1Bgt5NXPGwehzbL2tFYbaZXuh22UDbCwixt5JQWvaHKCCsYmNT8KS7IGFgDlBErCLCNYhtN7VuWzF1RiqzdoOjQIGNgDR8LQnrhGyQQYCJE7eZk6f8AmZiizteITZ1%2Bhb9I7vQvpn%2Box7P42ke2K9N0iyo5pCdXL0Tk6W%2B/lSUXJAKD7oDo6DkTmdOyNkZFqL5LjmDspDF3%2BiGwd3oq1REZmAys3XiG9/MsdcoGuEghZgT64Rdqx7ISCSWgLNZSgtbXpOf8vmN9t8I0GRWVkDFVpt2uAPVOFFRr23o5Y0xby9/Ln84R%2Bllm0WH8VZ28P4W%2BiGxvosvnGCeXcadQ/gfNvUq8sos64Lqrg1otcbIBFUOlPeCH1x%2BS9jdKkerRH5mavsxy1XzhWV9eIH7TDqTc5uSH5nluS1MLnq4RjrZgv1vqBOqYEbmbSVgqN7ljC/MU3jWRj7VjfqNxSk04j2NUmG1KYdgdIS6JAiUAcmWdhw1c2XZmppQnDS/kHr9ENjfRZnKv/rUnNGp9rD/FFWhUbhdSe5hGjl2Q5jgC03EKvujFTy7Wu%2BIBQHaLL4LHZWl/iHtKwG/t6bRh0LYFxW3mNwNYtuAkABCbFhrn41UWucfiSxdkE1cJRRMPc2dl6AwC2m198mzl3QMGXpl2um2dhmotWPMdSZjOrwk2pohrabQAIF3IqbIymUphfNrMafwsxDzHbA2G06Yw06bW02DQ1gwt9Q9ENg9H/wAtlmScgzQAsd851OgjkkkWRbthU6xAc0nrVdQ1xhwkH1r1iAr%2BpQJWzvWUYWbxJSeyRI1QGAItpjEd7bZ2R37Yu5dZu7oq8XqNtx5XKki26vVbqXd9GNje70dSjWYKlKq0sqMdY5ptirRE8u/87Kv%2Bai%2BwO/Eyw8vT1QnfGiJgGJYmlbiYXG8J29tsHeehPzTgboX19sAR1x7%2BVzzMgSH4ro4OHIHfsw4p%2BNznL6/RjY3u9JwOssnUs1Te3SGPafVjjE2a2e7VyEXRuouuyNZ0xMGejoYSNhmTGjpoSPDriyUT5CkLdCRwdtn/ANflrNbF9GNg9JwV1v5ucb/JTMWKw%2BJvtGuMbCrDfo0g6DCdFicfbErY3rboSXv6tMCVt90eqCktcCJdcWxoHfB1Rwd154Zkif8A0gvInoRdId3o9UcD1ZnN/wDtM5MTbD4mXGMTbb23iFC6OrktvhbIulGrbE464lB1yETWcadMT09FjgrgbOH0afXTWme70Y2Du9Jwaklj8xU7Q1vLibIjp2QA7dd/KYt5RHSXInZCN8UK9V6S5dsVeGveDX4fULmD4zlqxVU0Mf6MfS3u9AajrGje2R/uWiab0u%2BJZqn2wXfuWVNVPfPquin%2B3nQyrXU2n5nl0yNXLZFkbpUaHaI3gR/MPfFo9fuiUS9vJMnbya%2BY3N5Ks6jUG6S34mG1pFhEN8yuFN%2BAIuxJQoqsI1Ww%2BoX4jTLGmXxIS5UlL0A%2Blvdz5RVabXU3DZKK9Gs53mUa9ak4E3seR3RKoe0x5QqubTPiwyxaidEdfJp1czp3R4eyLjFgj3ckuSfJr5FaUMB28MPiFo%2B6KWctp5neYUuCtKaQohRb3iENvObsb3ehrZrARk%2BKOFek4DdbXwDzqROk2xob0t5OnXF8TjTyWck7z1cgjp0TlXoebdHlUG469VKVFt7qtU4GBL94xwjhbRPJcOy1GqTa6v5YdXcdtUnks5J8wbB3cye7tt7I3Rtc6/qug6z6hCx0sit5LVznDXfvsvpPlN3mS0iGkdl40tOtYEaOTTGnZHsj3cvu5J3c8Ro1xSzNRuLJf280Z2uSNx2ZKjLUlsxYppCnr5cOiZ9nL0nydkTMSC%2BqJS0p74nEtIjt5S1wxNcC1wNmF1soz2ULf9Dn3HiHDn4cLXMqn82mLvy6i9UevlPRYlyd0uWUWxO%2BPf8AZFvZzqWSyVJ1fO5p4pZei0KXPcUUgXCG5VQ/P5wtzHFK6Tq5hJNvlSVBzNvfBHJ0SJwkpNHdzJ9Oix9sE612jmeZk2j/AJbgxfnsgQBirNDfz8quiqwS1wjhhewltRjpOY9snNc0zDmuES5miMJ6auaImORT1a%2BS6LNsNY1rqlaq5tLL0KQL6tes%2BVOnTa1S573QOM8bYH8fzTV8hWvpcKpGbKFMzBzJHjdzNsMN%2BLv5dkLdohrhoHdyWxbyCUvfA5gMVv7x4FQXKVXOrcfyVIKcvU3QOIZam0fov/qtuthrmzBA9fJPs5NMd2yEbbyW2TlEl98dFi3kROzkMMyuWpVM1mqxw0qNFhqVKjjIANbNIbx/%2B4GMq8ec0jKZVRUy/B6Lr2SSpn6l7/hEhzcRhZeJvf3xZCpz9cM1Q3RdybOQwWODXNc0texwDmOaQjmuaZOaQYr57htMt4RnKhreSJsyVZ7lqMZ8lEuO6LoNkLyJHtiz28lkWQCnujpZF0WRK6cOocEyZNFrsNfiWZBo5DLacVUhar0%2BFix/px%2B64nVYP3nFKzQa1VyTZR/ycuDYB18zVytCOTG3vhQFlCiAI6XR/9oACAEBAwE/IRoP6xKw/AdPNSnT6HXURevoOvpDgPo8ylw%2Bhe/rADj6N/TqYOCjwbrcrq54PpFOD6fSHR9B3%2BZXRQdal39GZU6PA5/EQYeWj98zRUCt2HyQqKNml01Ri49IsaLXFlrp%2BIxReNgDyUSwAenEXlYArzuA8kICc2swwiY9wczoLJrmJmDsXBjWrR9y4dAdKmF4wH5gVtopSAaeGFAAC9GByt3k2blYUW4yDVj7SjJzSBnG3jEp054PFdzwHfB77idCr6JWGH0OJTp1pKdGOj%2BpTdfpETgK4wb%2B2ZT/AJPr95TdcuHX1jo4Oj8abjk2Gng%2Blzc%2Bz4fSev8Asvp%2B32j61jcL/c/38e87xz6%2Bk/Xt3O3z9v6nLvnqW%2B3rXvL3f4J/cs4zWFgCUdDGXjO1l%2BOyt74U4MvIaZ93ZY%2BG5RLWs4ODWugSiG6EzlXs7cMZHsaWKnP04dSudWLBRTzkQ9kHgFzbbS6vywUMx2DWvebiAPFUqKro%2B2sjSifHn2lgbQCzRHCBD/GUBssg0VlTWtgdbw36gYIigRVkAZMiCFawXHjPVXtmLYjGPCxu%2Bf681PrP168SuPV6n2%2BPM9HO9HiVy/36c/SevY9E5/fruJB%2BPX3nj50e35ufueiefWIefzjH1qePv7y7/H9T19t6nr39sT2dfhllOa4y6IkWewF3UNqytwjVpvLz1LDMas2srNzGAqLhjQB2mXVNm5mpYJ17NkbUmQr2iVSRuB3gBrWq55TjCZJer2j8JZAKNVvJpWtzH1TAqh2cFYRnFisDDjTStHmdgCFBxwNrDuLkAygrbLXY6YlyhYzk2pRnKC0xUfLlFm1GajesLEjxXfgrcVx9OSlQRZwzW%2BSDxyAHhUlZkRqmxUpHnKq%2BIe/jHrE8fH28T343zx3Kd%2BvbM69GM5%2Bs6z63P64nk9fM7x348H2m/wCtf8Jw15/2egVk6l4%2B3B9J675xPfGN/SePtL33wfeO98/X5nr7y/eK9Z%2BUPa4%2BFdatbjBtzEctZaaCr3Z26IHTfHYKXkCUp3D1j3nWGyTBSiklX65ew5qdGAO4XLNmNlxSEuD3jGajQW8vFZOIiskmgnF0W35gYILYY7sBLqOwam2bs7yaubbJQsBUxktF7gdMezReEeNmacGlMdLpm%2BIVtsrwj8mfaO4BpILktoyj/s96DL7KxT4d6iHA/jnCiAuftUrrIj6tS/NjnudWTUNlwRFU%2BYaw2mQqy1n4QQo7qvjh81HrnbV8cVH24u3f77nHrxPoTeuv37Tr13ln239L/E8MefXE%2B8PsZj2%2B879cz6fGJ6xHPrGofT13LrFeql37fTmZD/cnA5WUaBMX1VtpvN3vBOpO3klRGUOeRVETZW6X9h1MMhWsubulrGuInDDAm4dczRXqWHsU5RFXaoTt5uAqA5tFSs4IcJZBOcKwHAbzKljQcMt%2BTxmNgTYrUL26VXczaeKfvDFvEKn0AqVtSveNY3TJVXs/xhIDkM8KrF7W7Z0BOTtXAgLrxKioXeq0Nlm1b3BRWkS3dgQatjlA4rZfEHFFO107LExhaGzh9Jo36CsYcyoG8l%2B/y9R%2BKsK%2BZ0%2BQ/U3Pn1uXmvps956PnvPU%2Ba9e8apxsj/EML3w5/7L85baYes/efXfq519/nEW/p4ipf59/PzCM8%2B631WxxKWQ1ZIOV%2BiTe/Pgq35Ok7CsrlUoKy1Wu8uCCUy90HvoxZ5JYEA03vNHEcqk8emnb3TkCu7WvL4ibXgV9wzzLMlt8ll0EX3qApMlsbllTnMTq%2BPAUwDYF9xVmybHvZSkz3QQKe7vrIociZULxnuXiWVri8INi%2BDrcMl14DnQe%2B4lTDIWWJWjA83M2U9Sz6ttSYl4CuzgD2eDOv1EOV711tccZhr2PJ2zaOHUsKckcNjI4UtiLJqtXDR4PHrUfoG/Nta6I8V4z85lfv8AfXEoJgvn%2B3iJbPWskdcar3iyld3wT8%2Bsnides9kveJ69eY6f5zHtKbsKtaWGy0d3H6aDXmGPlMAS7zCmSvTZ6BkoXzl55CxN2uR227WPZuA1eXHj4QYqx4DSxmxyPvK6DdFXQZdOrGKbK4w8OPcStBenGLGx6gtpKCzRK3NW4iLCLu717lG%2BazALWjoJzYt9EfI6scOS6WcnzM2XUGEiLbc6blNrk8DdjWZfJmcLKxS6BsEOosKNEtavdi/aBpnB2Q25UlcDGfSAe9lj6IXvALH2vqHzE7AqmZ0diy3vsbrthznmocCVqLwBLQjEfegKzo2LXZxUpkVszV5Iu8VXzFYvF1lxYpfc%2BrHpvWoz9n17mvW5/vxWpVcvp4%2BJ%2BCei9O4PrmL%2B/Xy/xf2c%2B2fEvNevvuP%2B4mG4QNNNb4vVHlghBvXSAlWqW6wW4NDR27CqS3AOC6gI5jusZzVIBx1DQYTYXHfKu4hpH2F6pFNt96nhFotTvF2211cw6aaXD2Lyxcs90F8YblRtlUIvYbX8wbEtkYKxhGsKPeX0LMc2tlKuXPMvZyHaO/dUzMWNs0XkYnTBYqBT4Q1dFrWaiNMjoGDgsqOruIMNvu8JfBgM6gAhoXWRT2vkDOCf5q3Y6hrkJjAhrNuxqKwGNAyLsZ0B5V4nw5s5LkBpDpjNrizmuKPOXxKu5UcdYUeGDykC4Rs0Qza9iIBJajIR00cOZru/X4nrc9uv7xM849f5DF/X8zvGvmcvb09Tzc0e1Zx8%2BuI/8fqPqv8AJeV/f6%2BYufXEuzr810xWkzMDzLoLd6jwrtAoipDkVUzL1m1XmoXst7QtkTlTZ4%2B8ZwZ5S46f8k0yUGAQxz6uFqK1eBfoOcwK9JyBvttlchVig1NuKsvMprhuw5PsbIIWYBmQDx5FSkFcGwbw4OyYCpDQ%2Bu6Uz7RoFni1MHs5QZWhKrubFp0wlyiopSIHBCDW4VZ8VIBummhyrgnAmhL2eQzyWxLHPDc0ZA39I%2BBzbDy5aaZknYYTzYLB94%2BaAY6OMD74TggpQg5UJSEamEd%2Bpwl/TQ3gL3RQxB2FRmUhNxY9NfqhpRTouePdqo/s1M%2B%2Bp69VHH5/5OM1/v5xFsfb11Odf9gS8e8vfH/fxLu9/TiWceXX9Rw59v3FOXkPrG0X7wYdRpKqrAzOt1btuTeSHXnRGcfA0aIaGmZbB0jNcvcsAw1np7q9B1zF0HJdq0rRFuzaZ3b3bf2lKvXYMc4QGyOk0arnKI1AQyL%2BCYamG%2BTCxy8lumPySygIV20S42H5FwJhgaGpRdvoHnBjyJgVn70iUxljsmHCmjZplxlCsxlZcAKdYAUfV0wMbdBKM8MUF%2B05pvnLd5r3upmG6076Wpn6MRtaOdnuFimM9zMJjTOLwjhcNXE116NZebdeWmJsMlrsdJDI/MzNe7hmV%2BXsuk1rJVZ%2B1VQWtKlKNevHH8fPHn77n9XU/wBifP2%2BZ%2BnfqiLgx82kt/HvFro7%2BZeH2PX3i2fT1fcfL43e%2BL9phQsWaHj%2B4DKZOoV4Vln3m1NmTFvwAs2hoisKjGFZw0cKWQLnXfgv9ZS0cOmWK6IrgsslqW8awZ25lIzACaxsq3mIA4Xung2K%2BbgY%2B6FTjDp1MXzmyuDyxzHOcI6G%2BF7q4EDoluJazdVkIm7ajjs%2BxG3lDTZS6VTZmNDV6WhjAzTgWNBET/sKFUVVXrZXYRVfY4a5c7/IjR%2BG1gHD5dckZAWm45vcE0BvFg5P9SAWzr9tVjF%2BGGESOn6IBviBA8Aauz36E65AyO9Laz7x09lDbKLtxhsRiNAJbWyDQYfX3Pif18%2BJ%2Bv8AZ665/Mefb6/8jp9nX48z03pWpfnfH7l%2BfVbi7/uXefc/v6y7Ax6x5gdYneBrocYlUIZcVZsT2TF1Uwuijbe3HiLfqOS2/wBEIqMRtv4LwIpsfIGuBvmK6uEaHG2iVF8MAvQlcx5i8zXWLxTTfUrY3gv9tYhAK%2BOXznEwrQC6UuK9mMlXfFfZXSx15vA4cH1zBoooU6w1jln96%2B/3hQ0aDOb1kiDXG%2BTw93HuqxemndclRQNVBZrWS%2By4dqsK2cl8OoIMeRIVi13Dq8%2B8ZoX9nTh794%2BRiULatluFIJ0OWiSgnnnFCADfAjcsvRRiPIhMtuRZXR0yq7XBF25MHiaW31qs7mPW5%2BP39NzrEfXrMbz4tnx9HuPtLqvj9/Ui5%2BO%2B/mLz68T38cxTJdj9vbxEl6VapUTYr3YrCtOjleznhCY5G1o1fALoU13Mp2tcC175pl8bSvPOWt1BCMiYChdK80mHNA0e/hVfpDdF6zycXgY5U0zxfDPzB5ZrS2nZuZzBqy2IW3jU4puu5QBirwI0Dn6RLzWmbvh83ApDJyq0477igK1x/tMXmbFsKvNlNFwjgpB5M2V9EZf4vnNX4lttPY5/UZNLTYZOcQ4hyuMtvgqbDasqWVDZbRvcEc7UvDaDy7iXNtUukRs4sMfgILGdJ79JW%2BZBBDQFKnzRFLVuvQvsX33CizrwaZ69bmfTP%2Bcz/u/ecuOuYs%2Bvye0Rb1r1zH39X8R26/HjmW5H6%2BuIxIphLTWOI5k6libDE5ceazNPuF5ytg8n0IHBRYVlhjIub0lGkSsgHoNPkalWrOVze6nd4NimP8jM1br13joqXnaap4Gmcr9r1vMoHvwNZzvq4g9l9d7FuUKULhObx2XN8tWY1wtvKzUDoxyYrpqU82r6gzXMzDCzKt2G3e2bMGwtby%2B3eGKbsBB6eehxBzmfCWZx8w2y6dGvNjLFA5TN3fjZLxseV05cBFijI4NmDYz7wBKvrG0sZK3tiVywRpo2DjEFmBgFE3bqbPa1rYodFLDD%2BzXFLIzMMPvMWe0vNb9bn1nn29Zucf39veOn2Zb659H3nP8AX6i%2BvZlmcd%2BvaL77mVHCV/t/EMrUzWwr%2B0MpVqc6gwhaqnRL8oXQga7qItcNcGege928xFQ5pujWx5UIVVbAtY0%2BUEw6DiqycB1M/hVYMfpYb24Uyns8R3kbwovH6uYwUBOslaxD93LwY%2Bkt6ferrVmtyk2aKKwu9nNwCAvbTeQHpYBQxdXfLlz0R82/J1XDxUxLNLr8txjmm287rDnUqdc/Jv7EyYyLNGvNxthyZxV4/dR1h9O/F9Q0aoL6rheDTkmsCMMltJzhBFX3Z17O7HT1FaxiXg8L30uGVmtiZQLSzl%2BEl82cTP4v5P6h51943RX21L%2Bv9x0%2B3/Z9R8X2JdfTfeM/E/t%2BovPEv/vc5eeevrmZgkiUBTVFUjbMLtrXdSPvQabWP3UdFOLLPpYiOU53sPcZRWKnRdV2dwgd95%2BOYxTf0w3xvGpZAc8tfi6iA%2BWXOPNQKFNmuK8%2B0poKMPXvm4PA4dsc3zzE2rSK5rx3Dw4vXOvpdSllyvXmqwmtS1RVXhDdL3xFa5MXV5vJrqN8jvyYrBxCXumk5OLccxdasz1jjslzDmkDn/pgq1yYrONVqIG7NjdHVVWZwStbLttmyVe0o1lUOBseJZW7sBhZvLeMx%2BMA9tlx8QxooNlLJ0mppqa5%2B34nOt98/SY59fmXisenPOpzR61HTjv/AHxH67Qj%2BdRf3vcX%2B9fT8RR9a%2Bk0XD2EfatxmydG03vaUyY6YQx9ODIYEC0zsZUYF0LnsIk2%2BPBNWxRpoAZ8ZnOqBlKyXVR0AMbAVjS/EFgu8819KiCmjG3PNYCal3RRTq4QejY5a9uoXtuUrkHX1xAOztu07Y6nEV2OMWe%2BSFF1jeKM55c2mwBfLOzzpYpKuhxWm75z1KSmHIY%2Bua8QXtsw8GkrVVHWzmqZnK/5/jDz0ZrmUABysL1ps6qCWqosgON0mMfeVWiqA%2B4LphuK2pjzK7apilzTiLSmXfl3MeauyGSKe97XMt/Bt/eOpxc%2BvpLP49EVfGD24hn17S8csbj9BsR2/cX168Rvn/stNV7b%2BCC2paFXWF8nzxFMU3VKoLG47OxCsh1AKrL8mFI4JZBd9zb6Qlos2FGdYU1U1FoVWWuL8QQHWrG4146yvxXsUwDzhC7%2BMPKRJZVfGr7l8y7GjZnN9qHWnF6OsedzxN7sy3qqhBcM8NG8HEFxGOs2UzXU6db27zxjiU51Y68rWsRqSms%2Bdv3iwLMbZRvOOYhnq6t4avsxRMHFBrGjzzFrGbObKMDPuGtUY%2B837pxivf5Iq3Idd%2BFXmZc0b4MVWAOPvKFUcRwY5LyJ8MJgRYXkzP2DibprqrYDIN56Ir0fX6jv161PxVf89ppYOvXJOR8fOZ70/gR/UX43%2BYPnzTx%2BaIdmPPct3phWB7%2BkXRpfvHz2VSRc1WtbDSVaoozk1xjZZtPwm19LOdaPu5lRnXkPjuCdtW%2B3nqJXd0uzeOUKjEcGA1V3dniXtSlU2K84qLCJxdmU2VmasBabwPcu2EFAY%2BzeH3JeGBb0QyHN1cvS12/DF6JZMhVZ5M7oWtVDKqs/Dmu/vBravC89u2ljda246sD8I0cAMdOhV5YZZBzaxi9WtkbbC1qqr2A90zszYgaub7fKVEwYl36A7ZcDURBSlyul9suo9otbqxLaxhgN7onCW1nGCpmwL3cLrdanCoN0QhtXkJ0ObT0OvE3%2BPMSrnrn2zN1nnMXDzvX9S4/mvwu4uPWNRevmUwT2%2BOP9Jcm2PhEpTQ1oYlBGrFGbFwWU7MqthVlg98rq8GE7hBMgR7JbqWXArkO2gcVUa9LRCbc3zcHLSS4HF2muEPl7duOOHvSXuFlYQBz4ONv0mUNd707NAnjEP6Aee1hGxF9ExAW2q4XPEPnMcUYa3uQQNTDgbvHIjG5DRsPJ8sSwRS3Dhere59oVnDm3OyHiMyrGPPOUvC0Yije4HCaKi5/fRhuvB%2BeoqF2PGRdpnSiwby67Es4y4A0m9wfQig7pRjBqUskTJUsnzgQo4YqipZxvvFQHRxBncGNZlppoJTWc3GoZsmM5x1zlVTFQ%2BbMtNkjleXxNBrwcFb/E7uPr7f3G/V9Qcn931P0s%2B9z07yR/HM/1FOH613NqMO/3C8EU1nCrjhlJUZ4HILBmy3iaj8psNtby6lAGjldZDW0BU774eDzcwTRdDHfBCpwjBFUzPZwJaa8LSgo69pQHA2Pgewz54DOxdaps1LhYpa85thXcs20bgXYoW5AizdcToYHs82kicvsX7We8oLDQnYOVLv4jjwwWaeMDZiDY9zLB9rmTZjwe%2B8kPpfZNpSfIDHZCdmOTYSyHhBbaBmPfR8QS0VZFOO8PjniL1tGVY1k4IEFsXQMlZK3CUOH3my3ipgzbitO9fVjkZDus47OkFqwB7UXGWx5IVbaoVCOuF8vODRpbzI3mj7fwevpDD4vHXzOVRHJ/g/M9PxcU%2Bsoes/uW1155515h8mIqwdWpX3BPNr0sgbULqXMfFI2fd60o20QgryBdFq8LwIOKCr%2BqlTFn7nYz3UYvnyeJ31NZ3Nqow8TSulRxAmkMfLxLFQGqNtKOCKpc6xL50JAlQRUNqMyz2wE8VjGUMO41q3sEcJDbXLDebenUOm%2BByKs3AZ5pXPLpmU5aAdY%2B9Swm8mEOdHPzHEeRDY0UbtlVR08BgvA1fmFKhMTDbOBkBRjzCbjr8%2BW7kDubnH4FMEpdFMbjiBDJ4iw5VZkGllcFjkcXMilo%2Byz7wVAHsSl9hiOYoa1MAmtDEXU73VRTNIs6Y620tKQBgtSrXr53B69bmvXj8RxXvTOF%2BK/H1Ij0VUi%2BftqP589fae/jU6nbXpsiBVwfBfcLSo/UIjnq1OCHufeYhVYdgxk0KTFKtPca%2B1S7I1ZyP0LXuX/eRQprKEGXJL2Csmti/Xb4JW6flD9eFy4BUNGi0pw4NOAFMoke74m4BQm7El0mR5iqyhnADbHGKFpnc/OHKI%2Bxsaxdut0zLUcJxiqOb4liO6Xnpn5gpyoada7TkmaYcmN1jZMbAUzPe4Vmu0uCzcp7a149xcsJ3p13KzsF21FxbXsDNZ8YBcsjG8cIGD2Lug0rKvyuXsWkGhiCgqeQwZjbt0dVX9ohO1eAG2btz9JeEd7d9FoUxWKhRgwMXhWKCj5j1y3Fj55G/XxH1/3xK%2B/U2/N%2BvM984%2Bdwerx6yP6nsj54McVj9w%2BfrKqvXE8L1Zuy8/FRj3gozhAGg9yX9e3aCtdaV7wQ4jsb0ZdWpSUwOTHy7i1C93wZJerKXZdGC4N3Uj8oMoywCThTCkGsskUfiMKWHKsqtBdFlwFkPhG6yh3YObmEPh2AfDRh4SilJh8G2JyRs5inlxlVaXZK/pKN8o23XBqXQmsXd8aMypZUsKR43msxgC19NORzVxfjC69yCh7C1DelAwsUFFYnJZFG/mP5VSnCPdzEsc/BXZRzaqBY5D0UUlnQCUVGWx6GuzhFyiIP7OZxoC8XovUKFeOa%2BvglesEVd7PvqEahgWhMvitkFNVyjobZQaPMTRUAmsVRwUT%2B/wC5n4%2BviPt95n9/T8w0%2BzB9/wCtg/MX/KPdi9%2BsVPq14/7Kfb7/ANR/NfeZS84ENyzBQf0gOLAWcAWHkbIaACWw0Lr2yT7H68GO2oiWDajz1ZGmBYGaqvPvMLcN1YhnbDndzH7EtdgA3YQfBUGVTKg0MNNeOoIraxlb8l40dEZ6CtdPLCrmUuNllaG091HqFtccdK8sYVw0pquxeJnGgNrwSAWHYznTIbX4msW6d2IvZhLjRyuNcFOoPvPtOSuxkecR5QORFjprm9QpYt0Asp4P2IkkqVYKYyIO8ylgDYtd324tABTNGuazCS6qX4Ca1eZmWdRrSvBDVCiLg3kyw1AubkqxXto3hXtN9Oeq%2Bk/zm/6g%2Bz1/cvf2%2Bv4i9fKQ2nhdT6nP7L3KPisz%2BzzqLMvr8y7Mv6xG8hSwTEppnGGEgwt/JneMyt2IVyaDzxNkrXs1GNhlUmzlN3dyzOGXm9YOajbbuXWfx5iq6eft3TiIYodqd/hYC/KG8O0xxc2tBWlP4czrBwWItgeSZa7WzQr8ldwNLENqDNHK6ZQBZNHGNt3EGw2RKt2xMJGixQ7z4lYbQviYa4thQzq7C6Dq%2BG6jybvJF2/KU8Z4LBzTiqm1gLRnxXvOpePBp46zFmL2Lur7y7qUFrTSZ7H7RAqr8jWWu8Qga84qmrWrv1C3Qzsindorc3w%2BsT1ufP7mO5dscC%2Bqn2enpiN%2B3rmL%2B/XxF%2Bv04n%2By9%2BvRDCLCWXgTnWXhCoHfudV5yaNDjdm%2BHsm0OBHOHF4dTMZLq8%2BTPxDaoL0dOb7hbzFHfOupfnXPQ2K4auJFhd2rJ7ovHxC/Ry5CP1xO6AdUDnmphCuS358dSh8xM225VBfAk/L38Ra3gasOG92EuWZXwxofK5gVxmuFayQYWrc%2BW30TfWb1nWuIzsuqrHQrjcMbonR5XXxGMRsmDJsUsszFHYqfp4u5hH7xvBcAoqlYcHPzmCwNuQWF58DSvDSgARz5l9ZpdLWBW03QAxjE8TXq5bn1/mY/T1z8w39faO3SVx8S/dOTGkX%2BfWZfbD/cRWiL9fWKluPXiU65ZlwiVvDD0Czj08g8ajXGJd1LyAEVrXmlVvrCC6C768HXMvC2gHequ77mnarL68A%2BGXW5rmYfvzHyWJb5O/aAGDBvhxTtamIw%2BDNXsdTOXBrB9e5Y2dN9kz7yn2mFxjDX1hWVrJzxlOqi3VNr70eNxWsq2zhdPPiX2YqqfbEBr5LUGL%2BYYi4UrActO7moo0HN5/NwuyYYMUaU1mnMxbsq4bVMh1W9x1bkwb01cbsvrVpVzmQeOAn6sIYE6AXGvaBLw1nRYl8t/gTFet5nr7xN/rnU9ZmD12xetfiXgn7OTGoqPj1nMXHq6nu9VF7D7S/ejPqt5in18VnFTARz0XI8Wxa7oNnLhlgjbfvpl49q9thfLdHlztTO/wBQL6ymPIYc5CobTOjWTPjmc5nB7Y%2BC2WFMoZxwPZN5tY3l9r1LwauSLcrreImRiqON5P0nN2yOb5ojVlVNWscviIQY5xKT2MzK14Nda37lQWmT339eamR6OZbFvxiViqORu8ZiSgAtx3sVldwNjbzvxAu/Djz7/WL2NNuDk/eOFF8nFgtlq2uWd6P9h0AR7nBFdoLW6rGtwB0ZxhzFNrlVv%2B5/0/5P8nrz7zF/X1jUvjvz7zv0S8NzyfDcf%2BeuJyz65jf0caqLyeX15iedO/3H59fqUCVRT5hoEX30pu5lWtU4trTDhRBFkCpUeBZvPL9dQAr4q9070rmLjJSscZ%2BJqLxlyfZ8QWmMXR9y6iM/ldLPGFR8GLK4LyYdMSV1Vp9I9Z0XV5/BKUo/Yur9pb0lNX2KG/Eef29j7ag5Qby%2BdHhINrErHC74XqVouzRot%2Bz%2BZm3YD6nKOYygp3edEaUedKb89ahGbx82l6z2R9Ga8k%2BrmOwcLT5/XMq7UqFhEyHX6l2KXgW1YvMFKZhxjr3zuFf3zLz9O9zn3%2BvrMf7l%2B/HxHHWvrP2/0faYGPNx/N%2Bqj1neY%2Bv%2Bx9fWY%2B/94lnBo3BIoWG6T6QKrDs1kRl7NzTJlkcHXMsYx4c81ndSkJBSjVLz2w4KMus3KJqX9TdOmbdjFNOfoXWZ2Y6OVPe6l%2BxEe5cUuGG3ma4xqNWIZ3vERcoxPAJqOU6xHJXjOGnzDapWyxzMrohB7WAY9W7UlrKEoQTJHDbTYTWlOBXFg5O2WilC53nPjdMBy60aM68NTCYwb9tdwBYZcefriJ%2Byecd/MrUbS%2BsFbOZUZcXnoAjFa6z6Mz5/KX9Mw5rn1zL/AKj4%2B9Znr14nD%2B/1Nl/WGN%2BI2/f15jzH11O3q54YZ78fuYdev7qXjbAfLpnZYVe7afMalhsjGOGys6hQcZGcOGr5dzEapjjeszGqll91eec5hFZx6HzB4OC83zj7xAcHjGi9VKs6CtZ6Hn6xPTOCxZztjEGhCIRzbGk0y1GsrVeTRpSII2AUXXKznuFGAwAebptuHVUAStVWL7mUABy6zf0zEMj2fFHsSwpUsuddZxqDkaLTzfcRUzTdcYjK9X195gOuk93iAUN0C9NCl%2B0X8e0z69XPy85n13r3/cY/Zv8A5Oc%2BZe/s/LDi37vCOttxb5vnqUvO/bHx3F9/rz%2B45u3xj2q4teu4g53mF049fqCLYiVxg%2B4ILD4vC/Yk8XWuK37ypuuOdV37ysDf0t5PISlN8YfrjOpn7vtaA7OMGeVxft48Rsq6wZvPzrErIVZwZvFi/ECMLanwZvvMpcmzgG%2BlE53UFIe3WWJZVAq9vDfFTBJnN1m/%2BSq7GqHKDr7S7Ya%2BNSiZN%2BecS6jOWvXRA%2BxMZ7ztFRxb4hJqC42suMAmAVfzn/s43fH6uXmvXHvLl%2Bt%2BI49dRtz3jb9YvrUuXhV0/A14lk9r6xFx18xzt8eKvEH43X79o65737TvzHm9%2Bu4s/H4jnQZx9mUrc1oov0yOzw6bt8GJ0WUqzk0ecQRj9uLjWTS4Hh%2BIdYG%2BTf2lb20D4OsLLVrFta0eHF1DoYolS%2Bvapd5DFRquBZjCxS1Ld55KhOx%2BTHObU4ldECxVKvh5j268VxdeY%2BIi4fvZzKHOQe%2B%2BzddQDadY39/acGA%2BrXHvHRjL1u6%2BJzJWedMz%2BdWZob00vj/n/IvP1614JffxL%2B51mDWPMv3qO/n1zPXt3PPmeD2jwR1znH2IPR4j39vn9ROePmO3x%2BeoP3gvXrmGGGIQ93D6KAPmXGzm68efMOPHsD7YmVcqvxWNeYUne79jrykRY5ok2X0zEhBR9R5PUvQ3V%2Bmtr2IVCF2aNCqJiph7zLLaKHGZjozAbLqxGK3Wbl0cyORXwQly3TPAzBR4SAcqYuWfkDEOqiAIx0KWFWXD5YoHeafyhq/Avol5aOd6v23A2bx5vsJU7u9c5/2LDnZs0q4MsD9Z9vXUX5nT/wBxKaz6uCK8Ve9R6Zzxca%2Beefb7y339dwbLbzZ6CWvnlrM8JWEePW4504rrPzHYeuYGfEfbqCvn8/3LF2ufXnuGGVcdCKvBSIhQrdEyY1iLnHeOz5KuEW3WPMA6XGRXruVcZkYOSkdnuXjVaNUpVK20sGXdq4q7tFCvWQL55BcHg1iU4vlxcEZh33LJm%2BUMxhTF/VNTle4hd0XzFIldsuQu7w/3BJLe%2Ba1Etf4mPMIm6CyKY6lDnPd%2BIgFqKz6rOIBly%2BvZgHMLblrqzzLQZYilstaGspWAOhQhj11LxL9eu5%2BOvWoucnk/2c%2Bs3uWx148fmUe%2BsYfmLL9HwS%2B/tuNKxfhqcJ2f894t/r17Qlc/x0vmCXj0%2B8ySFpKKOOkM9x0oVX6aTzuD9nn6fSCbdKcb4945w4c6quep0Ydzqt/EZom3J715uW02KXRgpMBOMypY93QHM0pVrjqO2QvXVpAML4VzEdwwsQxMdmsyqHPg4rI4uWucFDgNV4ROYDelHgNvJFsY8sO3glvOg7rXHtUCl72fnXzMVJvrpx%2BeZlasxChyOwMvf%2BG06ff83L/79Zav1163LzxX7l78/WX/AEfmuLi1x9Jfx65%2BsEpHxez9TavrF1/l7nLWfX7jdfT1qPn/ACf5OPrN79eIjVdxPhDcJufFh0Pc4eGYjjo5TW4YQ8R1gc5x%2B/pFdYyXf4vsFYG5qtLQ1yepSZ3gpt%2BveYaLV2Z1j%2BvvKThSsAb9szJANsXXm%2BCUxqKsaotaHCWAmIX7Z3c%2BdgVuU9ocA08%2B2YTV4oV8Yuoq538udEBurKbvuA2bcGMmse8NAquxcoLRAgq1Wnazfq5j0%2BrzK5iZ%2B1/f4ue/wesy3U5ml/5M/c%2BPaB/GOq3BfZzbzSL3nqfnryfqKvXPUbf88R6lfwMsntF1Kzfn7S7yqUKmmGpn3iGrH0HtztBszwPWea9o27RwfcuurY6D1psGNgjf5ibPZDX0M1crQFDTF19ATqKreXirquJqQ0bCd8CAzG0r3Zn4WEm24ZFaMfWdEwTvzl3ECA3z%2Bh7mQXhrWq8V4gFM6xuIYaHKZfrzHLdAsxoefkuJQCGrh0eLuZ5hf%2Be0H59eSX0%2BvEv16qen9/SY/wBjO6z9vrDi%2Boa9PB9Y%2BthzG%2BtnrM5fqNzuPvuNxqi/VQ4VzjrxHjvmOmpYRYB4HCvTjMyq2OwNCYoB8z21VuunARdjKK2cr%2BkTjLHnFmfhCVQlOeeGodcGnC4q/vLkFLC%2BuE%2B8Kyyot4sg3TUXbX92mNVwwZdcr2Ps3MRo18upqHBd/wDZUUn08TKtA7/yAuOjen%2BQeGinOfZ3LLfdhmbr16YOXLx9M3ClY9cymLzwZ1ODfrgl%2BN8zv8eY/p/Mefa2N0nriepnDhlf5HKkHffq59PEX4/Evn/fmPmZHrMz/cfoolkEKtn5fr6ngadYPfXUWdA2rp4HpmTLyuf8z2MsRyjzq/b9S%2BWNPm3QywK4U%2BMX4KmZT%2Bzm71BKLwMbX7VOAadVj6cxluHDVV5mVMuX2dV7TJE4HLd8czPCiwWvW7rcGKqlfDweZYcPA1i%2BUN43COq53ibo%2BvrH45uG3q/aVjEMb/5/suv7fp%2BJfz58T3X0cy/v8x/zHvKfR4%2BYe39X/sXffpuWN/wXn1/2fn1zKWbHqpu0SzivPccX51Mdyt/gU8hDGdbv3PaWeseH2GZRxq4zwNc5jGe30NsPFZOMUY73mXrG2uehuMDf9fOJvF3VbzR/U5LK979mJ5DzTVvvisTlNjtfWa5mVkZdGi728y9Cwxii%2BBMGRsrDwPB9Inqv5wHPvMiGy/HzOQ03QjsuF%2BZ28bruf9%2BvtHKevVQxXnzuevXtH7e/rFz7pb8/9lq%2BMy38fb%2B4Pz/gxyi/78cGIvy/mJ75nv67imefH8XGFxHA21UBbr9hf5MZeGA5rj/kSNTGezlwaHLoJdpQPIw/GYhZn7/9Jb9n/r1Mi47v1iBk33xmZQ1/%2BT2gk%2BBvl0YjH%2BYw85rcbSDLw/boZnIz4OGOO4sRp/v68TjhbY3/AFUvrVnXzhgoUt%2Btcs9y%2BNn9sNDd%2Ba%2Bt%2BzAb%2BKO%2B9YqEeG1qwWWe8IkNpyyM12Ao%2BkwY0ppT62Zmp4Kja5AFq4mek9M/EWfb/nEp779HMej6ftNvXtA5foceJlcde89a9Yjwia/z8/wOM%2Bu5laFr6v2j/OjZssPN3L14WUF8HYKi9VspHFXzqpTPiWrLvkPqXfbbn1mXoeS/ChzLi/YrDi6r2qGmOce5%2BMTKwoYeH3pudFZultb68ZhNKvPOWqzwwRvueGMt3AJjqKrF38yijb5wc627lKrCuLeruaSOM2az4mUo7HBijxK00u/YfpUFLpnG/GJUAHnzaVg4mNrZwevMeXkEvDXBEZAI%2B578ktVaChx91mDYKXEWrMCxqZdp29%2BwllTQv19Jd/J8S4%2Bsd7ItP09fEX6gqn1GYPzmLp4jw5qejOH29EFhZbtmz2O98D9I5ir48YOerljXu6AYgEKfMP0coZtvXYLlaHXtfYQh5fy8GXW6cc48e1EQXk3n2v8AMy7Ke/fC9M8WPOavriU1Rzd1152Qb/N8XrNXFdjhsd/1Do64570watfj44YIFJa6v1pKnXt56J18drnPPNTtHQ4vbcs3DnvrHFYiD%2BN28faHDzjWNnb0My99njF%2BNRtL%2B3v7zkBnCiRBtRZ8alD589vlqyeO/FzJfkZT/ssIZovPvSTkYOs/Ef8APl5g7hi6fVeIyPWd7DcYzx1/UMjb7Cv2luHgmjzW2HIrugZ9kGacZLcvuZ5jdmclbweICmzZdcxigndlEtmfNGZWFPgDF8zLvXNGL%2BmJkvbeeebpiqsycmM%2B8sI6DvVYrx/Up1tVtLMZw9zJh3taccSlHn28a44mC945E%2B5tmjrAL3nNfERYNmOBVN1ZqNRnHnDXnjcFT0a17ZzDljY8ccY3OGXdb%2BmGUOHBssNbqYMb7e%2BMR8lNG/rXvEaQwFrnp13AFoXqytVh%2B0DB7Oj/AJGaHIW7ZZge1FU5LV9zb9MxDOMZ/qEqse0dqx%2BQv1Q/IYs%2B9nIxuvz4xf1J2FfGIrvN5a19fmaHu32/qI5AIHgQsrfw5/biXPl5bb99zun9hV3W5TwvTzf2xMOM7NW9/eFXJ9OuftDYACLAhdHEZoWCA7wjYo0pV23gv392aY3rtzvPDANQt3%2BTMMBNZG7t7RNinnv28xxF0nLrzXmAm%2BdsKre9k8HF/T0MG5av0%2BxncrCDPXFfE4Xurd/eFRyp8/bEPgXXTMWO2AaG3jU4Lc6O/TLbyjTtdHNRLitPbvuvmZGzwucjEDdGa/P9Rtheq0EWXZdER2BDWGW1n%2BKYA%2B35%2B0rn1n6weLT6n/IChe1XwjGcRBgq%2Bfb9xGcN31%2BN7lmYq7w7eqlw0KvVbt7zxKKi1HiytfSWu%2BeW6%2BJZWY0V%2B%2BZymK9MQts/n2harjZpaZO%2BYDqBRxYc%2B0Pe526r%2BouBXsb/ANlmF6737fEE2UEJm9TpHUlYo3VhTgRBKwzS1Xi6bK%2BJZi81dnJv2xM5fivt7XCy73136uXp0vTycdVMguW%2BvZdYguL3o4rD8TPnlzeddS5eM%2BkqIvvRq6w%2B9xUoLos%2BnxC0YLzblN565jx6G/LvcViBWg37nrEEeKrzzn7RFvu93jHFTcsBjt5qXlb1DA5UoCEO/Ykqk6pmrwan3yeusx/5j6x1qjHz3Dw7wHn3rxDKWwDZd4Kd1Klv%2B18RL66zt9vieLtnr7lRU9Epazk%2B0LTZpaziurjQ8vj%2BoQMhc78EGWWdeO8e0IDd98PGeo1Ol9fl9JQPOTjzwEpuvfeuPmpwPWHhma4b33KAbNemP4XoVsIuhoM8cXMsSBNYWdmoFDBjya8bjngP5dSidWldg5ww3hTms5/MEww/Ip1ziOgmDkTTi8c7hdVe%2Bb9/HE4KVgcscD8TC8Bi3nDiAoUZqrHli3Ex65F03fl9pwzLWbTzrEDTsSvdE1bb9McW8RVZeOchxl6hwqJKMjDzcEprCeUW2Be/dq/lu22/fOGOqPRq8R377%2BIms3WT/nmJvo1wHB9ZhWlTw3WsQxp%2BB9pZfJ7/AK4YM/ajNV6dQba1t31iLiGgM7KN/Mfis8f53Cwtc5OA92Ud8lf1ArTknvd2/EUdb369o4Lg%2BXX4Janx8e9eY9bL/tx7z%2Bx3jrcqwNvuc/6wvEocRdlSI4SV4bkqT/alxoYhIKuzNPxnMHtV3VxhquDj1UpUp%2Bz6zMgT%2BmcVLrYO85DzyTBadc7%2B25eKNO/%2B74ic8NlOnHjqZcDOenx3C7wcBVds1VzRvLF/r6xCsMPDXEylg3OMflYQFfMqwJhoGu6leEbrAvXvSd2ZT7895j7U8162y%2B/Hd9RB7HRvs%2BIU7nGQPW4NZB8DvOfaYiu5jGHWob4TlVnt8Qps4zX38w0THJ%2BX4hZFtCvt9xP/2gAIAQIDAT8hurzmHM24P48cT2meScXPEznOZfFS/MucJ5J%2BJqDF/wACrea/hQIbDPeUHzHEzXmaq2ErJ1BOJzU5SazzPnExPiYKn5jzjM43LepdcxqWhdfw7m/eLftPiY43CN65nBmOONyhwvtxPYJpgLieCoFdQ%2BaCuHEM5fmb8odzzPEq/aXuzM04m8/xx5lTuyH3hmfWe01uDxHohMx1bMYmskvxPjM6Z3K2GbmmvtFMWqUyxSSsKmzuZ0S1d/3OVYc0DjM8zgJ7zcuY%2Bf4Ce8%2BJs/jUtPeZJ77nOJmVU5nl1Na3/FlfE1C4tcT6SZS5Qxc85RuEXa5lDZco7l88MzoJZuAntEqYI1AbiAUss%2Bf46ox/GvecXxPMxPBOIZ5xO5zn%2BOqZe2XPHH8ONTWIjfERWpsLvqgyG4DExUqri0AGZ7INMZbqorlcy7R5bSnNMSj3IFvYIRdVzJfE83mezPie04/mye5Km/KcSsjPeD4l/wAYnmIyZWVYerh1bYmScieeZbqGJd%2BI8ZR3lHH3gvkufMGysiLYhKvxEyQpNSlCNMtFpL5mP49v4rP8Vi2F5z/DEJ5ifWeWfmX2R8RxVwjNAndgB4ZrSbMwrjU4zuX9YLyRVaNRvBHSc3mfEOAzJpshtmILhgZX3Fe9zHDTThpNkM3mP2mcS6Z2zdw9/wCH9xnfc5SofeMe5zLrEuWJ6EMXUuyiAHMV3lB7y3DEsGLj2RAsxcmMTE1BFuUMcR8anuVHFUy7xtEzg9RE4n8y3bddQ8TiNtfx7Sy6nux/FfWVmfme4TXMDFz8w7lj1IUfMtRnEKhCjzKhqdrjh4iBJttmarzK87gVlmewTFHOOpJZmCVEO4g5cSsZMSmzmNoNDBEP4alwwwScwx7zMNzmZ7hn2njifE7Qg5jmGtGZT1GJyljLkgCbjKqOxZZuD1cXLbHc6GJPcdTVuWFkJjGZ5R1uOhLAxEvJxEcjP8VfvH%2BCXPDKmpi8zxxKfZBy/wAPwJOMsJsS2TUYGoEGuhj527jxmHkjBui5aaVyyxcQh1DxHxEtwwqhoU/wK8JlI5EeoM5h0yzr%2BDtK0zc8xB3K%2BkHiL7nLqPEyVmIFtTuhGXcppcRcPEu5e4hccs7lubJvM83AOeoGkYu8xt3mHHMyGJeLmmluEfGhPOYfImEOysX/ABb2fx4meJ7nEDr%2BK7lsTLmeI9hOWYX3jBmUWSuXWXiJDnmLxC%2BYA4dwRqsRlpikrEzY1UBQO47dEwtU4qN75g4olouIeUsMSkBxDzYQ7V5Jn%2BeT%2BMfws8QqaZ%2BZcoc1CJiEw7zMgvEVLpPZdR2hC7gudQ1AymmIx3cF08xLdZjc8DMdlQ1VQ5gOMypsmbxMFsu6TM3vMX0YM/E17z3gTqczGe5hifSeGVcyF6hRBVam2mGoI/WKV5lBdsr6Tg3PbU5phTiUuNwqjeo4utxcgIsQnygozNc5l4iwlhe5oXmLWIMdKiLa5%2B00Wz2/gSezOpeZzuXmqlJjj%2BaNXGlTG%2Boqu4i1Nz5IWnmdruZq1g4xiGVph24g3eMQQ%2BIqfEck6QRTljljzUyYQ%2BJRC8ZhivMRm4wIcyq3/HT/ABncLHcY%2BJxnczHm2ZYX5JzoxMTEwPcZ8seam9uJ3iCjSa%2BJqKpa1Mcbm1sqU1vMbczOEaolsJUJ9SIE7godTUjw3uaDmVvGZXU4h7z4/jcviZucRqYNM0IJpklVpLM8xM3qZiSuk4kwgF3MciFnEAzkmE3KOW46XKrjEbyRp3UU0QyyRypMTXUabZURnWLZDIfea/k/nFsr6R57i1xOr3N8S4g0GJUUprHOqxC64xBvtK%2BJ2Ny7wIuVmWAuuIU6yxK5xL7l9wypE45lNFbmkIjpmPNGHekscSqRKnEv7IEuhmHn%2BMTc7zN45i/Wbu/45tlZjgT4k0TFoS9%2BDUHlh8szRyZfVYQ5Eo0cy0DHkCXdhuZ6zG6GpdOoU%2BZlGAsWZbwKwP4AvIio%2BBKA1kjASby5dauJxBslhzL%2Bv8adSsyzNxowEznv%2BFpqXATg6lVELbMW9aRIYCxgub%2BychKFNx3jmazzMo1iPLWZVcK90MwqSuIs5jVrM7NThC8G4KzGKGF%2Bm5V4Dlx9iWcAue88RnGJ7ypV6lczM1snDIv6Tnw6DFeZr2YliZ6m8xTeGJVl7Zv8w8vf%2BKOS19yhCJ7Y7DBvIxS5us5RfFMBcI9CSYDARKqHeCDv9Z7wl88ypsSrVFllEErb/Kcef/HvuVmDjcxedRepWFhq2dS9C4TE3LIXrMWMxUFPhyzDEfWBFhGTBcSyOH0hcBhqjq1HBjcNJgHcqw%2B%2BA1Kpux5cQdhVhUO1x1K1Hodkv8RNsU/Rge8mg5J4iP8AGYnMzKz5lm6zNs7l4mMy6Z5G/sZ7LTQIt9TbeJeTiWhYQh2xuY4R/aIfiW2m%2BsMBhy6xUtuhVCWHiWgsQwLz0%2Bs4TeGNKzUbTlyuJemPTct1BeLlUsC3H0XPIGX5/hnv/N5OomczyxrX8b4/h%2BrP7EyM0jo4b/gu8mpR7zASGEtz/EbC3KwTEUKjVnM2K1cXBW7gRZVERchqZhe40%2BZYHmI3dxJtuFqyEB4mXmAX2n4nc9tTbX8abYXDh8TBmUcTxWJvcC2WC/oylubOeEsXEwrhFEwwW2gsihq8RxlipVxzRZgizFbgtJLAHMzgpZvJLDxG8ZigWn0yh94cVCgkrqbP/qw%2B/wD4ual7qYzjM4ZY%2B0vFTole8siFV4%2BkqdfUFWdTJshdYZXSxS4rcXWKaKqAsHMPYwQzmLjZfE4rbiYZpBvcIruDzL1DYsxFI3GWVlYNFoPp/Ht/BRf8cJiDmcOJk%2BJjfESDSYjXc8PZg%2BQYD4VDsqNE1mfiFvU/MGdw3NFMyZjkmJeHOYL4nRgwqCNEoKxu6aiCvBLYmB3LL0BnCiecc9%2BYb8T2m/5vRz/FYnTNmZgzqW0dTl6l32p57JlSbKanMXY1E5giqbuAItEGsw9xMeYmKvNzucR2YiV4mKhoJxGuTMwcQFtwphdTNaxDzU2bqKX7Q/5/PmcE95U3K5v%2BOqI5alzliyp7R5YfuHkl6k5iKcThAyzz1EeIBnDcMLhgXaSniMp1Up7JWAZkXOMcTtwTTdxydrmMqxKxBsLhOtwNn%2BoE4GD%2BK5/hvGY1PaDqa3K5JWpfmF5ZX8bQntOfnMTmIb7jjFit%2BI63LLqPlCU3hC6PpQVP5yLMTEVqVFBWKhSZhRluYznEq6uHUL0allkB7GcfxmOa/i4dTMxf8VkqUXUz3PiY4hmp3cF7acc5yFIZBTmWLtxFijbOZ25IVvME3F9YUKawS7l02agPeWXfEuB6zNZ5IjludXLnvOoNxsqe87/8Y3HM/BMRqpt3KsrlA3Z8BC2YjnSpmmXMCGnEHFy7whjqNZnOp4iTu5tmHFTwgZqCVnTKfidP8e8p7lt/xWP4cT8zuU34j/F53Ne0amplhBA1m5o7mVvMeXU71KocYmWpyQTNZjSjKUmBTLR4gM6vc0hoDcN34gxD2lVPioEuuJ7R/h%2B88Tq3%2BH%2BO/wCK4jrz/NTccqPJifcTCqXzzBiNzDJmKQamcBKilkAswQ8ZF8QHGJRS8y7RSoM1AK6YNVuVDlli%2BMTzNfy/x1RLtlVv/wAVfvKn6/neoGsMVnDKZuZbZaInBMBtHNqpdbF6ounMx7jZYqsHWMmLebuYVMm5WQimGLz7TqtfxxvP8fmbP4XM3/48xj/5PH8NXypfN7jrG5kRkySleI2lDUzTHCZTJY5lS8IahwjJWYctpmNHtMC1LYEupLxMpROJ5qcUfx7zuOpXc1i5y/x%2BZrExHr%2BD%2BazHMQQTKK3LkgeNwWZjOIy8N6hbuUO8S1buZ5FMd0jYy20e%2BpUIt7SwN1M8ezPMzU/MN7ndMz8ziz%2BOZWfM%2BZ7fw4/j4mtxJc63/KrobuYqbTojVKu%2BpVELsEKOYpkJgxv%2BAdaqUGIVqXVCC3REuvFTH2H4l6hnMCa9oaua/lZ3/HJ/Cz8wviPtmdE/P8eP4xetmZi3OG6g74issRhcyauUpsiVg3EKuWFqIYCsw3DN6RsTUFCmYr3qqfMvc0mpia3PMPJ/Cz8/%2BKIzOZr%2BfzKuZZc2jhc6mBlZlqXOoockbtKxDqPLzLvJxOauGtkTliURRZOo3q8zN44ns8mDmXmfmOz%2BPj%2BHX8d9zX88fxsn5/n4x/GZSj3jM6iB6YV7pz3NYUWeKZsEV1PLG2DL5sH2xPM9pnT%2Bk6jN1H%2BFlteY3E6Zg9pjdzuOA/jPBiPEzMauVmpYLjgbx/AyQAX7O41HGYZL4hpAqgy0DUF6xE8I4pbYikaaivEzdm42Rr4gYzCq%2BHyQn5A6iLlm8vca3/L2nzPaXN1/HmbqVEnUZRzFQGR/hzTQgcbjmy5T3ReJjFS3c6uJl3HzF7lFUan1IDlERsndym4jnEO5EtD%2BCeJxX1hev4xMBOv45/8AHtHUBdwG43q5TUhcA1CvcNOEPvFqXWjMXxPDDkZqsy%2BLxHOY1slXvUcVREck0YllRxjcVyup5mI0A92dQg%2Bh/JTyxNPP/jlL1OZvUtNCty1wdxhgDgR%2BiGXxMag51qFVF0ZnvOQanKirUsXwYcl4/incrrcyfMztKW/wLdjC8HcfWK/lr6T7xMyj4g4qVtmeJrHMq8TmokKHmeErFlGJZOzuJcViknkaH%2BTVVK1ynKXMKOpuV5gqllM2qKntn5iJTCmYbBlPcwm1lo5KPHmZQHt2yr1GtEc1iYViUv5xjuVzMr4iZLlaqBhKgUzZncqI3nU96g/ZC6bmef48plyTUy4YJTYx67haXL4JRHE%2BI4PEMitSoZV1G6bIZYJw7nvk/wBD9ywqtT3Sr3/DSyVZrMxKvMxZGuIbzOTqGLltz3OYvEDU1HqPTE3Xn7mAdS%2BLnLMw4niUrUw3LxTK6cR0EeBgjRzkg7Fl3jiZu/EBuKSXwTmBenz/AATGv41A%2Bsobl8CISqDEoj3LcyrUgZmEga7/AJp1/DobgBox8gm/aCj%2BKCg1M4eYNHiWHmVBsxiLlcR6m7TwMSkuoEw7jZRtlaf3M4D9x/yc/wAP8K8yvpKHWoBnmBupw9RJ/9oACAEDAwE/IWXoSfGJm8Tvv%2BDAfxjmG4ubi58TO6mMx7JfE/E/MobnxGrn5/hrma0S9VuGs6nNfwaZedfxiceYkvfU8yuf4dzHUvGJzPzPNx6uVftMk%2BJeN/wXc13HlxP0m5S1WfzLPzYZ0p7S7SmNYKfEWIX%2BYi2yih%2BbMdmGGc8RnEPaXmd5nJCZlTPtFgxCcYjnUJz/AB%2BJ3N1Bv3nOJvLG8k67inLROLxN4NzjE6P2wuVYXVVUsIXxC2u/48nCUglI3WDo0/Eox%2Bw/uZlfwb8TV2TRuXmZ8XOjmX4nHmatnMq7ncKuprb/AB7an1l3/GKhzU%2BJqU6qcPT/AGZKAIMNsWmyG2ZjWYL2wwziWvE9iA3m4ruiKO9zLSddjlg6jcPaPGcTbuWztnPU6ntqe8/E0EzviYCduYku2uJ5ZsYPie87hx59bikoWtvpLvaK1M1ufmCHH8cMvipRvUWncqbsgsVhPBlILZbgR6rzE3l63/H4lnUep%2BZtzMXNpdTb/HbFqe38HM%2BMTzzN4j2RsvESg25eDr5gAwERWI9NypsKiNbh1qFOEw8yXeFRKvdR6jPDUDhftKTBmFsVGFxAjmOR1Mk66nUxnM/NTg7l8E9/5X6/w7wx0zfMyz5xDVVM8blNgtb/AKhBSdBA5OZzjcyywa1mVeJR5gPCBXMpSPKFzaNcJbDAtsI9j5lqr5lYxqVT3HJDY/TKg98zN1F4rM90yTtnzP6/jjcL3xMzrMNkajjnM7/aSg4Wpl7zBtNxe5hyzRjY6TGUe8K6xAKUxLWqSgOZY95WlGJWkj7YnRVSgYae0UfG8kp0U57nX8fP8YuDlucJ/Gdx5j1PiCUxzPEQZMRRvX7Ebx4nvubXFRbpKhf2hwgSMMuWJekYTSRQGVMEUg8RbQesUghzcQ3uN8OJZTX6wW6mq4zB/wCIP8H8ZZ4jiX/HTc6uYzRD2mmN/wAZYrvGY4FwUG8Rbg3qBvqPSeBw%2BSDQJDtEHK3cmYI4EcyVLrEUgEMMbliANM6FzHLE4NzaPZb9v4O55/moGGXvqZ6nVTWWbt4/hK0/xa2YsFOpgvaJkkxQhYwZS6lp5lCk4BO1TBhgYdw4VHKtxzdZlRTM5ZVxlmGXU49xw6mDTmIK0ynAuY1xPme8fE4f41M9z2mPj%2BPM4XmK1zBQmqallAlrDCm4FJZozCl5/gFfMKCFUVqLwGJWWaqohuIsmo2zBcNiYcMS4x/Gc0dzPTEuRp%2BGZdQaScTNeZzOCV3v%2BDVXMzVT8TuESbQu6H3WLhbzAlshiphgJlOLC/mXmWPaCb/gyuY5bi%2BkKRiULJWziOszMDUAa0l1Ki5R5mrSLBbRl/T%2BdVHE/EvmbKP4LOpu5RwiwYzB1iWIpq2zo1DdXuGOY7rucsTWOY5ahcofEvTOFgxwjuLqu40%2B82XCDFX4ndHDbTDlhFMZ%2BZ0Te/47qENsvMuYmOocW9Tb3LuMwRNsOLmGpTqHNsN9oNyppKsSuI0zFijcBmDGpklYqC9IdtTUJzEPMdUncpiJj%2BOf4yE4uYqn%2BHqY1/FT6u5ibZgpOpRcmVYnW8TmuYZhLxrM3N%2B0LTf8Wxe5hVyvENagstNbiaBChicpOoi2ag3fZC8ovifOf45uY/iydJep%2Bf5SvzG9rAr4gmAYmcJguELAm47fw1eswvM3FcxM2pJ0W5ojeYvmdmNls604%2BWH8wG%2BRGHZF/wDPCS8Q5lc8TOMT5/gWs4CPvBncr3qeLH8WQ09oOsyumWTeIG85jC6OoKHiWB5jszCkE3O/EDzKXDYeKuW7lzEJe03ohscfzp/gncqM6ub944uYnZlP4oByixmczKkTcL9w2DKfeDW1iyoHnEccQMRYi/MtvMbxOINplFb1MCrh1PkQUEzCLhOA7Z8YnPn%2BKnJO71M/WN9zU9/4a4hGB3uZXmZqTNQqNkfgoMAHmUkJ1ETriCBZRI4hyn6RAPeO5cWvcv8AaYsfKWZVHnecwNvhzO%2BHcTheKiAzbAqQ0heLf4Z8fyzAZZRX83vuVLKLi4C4bf8AGAWycGaUYi3EOBxBmAZKErlohX8WRV6jx1EzUJcEDxuGYs1kAAiAxWozgSguYBjrcxHzLKjkyfqy2%2B6T/wAXCX9P4b/ngND6tRIWjKu78dQsq8ILvIa1iGtJgGcQrZjbbZNMMcSvKjKd2YZdQcI%2BxE0rUa3L8w/JDMylajj/AGalfg/qA3DowGiHYpMiHvNQ93VGBcTiKU%2BQnH8o8yuCXW5vH8ac/wAbbjfrBSgn%2BohdeYVwUTVepU4idEpKvLr/AJFRX5uCNdXaXNRXEKBb53ClU2MUaV4lu0NG4A7hp21ib5fqU4jw39YuV4J/s2CJv7L/ALlR5hsYhhMCUuVAvtH4in63OtP6X8d9zM4qdzO/48QmY5E7hlZL98J7/Sqwl8U53F%2BzEocbway3scMUKsleNfrcU9lDqVtYVOE5uWcCIuJwCVqYO0dlRTn1yQ7lHlnxhDMH/PdxlbW9ZId8IjMP2JmWS4auwe7ghrMzXj%2BfzPExxPeef/HcEPwfpLoGYBzMA8TwRDUb3e5sMNG4RKhxWMR2WCZxLHeYHNzXCYviBGdCUpmNtN4IF7hlUCAqkN0ds44kp/rEnn%2Bdz2J7/wAOtzNzH8oi6s%2BCQLRuIwaisOn%2BCsxvGZoYxBeopqsRSnMo2tKMTNbLcEFxvRg7cIXUVWsx%2BGGLlCz5lJUxufLIZcEOcz8P7j/4v/zr%2BPmeZ994vHq/h1CK9VGNOZ3hs9/4HC6hLhqXvMRxcxdTc7mVTRKy4zBdzuYhRuo8El7mZfSWCyd5YgZMy3KBPJl9T/P5f40xrc9pxPz/AOEsTiaEyvz/AGIrJn3glBUrVcwwzFMX8SjQiTKclPCKxnMSsMzQYLJoWGMSr0ZlY1BIYgDjcwXyzh3FQxX0ULQhf0zE86vZqbK/nPX8Uty5xHGv4/H8pqKPB9A%2BIo11M5Ob%2BsQB/gJGw7gNwC2w4WbXKvXmPJ1K%2BSVDcq0wgL8IYPLM2Soa3MlMWL4iKpKDMs8q1P3rRbP1lY/8h/Gv/fioQrOF0nZw9SoTgS6QmKWRaXOYm6kQ0w03G%2BUyymzuIp6maNs6nVF63F9yaYhikBLV29Q%2BDmIpdlf59v4vE8SqhL%2Bkv%2BT%2BaMiy7dBi5WEtyM%2BtLzvMIYfGpzqZ/EtVHv1MFnvj4m9e9zCriUBeUXRrEMcy6DOIo3MlcTap9aEezMtv41qcTGMTE1PfU/E58x/j3/i/H/j3omO2EnmPOZRUUWRa4nJMTKCXJ/fQW7SmOY1C2XnE7ji5y8wjB/AHTucfx4lzZU1r%2BeNSi7f4vDiV/FfxfX8ayblAQ%2B5jmpml/D5IQaY4IH5gSakoUZbuXu/4VlSzlY2BdZVQ4JX8VDef44uY%2BI9QqfMej%2Bc/%2BGWpyqKtRuGUWkEZajhhmcvF8QPfLBfXtO5mWXbCKczJCtLhp/DFHSPuWOhX%2BOf/ABrcMy53/wCPaX/L/N7safEeKdSiKgtiX3i1iLUMF7lJVVMxL8pXBXWCOiJa4kaDESxZ9phtzLTEWc6m6KiyoXBr6f8Alme5rU%2BP/Xv/AOm3QYC8hGEmYKguczPJmHAZQbfzDtOU0zeLuCyiLzz6zLqEXMxrAAEY1axBZFzFuEBoxPnP8c/%2BLJ3HH8eOJj5/h%2B89v58/%2BDRPb%2Bo111KeZfNHMUeUpVsh1mZrIaAiiEzyCXKGo3yzSoScy6E3cMA5Jr%2BK/jxN5mpczOz%2BMQmP/gIRQlbgG/7%2BYuDAKqOcYhCyyc15h1pCkOMW0uoiXGqsNy8bixTB3HWFl0LNwef8/H8b/mvM8fxefP8AHD/ATxx/7WT7RTDuAY%2BGpUmhOnEOTqHSCrxcozHGeZXEMs4m1hlCKwm4rY7fx5nx/Fu/4/E8fw89T8TTPx/Hj%2Be//OQXwRS63DHe5fRxMi4iKEcxMEIu4xjuAkVDUz8xagQapmU2tyxUFD6rn5nczNQi%2BJvf8VOK/nB/5f8AyLy8Jr5/gkYBFoA4ZW6metzW9RVqKiaQQbcxsWpTTOrmgE99n8czx/HiczG//HzNS%2BOJ7T8fz0/jf8ivEIa1MjUFwk95F38OlC/8I/MNcWDAT6QxuUOeZ4x9AH3Idw3cHc/H8sJxPaY/jEt/ndyptr%2BB4IM1tHY/ogcMONRwir9p0bj9cQLuYMXCsj3KGi5ZWAlqqdkmXMekUD7PTPlqdxLEY9cRIw8zwv8ABrE4JwS7/g5l9y8fysJlHdDMw3IpxKx4hXUHUoIDLiGuMzommAHH8BueGGtGZxmVk7iOjEc2JlcIxQv7oO8iox1MTxxLxN4jiddTN/8AmoGZiIZarirKNmZUeYGuojqOcVCscMrggr3mkrcLgXuDqoV3ALbajjF4hZDqbqmHibSQvxH2pH6svcPEFjURPaH2/gj9o/x3KO5gr%2BNZi3BS%2BBjTiNMQoq53KQ3Mb4gJnifZMbm1cwqJpPshSbgEvxLEw4jb7QPrNBl5Jk2RA8ufZ/O0xDJHC2J1LxT/ABxKieJnMyHqWV/HMumm4QOn9JZvlKNypUvibJkvuEF4IGi4YJWZhuGagNRVZJiqhumeTcpMyg8X18EPbbg4Do/i2ATg/jcrcfbMqidfwua5/hpf8dfwZPEPK9RcGLt6fE37T5gYuI7JkeJXAhmzqAoRRiNX3O7mE3KBifZDE5LixArBWvUyYPDvyz3h1Pz/ABxnUK2w1MI8hClbxPETg/jTxNf5%2BZdfwLief9QZYlZ1iZrG5VYZ04nIQMvUBDzMhcF53AEHUMe058fxteo3CJzNj1r8n9EdfzX8F56h3DpjsqOo9/wTnEwGeyD/AAe06S8s0M64v3JWX8WOLlGyG7CbIJqpvmOJqEPtMcyquLGxmBt/ol5aOjR/syb/AIvc%2BYb3DucpbBj9oM//2gAMAwEDAhEDEQAAEMsHLcs6d5Qj%2BlawjL6RrheXx7inXv3D69zR4iAT/HG0NMC5i9G%2BERahDGh3XmTzbYWJ6hjr4Il%2BH6B/E/FiW9qD/mxDlCk6Li1%2BwtnFpnQkYCIZaULCtOTCk2H6KeV5jdYyMOidnrWyh7of0meh52QKURvjAiEl8eInOOQk8NZ0Sg2C/m3%2BG3sWF46BXMsQxAa04jjvjyuNyEw3mtNOrDsW2LaxO1p2BWMJA77rB8rJqDgfJRsvqfSj1aASVOOCU3h1KWRbRKBVdDnVu8toQ5%2BjZEzJjLYxtnOX0Cnn68yxWgmDaQf1RFOdXFfb0%2BPUEP7kAbkFQldpppTUG3V5jCe6j7TJpXAC92hMdDnWpHhonOjeVPygG97HKJB//rKHvMz%2BeF1lxET64ngp0AuVmR9Op%2BH%2B6/dT7Bl2KZA6U1bxJmHQ3jmpXPAY6wYv8jkbIRajMejJnnH401st6IKqdLevr6qc9RKy5wRCYqLTADC%2BoEQivreY1WP6%2BxQFsrrcET6xRB2X/P5KSYoOPCCzAJobw4XAlrURWvjyEHv7nA878GW/U0//AKq1iYdQfugQL1VAlIqpj2gzC/ip64xMMacvldzngzf0zis2Ny6bhteI2XtwHm%2BWLq%2B6%2B0g1AN83lw6m75xwfEqXwDWKrMAd8UZ5j190SyUf2vUJRiY0%2B3x/5E8iOEWxErx2BbIYc3PDJP4V6AbLHBSronMR0Y4x/gUSYE%2BkJnYItxPFGYJzj1oh6MpzUH4IkECpNnDG2eksxBSFDKFe4698m9b5Tzj8obhAjtFHONb31KjXOqpnqBIXZayQe5JtN8LyvAApb%2BHG8P3ro9fcE1tMvD2uyG89vpaI6OWkg7F37EliJH4VNTZpc6pMcb/SKh0thQMTDHuQUaHC975uiBdoWkhUDIzJ%2BeBptG0nD7tE2lxnBqM8xmsHRkmojNOAdBOqJoiIgvXsHfszoCJVvfHgl1Kn%2BrlstVLqo18iQtN8o0vIC0DIYL3/2gAIAQEDAT8Q9nLdKcSurF1nfcvoVXY%2BAp9y/wDuQ856LcyIsscsShJposMjsbNfMEIrdY5xxk2j9IOKwngXjhdPEpqzG6ALx0LHxcqsKmqxaUYdjLhuqscdUBQPHmMbxd0urOnAS2ZrFaw5WlWZZmIF7AIxrrm7ljwBSQAVjfGKiDJDRwWo9lFdxCGi1Cv2Xm48y6CbWsDZyaMsEC4WkFE7BXeAvcESQYaItOkxXyYhIA4opQWsUDbdZjqgxzFMSquzVykEglc5AEyoXis5mwMVwpnDl2OVamiETC3FtAyXQYJLoudL6mDUi2gIZiBBw4oUoMVRiivFhbYPyjWmNCgmQsoF5vS%2BYVYQCLse5gcOOY241gXM1oaD%2BYloWl51SpTLdy0FYd4Atvozise8qyQwZo2tCrqk0Swac2w3jbQYH%2BpRRi8LDHTTNfSN5irbZd5EsdMsOGENzsmRYZ9XG1KA0UM4tsnPp5xoRAKq7CAlFXESI0rMzxeBYKD%2BoGRaMXPgynvqCxDgTJ4FQ4fqAAKtUdYDizS23fEG1d0WHF6VfGK6iyYQaEypeSrkiQWlbcKJq77DEoszWElGQDwsn0JZStYgaKorkcU5il1S2LwkCoXeoNmRtZ8inm7a%2BJoroOQLRyYEz7VAs4Gw25vyWr2MWIFBYhAFCuHw5gBZMQNA5Rq7s%2Bkd9Gn0tasOQPNTfuCqh1Fkl10OZcjxNXKWlVyRtgmP6xCSBzSiweTMPWQu9CH8sMFVzcsgEgG0%2BugWzBtbDIboUqwLiNwhCPpt4Iz8SA0AFe5v6UgBgiyoKOQrtZuWzkBo7Wwe4FIjknkcqyrC66MmFYx3rYC6HDFGRUxqVVG9kA7BxCAJDYsdibFNpeqh8Yy2FjaknJxmWgsFuXEtUsPZlieFXkdyktYgOboWOvg749od2isiXbVKNrPEQCKw7ppGjtwalGhAgFhCuS2jY%2Bxmbz8bZK9hajfvBDwVras3yaQyGQ2c7DN7btSjC%2BLGAqg1Qb9CBV7rFt5qjAXbdTBeHk2PFIUXtEYCgTTLTRovpDDm8uvGKazj4gGKLpTItycjVXG9/DV3alzQNd8EsbU3dospz0JEOQ7LUdsvG44CnBatOG3AgSwpoKAKsDgGm2YbxYcpinTB3mc6Cra3eZHyLccpfYqGiqMo5uJ7naWBB1QjV5xi5U64F1MmoLYARCiigqWJrevGZ5AaB2tVBrDAGCJkXKwm9AQELtpfCQro3tVwCKipUGwMwN4qjopdXHVkG6Eo2uZl0wMBpvF16XoKFOLwwUPLqp4pbilwG5WFudE1UcyF5GKhSKZRvCyYVDxAiCoc0hQNajba7uUre5B3DhKFbhzMqHzWSPCaCot2Y7LrXaVHL7uMlqC%2BrZgJSd6UCNPWMzDIajNNStBtBzEjSA0rhW1EFv8AapVGfCF%2BBeDi8MchZK3l2WIrSQTeryOKoLwyUvJL2Tm8ZVOQo/KaoYK3jnIqjdcxDVCulLCiuqvr5nGxPbGTgXZ%2B44Qii21FlhjiEdKS89HlQMnmN7mqcNFclDXFSmMWhvlgWaXPWKmFFpDC8FC5bQIN4cnClsgpWKg1fAuTlDd4/RKrsioLSxxjunuPImDkMpyo5reriMKFtQVB7LyXGhxkpLFWgU4aRr4lMmAqVdms72gTGwpd1ZCJi2rxnxBZLFJUUFSyOCwhdukcgASVmEPvR86LvCkLLraXzECh0i3YVuJoBhx2BREhkraxVHc2iiYR3QyVODfMwccsLRNEMZIwCjlbXFcjz2M%2B0vAgUuGUmtsxyVjvVUhaqcDWRgEurWxxWsNxmnxDhTfVsCTBYWb3LATWYhRd11qwDuFr7vpOO1lJp1V0S0dOKE0bWOTA5qA8mi%2BjaOMLRQ2ylcJmg2AaFUEoDuu8fWDVhYZQ1C4AVlCpnAkgLqxLzmDsEIWJjzVlQxj2ZcltrLzysfgfEaPQAKzlhgtjmWSgFqxVkE52aTdgbycWqAQcXTFFOBVBsGzuyUhexB40oe3rErNMmBzg0dH9Rc2lMjYYN3aKOzQrdAheG7uqmRa6RxgdttUcTIqcEX2Rlv8AEoZdCjGCJsLool4FJVt0mctrTddylM0zVi1IO93niHlHCcxR86F/ENjKN4OFgyU7lKDg1lwWUbXWOpgbM4urW3j2r7ygoVF5pS2Kz3zWWIErFoArdQzSQ4uXXZ2rvgc6wJrgXrlVl9lEANkaLCyv69qGruhqpinKzFYIILybLhycS1Vlioney%2BTY2amZ6KkrDsR5KVbElVIyoeEkRpPMziWwIlQ2qjSX0QmLuyfUa6OQ4K3MlUoD1AsVVmzK8wpYuUVVmW9hWQWw0KsEbIGABRYGuoiuRAdyBzgVlXEp6IpFBbkXZu0XgxFbkVPAsQ7ZmE4qGAoQM4A20eB7WcvkhSjLWENuiWsUR5ppxlFgtrZQqjLZscBCFHCz7sUgk%2BzSBiKozi8nnNMpSo74DcEUtVeG%2BITsl0t7JQUWJfYli6AuQKaFHNPeZVaLUKpdpRcCDXcRpwZzeqppExkuK7aVWFC82eWqvZEW0yWwVnpXNTrFGEb2G1oU7YhmtDXii8YXdf7EAtSjQ2Bu84VOSVWoGP2Dye/mWugUJe9YZwt8Zh0VCBgpptXWOojauLOVu%2BTTTrU6yeeMImLDYXBS2cuWtBbGn6xAXXIcLxZdaIQFOAV0FDFrA%2B4lQd8IIJ1gtZAuMxuUQNaFAcz8LmJQqWC7SAuVGUIK4IMgAVEQ0thiJ69gu4AhTSrE2twTRQBo2GBKycjNJzjEpopMmcir1EMkNWVrlobMx5hYgUhIEEtFaKc8ssp8mtgkwkxcMoXAIBIQIV3wyhjMZSLWCUEaBl28qhbVgSGWCm/d6NQCySCiMZs3NpioCgEBoqNQo8DN9QpOFBeIPQzmC2e5fINuCM2BK8EaaYg8gZQcEcXRZRctyiOSNabgrcF6loM2RrSi1F0%2BFqrgQRLsN48GBiqcEypA3wABYwK0AeJEOxAxlbrDNsZKERSWLaRu0RUQlVbKHCUtgpZ8viNsDYBtAhaDXDs1HYVa6UZMKV/tAGAzgxjBnQffnzARVlVb4Ew1mAgnFq4caCqss9ERLbsTb2eujmbr0qtcqbM6qoVglbzoudHGeZmkrKwDFI3Y8qmXQurUyKbUC6iE5LbrTVWh23Bu6act1d4oAPlepgupVYVWbZAxe5gyC7Xd8IFCmF9o4uaNSAA1qopLRguEAC2zIBLQUEwDED%2BcYUxhoDlFRgexMKbBGtBO2t24VgPNexqAWKOhqOhcLJNiu6eP8TA0ksK4BSC8GXjMY2ECxWlgtAVqi4YZa8kBQlomhb6qc/go8oV6HuE5IuDyRUCCHCnBpuwQmBWKFQ0GqUOF07qJCLkyK2BLA1kZM2RHWG/t2gVkLNs4QMyDcDiQOx8xUKaneVstwZyz3NrBYgjDIhocB5uFtogRmu4oUsqfEv1SlgDdRbXd0e7gTSE1LAYiIlBdi1HmkEkRBs3ZMOBrFwFQQi1xUvHU32xo/StVRTRCQoCmHP6qcWIuka03B7gMtyHCwvnVUWkoFXBqA3aat4uNNG2aBgttorZrqUfkRzyO7AyD8wctIHY1bHD6srKs89tqs0cEsRKs1L4FGnFdmJkzzOQ3bA90BWiVBHRSt7aWXzVlxCm3pAg/Q1BTLgX5AGzgfiMm9bBR7GCoLeAAdO1U7pxbaalCsjuFGVM2gaheq6A2YcUXhSupqpvdFNdc4xuUOgyrZKpqqkbKMwNKSKb94G15AL7Cfg%2BhEZiNWxRowgLKHYfJXAznEZrvAwLKHJOefIROkSlFFMTEsKqmGWtoBYidQ1PWjGAmFBKqYuEl4eFvUxxpLrc1HV9hznMZPCxkghaqXdFJTQwrO06WFWlG5V1leQbS8Q2s5vuGJA1pSygRRg2XuSlXJrOnPojDZsibu3Uh96Dwqj2gMhyoHCqFG3LrZKMw5KOuiit2uz0xyMn0olgsNYBZQSsYVd%2BxozbUKyU8x%2BIaVmNhYaFUTomRkZYxpiRWtTcJpC3nflpQ7AoZEh4rb2mbBhZNi26eIhLxa%2BBroEdTjJYgtQ4EKHN3XY3AG%2BKRWCLGbcX1ANsto5pwlUDpRTAzxWUpDlKAcXbC6rDF6tvurLEsqBQrduBNcMUsO3Lnpm0zmj5mzCIpcrTeS5YBgRZdlvuyN/1MGTI5LsllAPZRuKpaG1wtXSBWBZURxgTKlxkLPN3Mmm03sLnrLiZ4FGxGDJeAbi%2BwLVN53ZsqiqJe5WmRXi2N6WM9lOanCFJreAtxLObQegdusC66KyXIyYrHXSAU%2BkRkhl6H2wIWlKu4iheUjmBmirUisqLOLcAWyiWms9pX7uS2jLLEZYDFSpkoMhtUEVZqscWxFL0JVKoAOQqVwSn2qqGoKzE3kSq9oGGYhho0B6BpiWrA0Q7VBfGaPEcQSNClqMIKq8F1AUigVadlFCjbacQRXRkTbK9bWgttS1ZDwDeAuyBu76YFmztJRAs3FNBF2RvgGliJsIK2uoyhJFxO0dAxabb9ozYEZgRUM9IY%2BYcZtFotpRK6qxfJqVl8kAQtEELV2FOrjsC0jfs6DK6Uw6JSyJtMkXl24omH4gGyQVQiNnd4DI3h4VJqybmVkuLZBsVlgLaiN2Ms/wBwG0ALtMVgF5a%2BsBbu3IbAcXjG9ysLi/dumxSxeswUxdA2OG07NW65lnAxYbyoaHkhDbG7yqm8UtRWtFZF5p1RwmfzB0vdt%2BxtBpOYWpVO%2BA4XdKdTBSl0GQbYUpKfaOQxeQcVsXO/aUcIgWvJQNg3nmXiqkspSoADZk4jJ4C1kLZbjOCiRb3cLMavuoKLA5hNRmASGRBTYGDK3Hk5FcDg5Vi0DKiiA9kxBQXjmDyYvG78FLCAdmr%2B5UeQoF%2BNDQ1bZV5W9w2kYbgUrLQvFZYosa%2BgeWwVbk3iUdPWxXQJAFnKkEBkIALhqzQyXe8sfj8YDXT0J0PDqFD7hViImcBKHguXFguQlaOS9MHJrEqfVLCqqyFiSryAEAlpIpXVJsEFuy7jjYVL15RxxU4G5ifqcVcJjBZlLzhbIipqfGQVgLjQ3uCv2wnFYkArM2ENwCqHWJoQilYCPPTiMBNqjVQBtyQNpTd1jDaTEhStpKaHUagjZaSrBFBSjYISlBNx7vExRgBxCbMBBR6MQ4CRpGcgr4TJnTZocHE5W7yoVrO6t%2BlQ1YBAK085Aga1qOc3eAC22OC8J%2BJeDrbftvFOIgWVlkvbFrONTJs1tWWR0VYeamNNkDQLHC7zx9obdF6GW11m/rKF2iBRusEadpz9paxTFmFcN3Z8OagopZdjlzotmx4qBMoaQMLYwNU8KoqZtnlaCQLW2BJliC84W8vVt0WtXe1x2FV4WC3G3L8e3aYUWI375u5m2xYK5VkmAXWBm3iErDcKrsgMFWKLo6zKuSCokWHSfAxBBIlFVAlfaNOeo/35IUoBQl6DRiCHbzcCYCCYKoY2QqOCpEVDCZtB3XtDLNbGaxQOSwrW%2BYglLISIuFl48/lLpVSwI0Rq7XdujBcVAZgCqLgXz3uGQbguRa1h5FmHUe6uaaELoJlobQyKQpDtuFXbz/xhgFNzYV0wMqusF2YLrwCCIQ5o6bl2UjVOUSdGFQmM2C3tRj1sF3F1NbQoGrUipCUnh9gVVBXZQKaI/L0135AFNHLjUYDOJsKhShLpQgeYwluLmYaha1se5pYPc7l8s8TVBIkZhZPDqzOW2wIqq1CLwVpbgE7lt5C1FCaFBULPRDvGQKGK15rnxF3odObGBz2iwHdLduhQc2g4L4lsrk2fLnzDQqlOrzha2G9MLjWi6uy1bmq4iAxnVtbpWW63BEDiq7BZZaYGj3ibbl0ZrKlFShF31GiWaWzRTQ2q5nPcyiLDAc8%2BjzFaxorQdYetAh1m9rGLaLVc76wKtDxoNA5Vo6ksUogEcd8OW7AA5FIXcebxSbXUqlpZgzCmRJSKNQpXQnioIgldANwKdJ5uVi5QIXKhbKEyZlHQ26ARq66KuNB1CydCZsQ28LmK5gUGihyi2uDJ9YuO8pYNFq7B1MkpSBsxQtVWxHyuDW4ARwFL43XlvczZhitm05Ft/YQxgrqEtaVV3yWy4c8EYBoBKVNvBGIYgR214VFDqsXKp2QBRFDnuezUaiuQucmMErbxWRxGfGYLjggbODNG1TA%2BKHR1HaKWHYx1H7Amsyk2ZjAsMT03XJtyFgpS1FjuW0DPukxbRSGYOSOlwKCraUsow2pcAOKQJHfDBqKiQhrIyYJR5fMwRM5fCUKC9sveFRq00bt%2BS5zZBu/N4pXQSyuk2PWfFLL1iXFXCsQtaDiyi8TPFgqETQoMiQ9orKDQnZkwpC6D/IYyksA8Cl6CWXHsGBao4KQX8ZvYSxpF5IFNG6KpuXBAFWc5CF6EGIsnTp6BlCCAjCdsHTLNrAlWAag3hVfQcLBEgLUxAuUAAATAp1L0bmnSes2EHgrBzfcoF5cbBUD1CrzWog1pWFVuZMtMOaEvK0N5hputiJUSzQc0XFlhgp8hFKXVWAekXRL4rFQ0UQokCjpd1xnUtUEYGUAtmtbl6NbIoRQGLC%2BWZ5iycBvK1a68wQ8mswzZsCg%2BTzGYxsYYsAqk2ml4qoILYRAqwsEbTxHrMUEECypSD8RDLPLYO4t2dTEhUWgAgYssjr2gABZoA2QGLHFZH3iAezSpp1ZdrLhIAPLrx4sGehXYwBuqqqGKWlSWS36rcdiZX2mV5VomOCKDbMNborMbOxVQvms5bnIikyB4SV8iqha1uSFdKFjbYpRYl8xdpgMPiDaqJqkoaopLoVqVoqFaBkNX8zkF1dB2VBVC/fqcHN5ZK1h8Zx/2YABRXkLVCsZemVsBYNpag0b268QFldAzhrSqaDLcuBRUtxmyjjKzCMNFc6WbR4MWzSMWBxeRyNIWrjoiwLYZXIO/ES73hFAJAMpY5alTuhRqeplVIUYCLVSQR2KdlGhvfDCIrGw1AQLbVChiMeEN87XA20XJmiBYYZBBb47z2YolNRU5ZvVKZXHvOBpRkLMEKBGu5lGFWhsaXkRKrv8AcBrVMAhvUtrt0wMcFhArwrCgBV2pRqZqDABhWChk2zQDaDEVKRNkRwVvmMVyAtAVTXgNprcWgMI%2BBSgKrss1qKxUicgFgPZk4mKAFRwhHFx0PtL02XJxZeSiaOeMxAVsXkCAqqjZ4giqAA2zgILtgKiWBKq6UKGMalvrkWCiyvI2D7w3OeEtO8rYYoeJX25K8qDWPTa1aCAggvkKvCWZORGtwC0QeUlZRlCOQpsY0BmuDAgKVBs9mJIsgCZYSuZk4XUdq80V3tpcWNauIeQhhwXRVus5jSAUcbqza2ZHc44bclNmLojVN8/Dyvp8zdLCyku2%2BS9eYxfw9iuW/dR5i2qyZeFpyK4vnMuTjZZhocUtuPtFG3QDvOXmqTYrYMM5WywNPGdy2AWMmqxeCzJSIcvWtcQ44uJeLBRXWlRhqE0f5DQJ1YhGNIdLxb9IhiWBYKpfaHIl/REU4eWAKZAAK8rlh8hKCluwCwDnF6mV0IGYtjQuqPtKbNMuwBp5LHm8wDKIeIeVACjxHGp5hZVQqAPTUQYAiUsQoVlmH2iJa02Orauwt/bEd1SICEOgLDIFF1cPtuywaAAAna1zfEYeDS2ALQ1aNGpUJPIUBAA0R8Vy4ABgutBLz6DzGRIVVhLMGg95z1BQNRVsZgsqcb6jiDMNMY1RnRysFWVCIIOHP2thATZoAFFcly5xtg61WRaDdC64ldXiFfsgEV544loulIbIrg3mYtAbYVL6mC1Ve6tmZSR7FMv6K28EVyAqlKeZlt2YuwBRExW5KLsVo1mmvxBywDstvZijb9kCFiq0hfZUHYeYxYKvHkTBmxxEUBSirQdAxisr8x8vLIeL6Z/EVBaKccllEqjiYk6sUq2Czai6eMxBTitGtK4Ht%2BJmtah8I4OzJOIHAWr4Vri9cTKuemF0wKAw4xUJFBOS6RDPDFjGbjiwXF8SA/sSxXjUwd44JmqcKtlaJAK65aIpBi6zUUSBGh3jffu6NVDMWCnRWXJsjRvLBFDa5sAwHK1gxsIwpqA2Ac6KL33EnIu1bbauC/EYLVAQS6bYLS1ZHALqgWKFQ7NtNV1ibYF8WtC6cLPoSFGxcmgARV2XyGNx6e4VUQcApDlBb95QY4LmbwoqpkOftAYANkFMHExwFVDsjqARNQUqIzlbmyiyRUKChvIc%2B3cHtAt4AGgUAYc8x6QZA1qiRsYo%2BkpA4dLLBb5uz3gMLV1BFFC01Z3MdJFrKBasAjHCMMojFhAVsQVz9I7U2wRDZjKMKvTUckEANlXVpY0PFkRmFaUUNgOaXsYqAvZMX52rkBAWLmOYG8ZVWUrWSsdyy%2BF07aGdEal3bW8LdW3rRvUA0d68L0WwePtALIipsABoab%2BfrLbTix0J0bf5KERd4aoRRhy7vuKjZxAWPFWY5lljVG3I1qkMe0JaMYdDWkA%2BdLgjkvQ9jNtKq2ZNuayrAospASxs1DRyu0tIsVgERW0v5TCpNOLKdj9ZVxQETSVVTaii3ErYMi0CRaiIjdDlAi3AeSgUNjVeblIBUFJsBSKB5qABNsgGSjNgzRolKGtoIDpRK0dXROBOS2zZUNumFRuhcGbEHCId8x3FNL6QLbyE9nNR1sC1nQJy5tYG/rDg8ArAZTsP6BcIK51QFSyrFQ4Sq%2BZaUML2INoAhgoYhYnwNx0UFAeOIqBsqiUlkYKbV8RRbQ5FW0tSBXJo9o0L1UsY4YTs68S4JquzYoAFUDd8WzDxyWLYMAZMRp5goWQqt1bs%2B8vzQhWAtkF%2BDcF1KKKqC4LqsOksxWFxg0tSPVKbz3M6EojYEN5vGEyZIOaAxlxW3BTA65i4XrPJPuYKzaqL2LzQlmavIDRwqpTSFoIm20GwXLHiVEuyqFlvOMNjKFWOCBq2FUbIzLAhb2CWOLcRoBew0LLd1XMWsyCBegeG8XUwAaB0kybbSx9uI1X4W7wNhjOvDuWOF3eHeBZy2P1jkWVV2bCtkXSJp4giJ7VBdSgG8nIdWQnJmfIEiHolVcesqqSV2/uoI3ZjMaJZGsiCW0WLE%2BUYrExLdpdA9csQgsGRaObNAx5ZaILIGKJTmhdeVngzkx7JQ%2B4RM0kEo2qALToficgDrACsVV215SIHINQoArAo7uF7a4tbiwBcDbJfUYQ4EAFK8SwhiwUhybVYWUMhANNZ2ksAFYq6A8BBTd29YlDUoY86rWyLoxRZuIqwWGAawgo4%2Bsei06pihSACfUQaTPlgd5Qb7JritlhSobeR/wAgABQRwopOSFiaDa4AgQwZzV8whNai1iu6lIs0sMVhjN15s5Mn3GPviykUYqSAU2PtCpBkpCoMjm1TTMsxqoK2tJhWMFRqlgMASxKRp0ya3MEQQRb01ei7GQqb3FVWe1VLsZvjUup3YCFlp7/ZCxKwOgy6MsEXkI4c0VjOE3qot2NOQVfcrfMWkIqWvVi3mh7xKg4HPC2J8H0nCrC%2BTOtVmnG413wFdi3RKa09ZiG8dkUQcuCYl0uE1RXR5M0LVMI6LyW71OpfasuyOnGJMZPgYUsCbUtpLBlOKgrSVgzxE7KgCypvXZMg6lhFWGBApbEUMPMXnFotCW7KWZ78QggqRSgazdhH3gHAAsIhhLtXZWI2ha7JFbONht1zAjMtOjYGbYlnPLBpFiN1YbINndblkDcFbW1TYG7IOgtW5NoqaQNt48SsVMLTViAGyqCYQ1EtjkPYAasjKwJW4Zs3C9rRYXiOJKKg5srY4nOkcXiJeZSCDHFxua0UpbwdKVlvcs%2BVHNS1r0UDN5lpKQps2VysyxjEXhktjAGjIeHdTA2Eqi2o4CD1e4QCPtSeilyDVE2JYBYXWLyBRhJksdgEmiLAzysvEOBBnILiML2ag0s7nUzwVfkG6H3aN1bMGrGlhs7O1A2uVg1gxjLuON6beVKUbzebhENiclho%2B%2B4OVEwq2Ci%2BSxzrULWAKNsZsy6T8kyLWAum0aWlwnynuDGISUWMNcX2yrgrqALi6ttaNqrAAuupiouhC7LLy04oqGLeRhTSmgxSZ%2BGBcBV5%2BRmyMUWS4RGD69EUfpJLATYLpVQAW9uxLVuCTJsHy4YlCKbbdw10S8GYBVAdw7TAaCFUs2M2%2BZW6BUE6jAGbKVcM4SWnGFFBQKGGkG76egIbVW7FKlNVFS2PmhuVKBmaXKoyYAgNGDaLZxvd6ikuRqg5gOek1Ms6uprJKUOMvPMrQojDNnQhc0F7zOAB1hcoAhoG/mcX4hbbQmacOGZSUK00Q5C7Wj28QeG4t8gvKimlbITquONaJjOoCApeM7D4TIBoEoy5MNBuWWtFS8ECxKnGp4FGbZAAlgZxBoS9CCNSptIjoROl03i81RKN/URrrMGAQ12CnkKqNYIJZE1VCZawmplpyipkdSlRfWIjlXANdGbreSm9wWqTrnti9qFIGMgsBR2WgRXa18oUjFF0COUCtqWt35lm8nOGt5ad2wnkSqezXTFYurhKYtduCwZVqqi6VRZEqnQmTJFVqUdC1qVVi/MZVIW7MqtcI2B57iu1zZZuuudHtOQWduCVsrjuDlizRpurZvTDmjhbAKHZb7l3FvC8cFGO8Co9jTTQgsqZ8nZdMclcDXKtRLge6YnERCrWNmZLKrFXriV5xIBBZsIstu6yZJbkZqYWyZoGrFVGRuTkgKApqcFcS85GlJTgNo1Mb1WgZCFk2NLPhuC9sRCCAwryHO4VsG%2BIBg3AiFdtRm1oKg8qNpAujp2ZNKNPbQ0Qxla6tiEsBkoI0%2BC7bVHbpDXIihqOTgglJWMrIWOlsBHEhdEzqXBZVhq8RROgZIOw3rtFKzkK8gNJNJYqyvSuRH%2BGxAqXTTds5zmCBNRwesuKIbUrGoMo9dyVyZxVjurgNwCWobQpsvYsDwFjd70Fx4Rp3UxgNABQZJuyudM3aWjUIXSoEVZTcYrqmsPbxcA2lcQDyVO5ZdAMBxdRguxSqSgMOAaLgXND54LPGMe8c0OUcYumrwmavEWVq74a8grPfmEyaJsGGMexvMQ0sgZN1Q2LvPnMQIrNbQcoAWAKj2wYqsgZAssuK1qlBVVuuDJ/2WEEVTFlKVNbUd/EsdW0o0H1OGOI61lTptagu4FpjXsxFhyGEawLc2hgJpCLFrcMXKLDVYdDQvwLt4gBsQ8IWipdNG4jhQW4rAhhDTvgqIh4CbOQsqBW4rjeo8Tp2i3Te1zahww%2B7GuPykpAJrFjIpm5rhQTYMsgVq/pggh9wJgzRaXCpQUKQ9QFnsqIC9mhFVgABWmUko9JANAOi1wY8x3FLYNUOlFa8GT3gGpVibACBQxhNkqApZTTigafb5hco3l4hVGAH/UZPNS10oRqGeLlAA6O76gLDexliUG8EW4m7BI0RybIBFTClCAEsjNJRq2QTJRUVSigXWG/zp1XCmQqDAjjhKkS8K5UXm%2B5nKoFDExZaKxWNwhgWRzfuWnrvxWbi%2BQrIYaTLjbipQf8It10pcLIFZq1JvNDM2iYE5UMLLVWrjBa5A3xcZrlaLchvB2h4NU7pcZUeXe4hJd2M4uqXvUtKWUpTOsvOSGyK3PPwRFt43EShbVVqqCU53niOs3kVVMoorFsH1lvS7p7nkxnOSK0i6LwclIUHU1fjFK4kS3AnJBzMdGmkEH0uBsUFKaUNQCYzs5BZuoNiVjel7kmNtVxKEUAGgAovXN1LF4FC0LqjI0rJHE8A62MAWWXxWSE73T4bpEHipNotMHjWiq9vWOAUW7lQJH/ADRGmlAIKuU/dLsFL5ii9cdriob4DIwykTm6gidcLnxq6jTADhWVRF1zVNEJzgsQgk4ULJRdRWWFQTsbURv2VCGmMwvqLKpb43MUZQlMhqRQsLPpAFSEVRYLAWSmjiWwHeGbAkTQUBNuLYuqGYaD1fEFEHVCgUqoEEC6IuZLUEHc3wuY4la2FVi15oIqrZbSy1QGQXXALZWCM6w5sFWoOCNmyYlyABIvEKKLzB8tgAFYN2sjkjYPqEVVpVSjHENgCFAa7F5d3mNsGnKOA%2BSa6icuUVVW71ejVIWb0LaqsmO9EfBxNu8UWmvReIpSgsgpwaO8Il8cTeNnezpcbPYROhJziqLcYjz7CmWQpot7SubcE4OXVFZcxQB7uUMtTNWm2XtmzCxYpfNlV1HvXs5lyMTKoWrmIMDIHpfkTF09oaGXw9aoSiu9kxAiTYbdqFloq8m4I5CUlCLRaJeuF3H3jBKpG4it4UxAfxQwhwF1ZYsEVIvGUl%2BqurEIjhmOodK7I0nHDoBWpczXwgDSUQBK1SF2adqxxYdgUhnjXowfjcY0rJ1Vr7BWBqboeVw3zVY5KzZth1L1gICKDcBFo1vUVco1CaFBJy8QAkCoQ9aSjVuE1D0xmXtN7pZhFZiO0T8wVbaNcGmA2AykhAqoBK0SK29DDGUJBjIkw9x%2BmFCwO3VoktYooFgJHpeGkuFmt8rZsZcldVkJUgSkNEIFWNtxftBBlWgBauvCcw%2BC2cBZzzNVcrSQKCuizWQ54pXGLygAtAFqI73lVCUKKTI7%2B0N9WOwUgtKUeIgylG/J8hWTDFFXWOAlFMhelxLQp2ItFt2mWo9WUio27TPlnuNQFfCrAWDojUUtoTBWEKqm6YjFPYGmecmXruPflG81Sm33TjRWBeARn5rEO4lmpYMWQuWgOahiYU7lFEI0MvEOym4BmUFEpjdwApaVbChDY4VgbjyAFRXNihu0%2BYdLQF7Dd6Yu23uIuwLAw1RUc1tjqISlo7YJjQxu4JmHt8mVmzWYBitYibQPoMlLLKgYA%2BJboDaN0I0o1NW0GiAzzioQLKgqfTEI4iapkNIubL5qBhtczoUm2uWJQHC/StJgYeb2EAtymUGiG12EEvoBRVKAQtQZHL83E6hoJoZkSmt2Wxc5nGuXqjFjDdx8gzDdCgFPAsM6lUw4EJDAjBZSeoh%2BHeNovqIRWr4jOIDQFlGhiif1KfQ1YGMDdNinPUArLGQA1bYi/EBjYAEKNGkLzykOuAdSyiJ2QOC8kOwuIRnwQINOLdy1KWWacu0UOP7jdKg7UYh8NrmhsVfaeLeniN7BTV/srpfXmIrkO0Iptuti3xECrpJaoSyjTsuXoEs5KBnIi2u7lhcKuGG0VoOP3LHFtVS7BhVnH3rxM7ec51dUGOXEVrBMa55t4CBuDCis0KcCb2t7jPEgnrFUKmzXG0K7omm2jK3vuUB4KkeuFDJAOZQ1uIWqooM5TjAChTUUboNizPPtD5KEu4VbUGjXjUyTiBBVwb0Cih3FChcYWItrwTKrtIBbhnFXbLdnJxTAYVoWG2YFqScFsKOtRvDauloPq3cQ4RutEC7tgtxN84Xx0YVa%2B4IEhVmQ%2BdCLW8ZbhvKrXdssqBX3hUCNBNAWwNWYDnBHLI9uSiKRIHLqo684o0HQsnhKLYVuBwFNlkqAWn2gAtOgJSswFUfDKYWSgFIsDkoPuQpn60pKoprL7SkogWAAWGZTL3d4l0Od6othgArDzNMAyxmhW8B1zEIG44arQTuW%2BFPZFUKVEs02M0WRayJ0M9Di5Q6UDVZznD6dzL40NDYYLs3HIKu3QunIIlxctClbvVNBaVqsxqVzqxpbKy2VLUIJd837NgGZ4WxhYMbA4ojVN0sUEbw7KKY4zcKrq8YU51uuacyq3G2nCwCO0g1u1wRstznV1bbAi0Thz4y2VhUTzpqcGjaLis3MfW49tRk2UeEvMYCFkCgQKbjwwkF7gyIFCoC2cV5ghFzXS%2BBUHDHmGGgC1BYKupQz2yhG72BK6tsIDeLdQHxOBgRuEpo5azFAiFBrFaBIDpreSYQkcEAgNFWhAWYxG1hCgFwcUaKpzuPVU0zBMslKb%2B%2BZe1aBFJQspQoN2scWABvKGEywCAw7ggWXQvaLt8wBAtCtC%2Bg3Dg4IRy2VgJYCBxuqpuMFCjbgGUrBbrdyhbagVq2drNj3aw2jQGqARfZMNhcdMBUBNQL4QXUI3W3Epdp/GhsluyrOYbjbY3RXZu6MrvU3%2Bk0NnY2MhxjiV5eBhUcLwKFUDlVTFfhJhTjELKvkDLbIKIKw/QBIZCUCQRbSZRQoZ9kbvGz7xUVq6x05AW7ffEzb2ndOx8ry4gWgcOE1RagS/dLAG7qmTwXGS6fmY1GgQTqnjWG4gMMg3Sxiten66iVV%2BQZ5N20834iMCr7uqQPL2zCrBugDNYbbquIZedkaMOTACqBl4ixQGkbdo6oYAcHcUsaECgVHAHh%2BZyUbxT5NjKLMFkWHQBwQqGGvrLdLTbCAoHQO5Yb5Q0r43Br2Y8R7ILZBYASrJnmglxg2DClG0BOjiGBlrqI2/mC/6j8VDpQKuthZrZtgigVLKo0GZbSqm6yKiEbGBil1TdzMiqjQ4c3WyO1xEYo8lZZODJvHTELRsAAodC6atjmItwzs0gDZ2NQtQnFVwIYpW82RrAJClltwWkmdwqwoqXgGjXuzgoHeoKEpe1U5%2BYgYYtoLyy2S3m8VmXhAQMCpq7Lrm6y5iLXgaQq1FcB06hyJEDGFJB16MqOliYIGaC6Glai8NXQRfbbtvGdVGayYLJOoVc7eGMi4qc3YWc1TTNBF/B8XTSXQnEbCgxlwY0YLW3ErFNcOMNKWqrAvtB2c5rJFpvQ1UQzVN7MO9mOPiBY3ZzXVm7sbaomB3pQLHhAYQPaLDC3KlJchG3tqVC2%2BX2pwOCmfaGPB1ScKFFYr8w5T3ZNAK7BKmILBb4NgpHIFCBvaq8WF%2BUdhu8HYKi81cYpZBm/9x8ggEVKyJAYBd0jAl0kEwXatMTK4cY6lgoBRuUb84Lx1jzLwypbnKnITORLqBsUlggYW4FAMmOoTAEuOhAGjACVeauFV9xdimDveYaHkCLY40NGrStU8wKY7BJQFKFCGTGuYZASjxNCAINe5dxSXBTBFxavrLjVhZMC0FUm6EOpfAaje28DvGIxcQNauOxpBONYxM1uJuhGso2HxKIzdm4BsGaOOJnoUhLarK803eBTmBKoLOAQQohTGEIZcEIHMgRa4gLKwKlUGqUCHK2PaihgbeSsOFvFt6gWdgBwwA2gpfvcHi4JRS7LumuBe2MZi/OrA27m/Exorm0pJSFzAKoUsKllco0M4t%2BJZsaocCZoswwOd8weaXOlAlnVpMCiOn2F42wn1I5co0svFWlCtNRs4KDgJvCjPO3mKha2xrgJVqFAV5hdBsBRN25Zww5LyOKXkrnfxlswNBhKsv6LHMAypvA0WwDeHHUa8GTam6aRqhAbVgW0LbY6DiFsQFLSXWTjkQfWGVwBXC5py53uDmP2ImtN%2B1hSXEserHpEyoraQglxkFagsEA6zUL7WqS2tFgjsf1ALWU89ilIFpx/eIFAutFe1wLMNLdwgKpzS5DBwXjNI8ROGAxYgvAE0HeIJtNLduA0Bt3uoJI2kLNQsU3kvyxV5lsWC0OAL%2BE8wDay74bDNYV7MxNkEFZQValQbzjEdNaSrjBSZFlmql4AK1awFG/J5gHU51YdhrQt34gD8VqaCtt4ht7EBKbpYUbEC4nGSy4hgWLMLbSFPTmOApo3A26KCwvfUUEaXmwihZXg5NQORts5VKJwBxmiNm3pQNlavdb3G1PkhxBy0q7u0lL9n2MIKL0ksRgWLplAaG7QFN8xhN1Rhg05NtF/eIVhHNcOmG6yF4i0mlpdi8DNWBcLlecORBRvkKfmNk2aCK0G7Sm6vhq5geFqRbwVaSrIimt2e11aN7Am3M2kaswvGqsLYaSrKCgvOLFGkIc2oZW4pu74fbLGJjhdt0KqG0oDoDAjxtbXmZAI8EVLAQeKQ6jY2bNOMW8qXETEQVgbuxbRrSbhMhU3T35qLM1WXmAxQuBRgLs6zHqdUytWeEWQ4Ne8bqgnmyoDbjNTMSYNFWtigle9SxOi2kV4RYvDaRUaKRtWiizKqROcQZCkEQAAbHZDpvLiKysEEF40kMQAWPxlmKoiaYpgfMJINe6iDCXzC03Q5LZPOlpnPkxKht2jLLOg2jeXEur4jPjNaloYASUzbWpa1JFa5a6gqcXaoq1BD3MjqJEQwWi5rIEcje4zL2rUe8OVXRs1xHDAC524IoNj5hUbEJsoKXCAZ4TqARyAlYClenLwOY783lGxWSJoVdwy9KBQPVFnIl2M0AUyN1SCSgiEuq8Wg5aeZQaUQ5VX2Ow3EWmVQvkqg4bfrELRQogdBi7rbZLuAN3suRbZgeIpWE33dtbrTY54ljdhRcC1xRTTjiXaAWlmhGB0D7omK1hm3AFeRTD7E4cl4Xmlp1LlvJbhuy2c0ZJrUzQ2ZTJc9XDbtgTI8YesMGnK1DjN8vZPmMuoKWxxWLvbRC6lspHVJj5cMfImTdydl7HUbBlQyBagNgA8zK7kFstFDVKucMOsKmpaw3bI8kSoWbGKVAUU8dOpdsBAOERoj2ze8S9oWllVVQiZN6LYoBKuWKFrFLec8HyYOBeMIaFrvfTEG8vcJRWIFtB3EhkP22xL2EyLIWSFHYDFst9ocS%2BMHEtG1Zbmr1NFtaeUaIgMRisaAYhopoa1Mw0MR5WwLW5ZnGAtJTVEOQq5T5kBvpdNhwxcwTTMsV5KsXyuJb0FYUKtZsrFRBF20YKcjAXbN1cEAHy1jnhUYPOPMNYWXgEiijEd2VFUYCHsceZaEoRKuysZaoA383LGPgYWxYYR8Qe0UAtwBdZQe0S2l3TS1fGMdCeJcrTm0qgwgOHJ8RbdtiQN5Rorob7iDLCocnRWFGC3s00ESbshWdP6jKNFsC4UrQuW2J0VFxsWaLt8cYlFFl3YFqYDb3l6AibD1VmjUYEOaBYM2dlKlzFNudlFMHJbGBUXcGg5rebT6RIDgoLRh1b1ZaCFrF/oYyFXG8pmBzmHCsa3uUiIUSWLNVNgoXXtzA3DQsrgXDBp73ExDpctgpNF4RnEwN7WGr2La%2BS8MNNg6FsygDLY6gFLsqOKaJld5Pv8AWVZIlyzJMlsoo0GWtKAxig9tupify2J2W02ct1eJX5KkYTT04zWlzBgZBsBXI0s5ZhgIALECxhUNwywICFmlILTeV79oWkCCQIXBWkp8TmWMq0BJyNt6f1KRyXwXLIpsFb4ZUCjAVEKVLC2RfEzqXQyNtsmT1jczkwK68apOcStEX2BJnoJnbADIELUAHKC5VzIixaRpBpDRa78S1mhtDmkBdmDChvBVXkC6rGT4gws2oFPbAacgSmlaaWwq3IucwFjQHAKc5VYsyjZWc7cPLTXkzEG8805urqqMGeMXCzLXU406c9PEsLtpRuwaXLd5MVVTMtcjJTJfFtXU8g%2BSiwtrdZhCVQivIl2Y7AmwdHDeycUwtFbaN5QDFnuw00Ws9muNBtjKztNDq2M5y6JXgAq28tl3Rqpg5KBrUA3XlealhTDhAJEi5BGq7hZCy6htLLtPYLqA6nSclItworzr5hGLSqLZ3ilFXMA1QCNpdCWcdY2woUStG2KFCx1l5iECFSmFtCwUVEVIC2C7JgLS1YVvaOgLQLGzd/MInZRNoRhTi1tldBsOGyjiAstgtS3qsGUYWBQHJqAaNVttWJAwGnc03GKt1a2MijipaWDAdCwrdi0HG%2B5bnIAGTdbLWzxqVLkAs3bCrAhZ3xEDGzOSsXDQLFbqUgxcA0FJbY5CVLhDuwbESpObphYUItYFwOb6H3hoZBVKlSDFpQnzKayVLC4cBd5p6ls40LhRPFWEWmTyQc5wVsQ88QyuwLLN%2Bcqn/Y3Yu8AKUVxw5itqF0XSPChY4AmUs6yBu28VprzggiKm2ukS6jtrmXtdMqF8UXhxR%2BJkwYbdGlCjvbjqUeFWZfdxi0J%2BDkVWGujXzEBDBtG8nALdH5iaNcFTmEHLd33VC29w2GLtB7Lxg1mA4C75ynNGsxunPuiWRERTksSiPYmwUxu105UdDEK%2BLZ2CojRR084hZYAVKAUAIttZjESgIwGi69PCvF3L%2BLHCMKayVxmNEQJQBQAqoPDPcUi%2BRuRDRKCVl1GYV8hUMmaAor4jS4aBMrYOomnFYY654SZ0Q5VhzI11wt7yVon46mK2aIwKqsWK7umaaTWhbshSbeGBbWKVryABdWXqDdNBo766KOl57wHuKsNNhsoIP1hYUnwjQcHLHESoLqm0qQAVg2sqqAatbIy1aVlXiXn1qBJVkKr3lzEC7VOrTTvGHDuOxcCwQKvKtqLxmaBq1YptsV13XxFChUvIXsL39rzEV7EdOcLquZfDkwFZVzbVJ53B65Kpi4NVWn7/ADOFV2%2Bh5DSlA27AYbs1lv8Ac4SVhoGGYxnzxLcHB2lWa5eI6WjV4qhMnG%2BI8pAJmqFeBsSWSchWCq7fBvcqQYL1i3AcLq4jgey7bzahj28wNU3fDPgdmmExMIrBrA8XlNhKY8VQBWGFEKkI4AJNO/iErBRrrNRU5XyQbBdW3k1ZsN38%2BYgwJZzkUYpdhzftGRKkNwS6aw5t0XcMx6uQfA4Bh%2BJgVY2t3sst39YeHWNkt2ZeC3AXF7sfb4Fa4tgK8mvK0qqgQGjm4FrKbJzKFzQC5cmHERl0dym7ioSlNXG9wAF78JOoW2BvliENNZutMsRxReVNQ6WGDYxIw5Ry3H7cimXldBnXN3cpCMuo0UDQhce4BMUKVFTV4DR7S0Awu0xYaD0IGWqYEFyN%2BCEvoOdk1Gv3ljeT4rDKZDQe0z4ZGww23TlMlV3ceFF7uhou9UNCLbyBtsX8jt7SlU05LTd5p7cSrShitmLKBxR3KKN6VZUtd4qDIOSmlAiuChujiFWhcGG3fJdFwTmeHAIm/bXq4oyqreRQSlbxFEZC7KCtWVjMbld7ocAW2VjdaZooaqlGjvK1uEkVACl4%2B1b/AAipp1duKvr2I2VN4aONb96IGqc5ocW6L8eYrkRduPYMXRGjl52bXlqUzErFIIFBVDAb4gOSQi0FUw4FacajyW51kGVo20HImw6mi8GxekL2Zl1snMtUAKjYsITMbJTA6UOmlbMR8tksDTRU1TZfXMyFQCjSwIa6q94qIAdzKhIsyLpth/EEaTkLYNHbmELsOggkYe2YvmD/ADolYx62q3YHWSCZLRL9YHlU2K8czd8AYLiAqKd6qa6FFlGlNClruOsF6owSxz0FMQAtUTsQyyC6i4gbJI5gTxYW0qaOqcWQIUocKU5lBSjZ7n7pYwR0reMmNXWbhuHPSrMORzm4JyF2aaqwlm63Gkb7UKqxo78oqzlYMN0qvHL%2BbhSlorFV9HGfPMAiDV1cmgKHR3CB2QXk4RN6u4cEVi9DnGF1ky5gKwaqGMibBWLNmBL21kK17Vj8QqW7BAdFt1YV3HkWsDgaFEpaxSVL5cFqoUaSshgxk5MUXan9HU0vVW6sv4xcvZcuFcFJb3xEbWtsC3NUYa6zOOhkZ4L8YIpYbDwoDN76I1q7F9G8mBLIDHqqnWuoyhVCrubjUKUtu3eR3arcLm2ULShrqjAqCFQFoA1bwaXeZiGGxAqstjKYBz8RW2xMBBVtVpKYIIWQaguFiEYfbMRjoaIFKEAwa5dR6xG%2BwVWAuYx5maNGjHFdbb4zUEDDpFDAWbHjlmugN0Wx8l3opFh6oQhzZZ5Apdw7jhhkKfAsrF5zNPVZkTMGqMVzjNww0LUgHp0EX8GNMlGioboLXu6uB7FAVYvVuuVfePDVNXbdVnnB94LdpRkotXxxwQFrq2qHaVaX4HUQoGylsZQV2gFzaqMlGcJgcKx7zRSqgr5qsjnHzEcXN1VGDeTjRHEKLwY0UoRLxWeooUAu7jhkp1Uwa7SihLol41jmZha5DJo7LLX6yzFrVijlG65vMs8grJKQ8mVUBqGM8WbWjNZmI4avCTG18hmO8BxehvJwG7ftEXaX1lw0ZHBcQZy8U83rzAyACvC9V9JYi26Mc1fHTmYXOrKvASvDEQwo0mq9%2BDMqeJARB0BouWxQ80loelUpOR/yAygL7rZdhvhy/SW9QmZc2OWBh8/MzboCqrokWvI1TjlxQpOUqmSnJkN2SzrsKOhdtqGI8KAlagKqdSuby54go0atsQOxIFWgToFS9wO07lkptQVoqqyAd%2BYdFRstmW12Toc5mcbW6VaFAGjJiJR0GikCOTezCxdBAApa5y5cm4Kvi5sOVaZC3xKVwSqsQ9w0C%2BWWiTsAjWI0F3cVaAo0HOVoeG%2BYlXulsNqzdYHkiJnCuGkA8lp9yGNJdXmsdts3ZB003eLdubrOmOIZubtMlmtcv7Rtq8P2Z4Sn9Sl5V2ldac6yuPEpLg9mVW3twG5YlV4CWYsBcmKuNzhi0FAKxByHzWIlrXFWgKBRjJeolenRy1VWz4iRQrZZSqbxdu3csci5ArNKi4zw7xEDIpXCLWcVfNRX0pijHGC7q6lQt2uWlzaGCsBcszsU80vs0RFWreDJGrbpzCUsLbw1ixb6gtslLbx71enMEyUmzfMbOjhihoK1iL0Kx8IgsbXLIMtSlo74uXQabYFaq27ZxcVaE3KEPJXFB1nOJjgY6BbZs3kHRk122USlAExSgWsUapiwdkwqruGrXC2muJjOiqARKWCuLyx1orPAUztVxxr5hRQFBcuBC3Q%2BkIWyAFA2oNabruUIiqFr2UhRrWiWZAiYhkC26Y7DmE0INnAYqsLyU1slSUcILcigbt7SmhAZSihpW2RKnKZOzdrZauYshVW25sMpyIzK7htW%2BcIRwxdniI1ZyrRVFuSg4%2B8AFSlta7JSN2Vdf8ijkWrMi4V5FNwLm5YoFUgLbn%2BmDWTaPBykuhC6%2BI7bL5Kum8C2qyfMeBwDsqkq74q40s6ptdNH2rcol8gLu1JGTAXuWVtE7c7MlyvIHfvVc5OItoKTsbKvDCRKQXAvgPDfK5jZM1hoLpjGlnUoLq6VRWWS7iqhWm%2B9eMU/mGImCzPv1eaOYGGEzfRvNU3uFfyD2rKN7CNyrAXihopxdZghNtNhYTjAUHtUNciGQgU0bcBu%2Bo3tXEqPNOzRoR94KRmXh2ayihL01Gg1AAMWxVl1XmquAWYabgx0CtOccwuGFyie1b3nOJecxqBbfCVaLrFwhRZGC6o0UAA8vEAz0BTlygIUX4K7jDQ7dgSq56RhdWItCjsIBjHMMZHjMYS7DN246gIzgJtrGuS5RvL4hVVVNgwMWC7qrMaKs5HPU0JRFADByVj4ySksGSdicmVygClHhcVvotv0iOaKEIXVBS5yWV7R2a0JVLjuBqitXKptnm%2B20p01A0UqFH2FeDBI2jQGNIUK6fhKhtpwylFBszh7JlVsVYpSFK2xfmIoC6BrChUtvm5kZSjdcoo6tCo55mlbNhaoYYZbWNjWrLWO4vUXTgDk2bQ13E1lKKTdZrO6SowMb%2BvFd3m50jRQHGVzdAj4j45OMYvN7XhgLkKxm1qtnbcRWAgtmPLjNrL2UrrjjHUpZIKwXl%2B2Cq0iqdafouoa9zjZhqxegH9QCNc0Kt4qVr3MEAJQjStpTPCuEedRuamwGgLsW0NdfWIINCq6UGlAC6cxwjQQ2Oh0Bx7w6CNrWbWF1jqXqrLWqQdqNPHMzWFCG1ClcJ2IyiFUAAOmwCkOcOWEAIVhKkbrsB9pmLcVNMAspFcwQHeqj5G7N4DUBMNlWFW0sC7gVQAQmiU1c2G5SFaSzpLStXbxzGQDaZMJ1e2GM0m7gvWuzBraUCOaDwUUjS6ErMbyqgAys4bKWFwGYFrmrwnPjtUzhcptBWY4rzBUpLbEN0mM15RGsqw4FKFdWL8Sxv2Oz8U2vWY6CuWq5xlWO6jY24cFvOLzoZU7AwULRXFYXy1Mwttx7UlugsmAbAdvKVecZIt3V82cJgB2CIchyAOrFO0JqAapxrPN22RKpwRybUyituszUZVZShEy28ZgtXpoE3nWd2XLhEFK5NjJPxFUGyJArYlKIkkpa6LI6orL2joJgzCUoLZTAVaIUU3TsQhrFzlh2lqi2qZKqlKvnQxw1YoRe3dIOYFkNpHRwx0eQaF9owW2SeIUUFyCY6uVQCiguw1VMAOHqVkFNp2iIU4OMjxCsWXQBnRUM7cvc4ZI7S7MBBP/ACKEUYRAaB0aZoYtkLm7cAG8bDoJUtgasFBvIuytKH3nCol0tots3T7SjsgEaq2srGj3YyvmSmVtVtIBWDatlAWZ0sGucxSMULAVa3hYUuMqocb1hTi3Ge4SEmsgTVEPIzFVmjgxgQaNs2cxPgIWUEwIQLU5zKsKIKZM2OQEWhblnZV90sL5UlrLcIYL4tCqKvd4fEtHOVjy1elvKQ7KKsDQbyQAoM3lAW3lABfld3HBSuksBEbVgRf3EcLA1Zecao61Cxg3bFgM3eBLvmBaW5sA/KtiZVZWm8IXd8c7JXhRi1CnJXgsdyeKtpTZvGKlF2GlUarT4uUZ6qAwLxbOkMwFBtRSqXJ4jZYRYICGEPBgxnpUCqnpFZYoIgW1hStxu1I5gIt2mRpxjWS0aCKvGsNDbORVoN8NQISozBZYg5ri4jylAUuFBaTWy5xBraBW2iCAcKDUABV/EAWjSkSyJGQgANhSjf0lovAqkUWFIBhbuqgUMwss9uR1XBohUEtTkoUlAdg4SCs6PyVFzoQM4xDWjFDQNXgq%2BIIubUQIeF4rPNy4N3nY5APQ6MOYCptFoVU0KcGlzKdBR0EA8BT9cwqkqbKyOPhNuMzAXQMAWW0M5I0GxQ0AXtkrUqV2KnuWMqDPUcYJUHVLC6WoHiOtRBAoIuHC1WzJLrkFdC9qbILrfDWl8Ps5vmURrrF8pilKyldaiAXO8mi5JxRdEWk6W8t8BYldubmCxvtlRac9hvu43OV0i1Y0jVXBEUVobbHNVeyt3M%2BRQ2c6virBgKEXIC9eV5SJALQQy7vTdan4AAtbz8LIOY5BRDRjCFy8oVLRhmIiohiULk2MbEF04hrv3pDABh7flCpbminyD2NN5xAiUWllUOLLgNdb8StLoLutFFnou85WJuTsrse2MANJGCAeJQgLYvG/aWNDSU81CIM0avEtvKt0m1T0vhauqhcoOWhUKD2HTzH3DWC84lFwOKrR3csUKght5qjTNNB3EpmigKFiMxqst7cx6UbKDlXuIhxH0qG2mQtAzY8bPmWhWSWg46N2WJE5B1jCcqLTt4lBpUsNxAkpx4ZkMlVd0pQB4HEAVVCoQKjZbu0Yuo6tUxEJ9yKyO85mHFiaFoVk00/1CFgVVOKWBYdj7TKuWqCaKrguwl6/2m2wvLqcxYCzGVmCgWFUpzUL8BYOAOLqy0V4BTSyrWAtSoUylsAp2lM5ruAzt%2BS6BXhxcBGlPZVAVSPNu5avIra6pFFJVlIbRaABo%2BSProavFnF10%2BIo27ODIOWQbBIrUKXhlZd0FquY8z5BalyAUB6mtUADc21aZ8xNnFaAu/wOo7sL2WDgS7WGqhmssOAkAX149pUvhu6xlqdKbG9MB%2BgkFCLGUbxCuIAWHY8QMXVrOel8xAU5Fiu0FGtH4l06QIo2yntGu/iAFgwsWqreV2aqniCAsMAAIoojTcVNAiz4ygppaAx5gKMYqar5Aq2lG7gWeVWiBcLdkzyajvnAGACzrk5J7wj1FhFRsBU1VoFsUAS6l3Aotim1byVzFFN4mhwbFsrxeOZYQoAqGrmECU5qWtClKWXm1l7eS6IZC8DjmDqAq%2BJmFTbVaAR7reCBvpMnaoSvAr66lU1DDDYUFdLvC7l4TKIhIEG8XyMRCKqtiKjdg2LpUVGiDICKKErRUxMzBAFqyPlNvEM8ighajQc6Hp1LIjw5DhSmttZmyzIKrWNGKZSEUL1q0CnIexvUsdEGuBVAmjOStEFK3AEs2TDyv7iLTaLcEacPL7xmp0B4SlG6yMQSpgba5yHeziGG5DHlpzd1LGLeFLLtFFaexBUnxt8lbNsmfeFbglVi0oyIs3zOWJsqsFFhyseIMtjWwIKaGNMdStbG10S2cTl4Ry0/lmwovriZiKZEXtssLCuYsJJbyAOPkrnUA3F6LcCKZYMXzxArjVq12GlV3uIKGzglGlF0Y1ermpUptogtFHGtVLibZoUW2ZZTGNdzUTJtV0UIRG88QiKlg1owsBRDY92QMpAtQBk8BAbxL4GzgUA2mQlA0mJTs0JgwZMOCvGIxSyFW1dAnk79pS8AwoohFwbtPYhna2iDYK1bvjglYOoFGK0tCFGBxFsIShKQlhS5S95lxQTYByoKvKu2AvZRapQyA9ZrzAQCtHDRoNU2YUwZSg0bKfu9AKoQ1TlFnMqHQtUKCaZygc7eWat2nMZAcgVyo25g1TRUYwW5MGBX1lCCq9cBR0ILVcBB1Cvhq05OlmMoKVbuLVKpLK3dy4CzAuBto6aDxAqFU2FwKoDLG9QGqj0s2qzTb6wV58EyYwG9c4l5SzRfo2ez5g0OCoGsNlN1T7S2vFUS3ash08GCtQQZ6XoCxSvdjhwytK2crLT1dEwLmEYC0pAqzVjlbBdUKtWtuSV7TMWVFdS1dqoFZUuaKIWh7%2By08iM6gLWSsyVgpzKrIMWlolq4N7l4VBkBu5NbMYhJtZRpMqgOAjXxC%2BeyZVVkUasf0liolEOKhVWGQVRqPNtUpAjfAoocGWEmhWOSi0c0yVxCq0C5HHBEcLfGIEHQAloBgKGw2tzIbLAbYFDdFjKZl5DPLAthB8RlCaJlFkQOXSYYXZgW2CrYiVYKqAJjy1QFGhnwiAoCpQBdpgw5czKCpKcg1at4C6ywFNiV0WqAFAbvcIFuwRXEomNtAsp1sxbooWtFuOyitbocOM4p2pjwRAtAgOCtKuzdlsHEUcMhcDkJlCnRdnIOMr%2BY8/RlBCvQwUI9xbEpHZaFmVYvxNUmdULyNWwV9TL%2BYj3EXp2GVgloDO1AUKAsqFCpteDEQKr4qOJsMnIxacEuJCqEYA2WKYS9QZSqTaBzb2CKtohRBk2QvlL%2B6UrKsAC8BaIRiClGwWBMlaPeKZpvF1Zl5NqlnqMLZwMb0wt0W3cs76CjDYFOcj9Y7nKUqxzrKk7HYx2huc6LXsJPIgMOUzbhOFtb3HKIpDFSurDaxfaGXoC6OAKgy5q4JqtgKpa3aFVZuHWhThkFVrixyfSXi1dptrayo2rFsSJEWGEporbTq47ANWyDeTbgPl3G5zcEsAMpXQzDPWmbAsnRyanNAWCG7NFmXxcAXVyNLSBSOWTiKssBAYgWODzBMNLgcADVU6xLBnINlByUDkdVmKAlcKCBbZsMveJVfbcVEyaNe9QJT9VdAWyEanbogAhdbEZWq7F5jc5WwbxQ0C96Y1VBdgU6rgcCWMzGBZVqiiA3ZalVkzhSwZ0p%2BWZUkTfiqqXeH1jjEl7Wpd5OV58QtOxQWgnv2JUDJQtoaWlSqSmxqK%2BqqolFMC2pdTDe0fALOHA98x6gyS3s55AoqOoFFA0tcioRS5IbYtBR3ChZQFY0WciqMILbttQK5wWy%2BJgYISCC%2BXBZnmMKrBrgCCDC2bmkociYs0m3CjqZ4paYAZ%2B1smq4ltgG6B3esdmplJ10kCbAFDCdzqrPpUrhfaUAig6QpeLiUvGmJqrYaGMBdi3NLgDKUsRcqbd5gkxuVq7DpW0D3RVNaoDLQIRWcc1KG0KKHMMS2n/kWy1i9a1Wa9KzHVaozap4tYT4S4yyh042duNVvMa5Si1qVCW8vYLJeOQ01WUtuQ8aIC0idqOjhSKJVHCN5AJtV0njLAxR5qtrwig4g9bgLlEXRj1W0CkVx4Djby%2BBkgorTYEmFlzjl5lUbBXI2lAUBdvOJlaLbtvanSxqBKqcAeIBaK4gQAMwwDteBG7lRoGqKsL0UEcdysbiWwi6hWzNwsCZbBDK2AumuIphazZYq3QTI1ZEkbVdbm7SzdOjMIAa2YDQDkF9/if/2gAIAQIDAT8QMz/hAFuSGDA9fb8TvHTmFlCNZ4Rc8vzc3Rt4lopKa3DJGhj5F/3FJe7Cli89xa%2BZnZWcy27cw2xXMRA0DmWGDUKLy03%2B5lBbwjWOB3/pMyWXiVFji3cSN5v1%2BI8rEijB4TBtzG3K0BGzz5i5Wq/cbQAFA8ZjqEpY5QXNPQEMOMmfmY0BTeoVi8OvxESmh/OZlDBFqDk/XrmVYqhODaISuEpF5hYMCU3xhitHrywbYdvtEtRFgF5xyaDFLtMQMObXj%2B/aLYUC3eIYFFmhWh9rq4gaakyKcVpYNYY8/wCbLi6AB833f/JhL4VY53dv%2BcTDXfbdP4yff4mB0PjhrnuCSxcVSPz9u%2B4FMmO8tf1KAWF3j6S25Mvt/wBg5P8AsEQDpYAByhZYQWRgTK0mzRv19I3i/TM2XY/cLgriOXQO48Uu0drHEd4249airdrK9XMqtGUF%2B8eYsUb8agcOK/qWiysZgyRNRooziULig3WXXcWzhFqlz%2B4lo65HBzk/UAXA%2Bld9nwRqMks/pz9ZYK085r26lAtAW%2BK9yGYW%2B8rSrR%2Bo/mZCklSx9ndx1DGvWoZhVXFgb0PjzKsoLu6yOqeyEKwfAVnFv1xUQIxGPf0cQVsGGq/fxNAM3/2AHh%2BYKrWh%2B0A0IZ1KDdUZyP1ltjd39ZRV4hslHIjsG7iUOM4%2BIUrbziNAC7evtMCN%2BJdIDB5irZ4pSCBosr63MFOPPmUYLg67nFFyLQFh9pYXMkEu31g2HNm%2Bv93MKVLeM9bv/kfApit2u7/q5WbaN/r8EOk38RV2cvol1AUGq6loA9n3myLMnqo3pS/Xoha7HkP%2BV7/aVmWkw/1xEK0a/wBlxY6V7aYXPRov7/3ET6WMuL38Pn4hAKroeKMl/fcqBs5%2BepQUulQdU2n3gbQYqlxeuZaF9kFwC9poFkeZywBxMBZmF2UzErHoYqZd/aZaOoKNm5SmxVQV8n4mhed/9l3Viz6fpB8MTanMbSXiAwoCBv1fMoUZVGAvO%2B/9gWhprS37ThH95t2U5fmWUHnUSMt1iP8AiAYWxRdarHvcVGshjo%2BYhimvcggQ17YlBwcYmIHu4%2B0ctU8z7MKRYjCccQK1BKv83KlBsuTOR8Y1MAtWj3Tw7qJW230QwIg5KyQdFrlKraWgAzUuyq4iUXXvnVTIPOiZrhl2koT7wLCsoGVuSNka8RQda9VEq2jcBbT7y8Je5hbV%2BuZY5ZPVf7Cnd84hghuTCNu0FX4hFPIO%2BuXidECJwP8AcyL5uNUVJKTR9V/UWqZ9fiILmKOh9JRArhz4366jWpEUxvrP66it4rjM1Fgz0xOCdxCQvw4lwjYHiBF0Xn1%2BJYEGnuPQcQl5fNdu6OIvBy1wcQRob/M2OaBhZfK5xdtLmFbOWA063mGBNaJQWW5m655hSl/yAAGj8xqgOWPnH%2BfxAcjCqWsvtFWiXKFKzL3aD6/rzG%2BHLReb7Dx51L3unF/j/YYpwZ8etQamQ%2B/%2B/iUovP1z56lVYWIW3VfpA5QvmQ9UXx5IMo6EacKs%2B/3lJbqVavbrV%2B8HnDy/iNgZj6drFbnN%2BueYmDe2Djhqk/Hj6RhQg/VMLgcV%2Bf7gYzLpu97H1jcwsMob3vr1qK9Le3j8xDWys6MX8x0C/EwVdy7ZZ6gAxbMtDxLWWVGV9jn/AJEyuGpyMnVfmYFPaZNGjJ/2ZDfD13Mgvv1/stS5XpliOsvms1%2B/MYNh7x9QOL5gFOsWAFZ8evaWJQ9NZlHVn1/2KEpO2PQS2i4L9JSaO4DG81KK0CIfievtK9RH0iiO1559dQmKpvHWPxEFiqfnmYYK1nfwc%2B8EoHQ1Z9fpLWP78p58/qFZZ06mQyhviuJXdSYSTZeB57PxuooRqZ0V10jTXfMzLSPNy8kKgDK5XcaR0fWZgDJ6/EzZygRavMUKodoKyl9Shuxktr/gnJc7r4iXW5dZ7f8AfrEWLdp%2Btc4iVXMnlr%2B%2BXmYZe3t4lQAAh2itQy9DxuDDPHULUHTuyXoXHF%2BuouiNeiWgHDwxWg0Nv01Kysj9/b0xLwBr4%2BJaziPPr/YaRXn%2B/XUUC4XrF/uYbYuUvPx/UKuQrzADleLjVBL/AB4iQZXfB57/ADEAcnn/AJMhkF9I29gcKav715YPOfp7xOG1M%2BPRFPZGpYeImxrP6iFxmOW1bUNB%2BqLZOKzFsaZjK3TKA8u/EyNnwli6%2BHmUAn0eIjhbwePfz%2BYmTdjt%2BsZYDK5fvrqCat5PrnrqNd0x9fmKsGU8a/r8xyznjrPvL59mPzxNBHhuMZBr4r%2Bo5oOte3HEBRg8PPF/1EajWsa5faJR0cOTxDyNXCdnf/NxCMWw3xzNkU2o5lgpirPfxKBrz18dHvzGKejw%2B/mCpRitRmzN%2BMf96iAoUq%2ByKTYv1qVbhTHuagiy%2Bb%2BM14eJbWcTBeooGS4ABimuYQtvT6IoRRphwluyUx2P6iNFcvxCiFZu%2B34l7DFoGgwEBohQ3fGPeIe7vIKZ%2B2vMEs4Fo/tINeRz148wJ8I9cwgudb8/3DRkLp/3uU6KD1xDGocPB3Fabh2%2BeAOqilR8uJeqKO6z/REEXGz9%2BYhSbfH3s4l78Ab3i/p%2B%2B4HPNY1zEgvT6xYQamcu8ev%2BSx3xl5Mah5W%2BGhv219Iys05zkvjzAUU1b7%2BsQBYzYjr6RGIVvms%2BJRvUme81LElus9ZamGjq/wBRJaXX1gRh1GsGafr9ZQ4S9pNlHFn24g5AZv6zCDXMRZ2gFqxfrMC1TARuVzwguzgu/wDI1tpMXxv6X56uBm90oML/AB4hal%2BPEBMBX1lYQdPTGvnfL1qKHGX379eJrI/LEQpQ7Ds9cyrFM%2BPtAVFXD8vrGZlW1p5lGcDh9bl6Gzz5h8wDjGvPcSCE7f6zP1zBKDiqTv8A7MucVk39feXalU4zf/SIDOS468%2BtQML2gs3ketRVi%2BP6ioTwuHhafXEGC7z8U78waBgSe%2BppN156ImIskyrotEb4XBXJl9fclq8vTKAb8/mYEFVAc6H9MTZRLYOQoBznz7QAhBnd44CDviufpAVGeDxAewxuXLTbJ7REpna%2BP038T2lfuUi8m%2BHEUs%2BflgHSKz4694IG1/UroZZX858xdlTL8f8AZZMeTuUCCEptm7gzxXcyERTuvXmNYSzviKiVrfsR7pTdL48sRHTVGj%2B2IAjg5%2BuPpBEFJuaI1j6wGGm/MtCBe2HvzGbDyeubhbi9xrbxLzay6oKv7QLtcmq7g2z3HscPXEQK5iUtFKiHDb3DQuU3LLM3pnp/cGkFvfr/AGWCwp92ZocrMkOx7hYp2/yFBC1f3tjnRQjWbltMupkGDXv4gGppTrmVUt8eJaJT/VwfJiq7zFACvBRjzqZ0cscdQRTRjwf3AbFtavkgNJS%2BeGGvqZS6WPXu%2B0ybmL62d168y43A9VCPMXteMX%2BJVcwgXjOsRaVVj36igwRqvr6YbCVSzxLyASmfLnWsS4upVBWBLwvpMLat9fiYPb0Q4pyalZtlsKIttERbx9IjvHhzMJmt57jZkqtzFUCgueW8nPUu01d/1AS6Dj17wweU57f6iMYMvrUQbtOvWYtKVmY%2B8ojJ%2Be8yt2uRriGh3d8PvDjWzviggGAo%2BZZxiYP7/RKqxVb89wImXB8RZvg/2cYcZ6jDBfVV63uIvKxiZrR1634iixZGvPrcEhMlvH4uYIADfvEUZMHWfMdKBb7nmJSzbrHMxb/hGNQuB2NMMmj%2B5oobMdS6o58f3EWDn7zLwBx5gZVc/MoJQM7595TQcqiHgGOriz/pETBgxh48ykKy9vmXcyUx1vPtmmJoXPr7RApw4gBXoIswb5Tk8e39xUt4mK4/z3j0Irq8FfEWrGDjf1gjQRx36%2BsebUWoAYWfX/YGsQF%2BZQSleM11K0ODq6cwMJCaBtvqvLLRZyYslIKXbmERoK9H%2By4I0q2df47hqmSRay1wxt5QVj6fPPE2MymLz8wknrefaWTdCVgNlfDuAIrD6fqoDJYItcK8S14L2RFX23DDt%2B5YFsnr4gb0Xr0TK3DzLsaflMM4u9/EsAs07%2BkLl7iq6DbPHt/UFXQWPXxBcJwfeXZry%2BnXriZp3VuvfEsthefjxCskp9pvCwvxfGYYJhNf7fohcAX%2BCUJzw931cwaafvv%2BoPWrjuNmqft/yFsEePHt31Mqrgr/AB94o6GCf3B5Lfp7XLMDXjxf4ja0HzU%2BYPpFa86H/PeZqbq97%2BIj48nhlVIiqr7xWdb1/X9SvXD%2Bf3FZS8l9uPb9yiU2X68QAdJjLYRUFbXfxCm3A4/f3nOXNxypKT5lBOHr/sbJHJcspaWevaCm8/qBsQHjmJYMrhOTz1mOC0NF3e%2BIDaLg0uOfXmPDq%2B068fuCkPCF4UCvfe/aC2OD7wIwE%2B2pXAiYqvn5xLIwatr1cejDP06v1UZRsOHfvGgjw%2BswzMLZ8VLQDnH%2BkzICxj45gb11rryM7Fm%2B/mAJbxZh94uGsrH4hthj8X13qILO5nx7TIszofbHrcd0ZxWee/1EpVF1nfxGLqKnyI%2BPbmBSoFXRuU8JctSpVyhFr37hVhWb9fMFXPv9ZUDNiDa6iqjeI0%2BSCJ0K2TrNa5y9dQsjg89eupYy7I9FW5dKQYr39otLBzUsk0%2B/x/WIEW989H%2BfiCdqqHHEUlRWv%2BObIpYeD1n2gV6MfdlVMDg5PeAKBR9VEghypVbCIQt69cxlLnd%2BvERHZXh5PeL2MLb9b%2BYUdmbo3%2BY4S632dfNxbEtHAOOTywaDDdCey3v2IeVdv4PtFhxWbITHIw%2BupQDn64lKbge39y%2B5HDnEpfqbYtpmQGHr7y2gbt9HmW3bVPiWWLDr9RtvPn/soRoYqhyRaDUajSY63vxFaXxL5Qq5dqb/ANr2m2f5Lh2zLrSWPWH6yzCKPiuSnG/pxMZ5qG/f2frKgCFVo3%2Bo1iF1zxrzuH3WdPqiGChm4Zi9vFQrZ9jHzE7He/XEXgN/mV8uU38fiNlLHHJv7O9XBlbz1vVRC1VMVQylCjNc/b%2B4wwXWLMYz9ZWQKZDhrZy8wqudvIPPnzGpFmXJ6/yPnz6rqcxEYuvv/amcQ4gkhsphkPslhagyrdifSKvXL6%2B8RhX1llVgr%2B/%2BRUZL6JeDPwgpnHtLi8Hf%2BRnvvxv8f7BtdI%2BIZBxX1liyX18wVvwD09%2B82232q/pzLXPLWyINnL9P%2BwVt9rv3izeBnZacRd7HrMEUKlL979XEzoKa/wB98wuNyedb/ufBKPfz/XEvLNG6%2BhK229KcdfVjMARQ%2BCLELg3dl/TuOBKK/TmZOjxrz3L5L2evMV2LK%2BfbibCXOvt7MuW4bRjucYK1XrEJNpJex6qYvCCjHPseaxqHCombx588JANKw6lCuVRDfLVYhYgYCcUsyjV1VcevvKPhUUgm779rqOFoxQVkhArSvNL/AMiBFFRDJVBtbxlfaae7dV4P69oIBszAajlVxQUfdF931/Uxri3PT%2B/pKS5BRgeOqiUKhTwfav3FFipgTONx66xtVNezWfaKe8sdppxASnyGDz4jXmU86gUynxg9fiOA1dvPXdQF1p9wP6lay8vXtuP2OJxeaPT%2BZSrxTr7e0sQdij6r%2BNRNzjnVeItAXg4oVs8VE%2BTpfX54jZyqma19Y6O1Cnxqv3EuEHPOImVINne856cTLZTqhwKjeVaOIY2UB95jiV5ftKVL4PEew5qoinDEDYQRWy9TBGAMb5gM4cHdWEN9my/fnziLeIG3Ho6lZ9HPP7iLVtV9OPaWB5%2BnyyvIdofavtcPVUTSnPAcrzFsFKRyjRZn6IRk5%2BdX9Gz7QQ/cVXurwYXgckcoMiHYtvz5PmbTtznk5%2BZWvYNVruUwa2fd8MunAZ9eIFEwcAI%2B8K1q3PPX/ZS4ByNNZft3D/DSjPuvcK4UmPqNcvNc9xSdI1q8LzZKsjZ2VnJ0eIlnGqDxXWnvQRsxkNY1DUcDx9Yyypdn0PXMLdqgNwKg24bpQz4zzLRoqJzaWzN5195s3qWbZ69fWLesVXPMoW5PqpYp/qllhu45ELd5gKFldzIDmWFdVcAd23t5TxRdTE1L8DXomRgnLZr16Y92K4NwSDmzfXrUEaho4/7D7unF8Pjhr%2BmGCFYFPt8wN6XwnnNYiequym6OKceZR1QrdSsBvF8vtUEhpqA7DHJqLe17%2BsTHQXiu/B5m%2BUHxdQWTkvP7hsrp9uoNczhabr7OvpUvqgMOB7fHmGaXO6bPevfHct4lUT2hTrs%2BuotqDRgM4/SWRvu/I8ETKC3jG74xr3Yjbff64v1qKkcyvOY0oB64RrN9TWKi/DK/GNdxoRNGCAd4uJVW1iXTjUypeKYJcMHMRCLDP%2BQRFK%2BiUOuRLAQ309TTgHxHhXBCFLUOM00e9XUXclIJROFd83CKOLXMqkOX9RIjWXsnoiDMDfUFLwiX%2BpSUWFmX8EKgSmv%2BZnEBWnD5%2BsS0VvHPuxi4l/b55lCi1DHGfX1lgElFf1FWKYP9lAmw13Z%2Bo1k%2BT9695a27c5v2jlH3ahg3YxvdfeWMwm/pbcqm7dzXnzM6Nl4dYv6QjQVwZycv/YTbvannv9zNGnHK8vpHKXA2iqqJThp83BMBzFxaWQVO4iN0%2Bydghb6Zlgqy8/EtvJhuDtwxMK4nd4NfWn%2BoQ2wNJwe0QVCChnAJ6uBC6V9YgXKN9Xfr2lmEodc%2Bz3Fg3VcxSLYb34ipVv8ArpdwIC45zjj13D3E/PmHtVY87jOkkriZF5rr7RUFnpWa89xYOPx57hqWaH6ezU5ldv7%2BsvGAODN33j0x86zr11Ghx/H19pUAL%2B/%2B/fuYF3Z7jUbpbNfP/JVQsPZn2hcKA29tfoiwKLazr1ruXOLiOxqx8YiaowIGBcz5nUArnBCmNc/mUdKbi2aGlfeF0XnbAFGKYgFBqBdemX9x80mIBTG1Ccqv1GwJwMBxzT1r8SwEs8Sl20a9PiUMQ%2B/067gFYKlNY/r3nnX7YmeBtdGtQhxWDxfZFssxi4XABauPeCapZ5/5NmKps4qDBHK5f1LGnh44gcgVv/faYIBZ754%2BH7RoBNuPHvLyCqc/ggPLMS9XHzWAHF/SDwrNc%2BO2Kj/o5mVEcYy/EIb%2Bxrz7QUUoXsN2Hu/iNVXrMCkVj9yqoeIENWsTJV1DGTN/EoOEphJbzCrGzzEqphXn/TMoR0VxKB2jbh7zVjVHd4fMoBAXnHxiPGZQ57SVr2p%2Bo9ncOQF/n29oBVa49/XxC7UT7PvMS3bXt33KzhoevVxHXXOOuPaOleF9VAqBcFX5iqam/qdf1KfQDv4gFXAek6ikv%2Bj/ALCFK3r4IKTtNr7ar8VNlyo737sEyinjgjUNfqvV1zDkEKOp54KKx/2XUbrv1%2BJzhF96rHtmZHWGxvFw/MA%2BSMmK9Iu8bfuJb/ZhveI2UBioUqyK3LvbfrmORd3ECxt8QyLuvvEIpxlAHCBi/wA/3KDFaDwMp%2BTzDpqNYvJn2qW4hYUVrjywJQrPXoiL4z3XyzHjYmPz/sMBVV9/NaIkNF%2BswpBdl3leYJZpfn7XE5tWZ59MzAzgFEdCyIfX6lT/ALGCrKcX%2BqnuTOYqs2uQ68keCLlbtNVEBbNN9%2B8w7VXjs/UMbwU3szG5Q/MUgDs33zLyuD%2BOblgbGR79X1WYmte86pwDyKl%2BjCxVf1POn1%2BYPCvMKwGeJY1bUzzeTv3gKvRi5gAFQsbG2pZ0My2EAQtdFhjMeR/UoIaqW3Yz30QRqlDZt7B5Lw6mvAXR7azHI10O/eYUC7/PV9MZSu/YxAoNPZywn19fuE3ZkrJrz63EPvOqm5KWv6YRBS4%2BPEZYVEc9GxPPctakXvzeoWm6fG9fiINGvrcvVwj15r5mJwJ8ywsCgXv0wKXkx3MrytxGLsM82%2BIGgYOPfn2iNguc4f8AJYYi1WegXQ87lJgwcHb2vK7gm7z/AHOzcyBRcCwiyhGmNhWnMsyOYAAGf7m5ohlbTHCzTEqUrPERwrTqFZDUNRmFK2y7i60zwwdlh%2BZmgc/X1%2BoWqUWHo9eI6hG8tas4%2BZUWFHF4dcRBYzfof1MEUlnnRzLmgUhv3iQ2yHk8etRWoeFVS%2B5TC7bq%2Bvv48xkWRcZ%2BkSkhz2efme%2BAKW%2BfBej2uEwH6VnfcDQtTb2ShEUVe/V8HiDopCVku/X/AGWXp365iShveZcQbL820SoBwT4riatw1%2BY8oZuItb/qCzNkunUwvLr9QJkU9wQukYCV5jpysU2MPMU6ICLcPX3m04fmJ7H4ibNPf9xC1i3Bm6Mj39dx3we/rmMUgHDw/EwFWPVetTFbX08/qD0yvMGwb9yOrRRXq/Mu%2BQ5OvH7jlDLGu/F%2BzMVTHnD9IEAuvjPXt%2BIbgGg4x1HKF3r2iVa6XGq4ggwKmnu7maDt9ItSNOPPiFK1/n9xqpqlqcv494AxkPah5gGRZr7QaAPXEpQjiWG2f7hmyW36JQqpUVXUMhTDMmTk4/2G1NZmKtwpMUNnX/Y%2B4RitxUvJcrV259fiXUDBLKGhC2NgT6x8VU0D44gQF5i/P%2BS2rt2dn6mD2Ht4YmNc%2BvrLCw4xXj/Ia7oT3hpTIPs9QLnWueMZyYgL19seYLK3v%2Bg6jqGm3H9VKXF0Zw4%2Bn6lcatM/vfMEot625ggiAPDnj0xpalXrdkpkzvXf/IHFQZA57mrGz2iVZ/AOv3F03Nlpsl5w7gWUvfmAbHvAckD7Uai0K3UujOyVkr5lKNpn7stzZtiVbhiAIu6QmaMQG6XBzHBWEypFM6llCNRaLParfoy8R0cENqVh6uPYsHw7/wCQUm6vHt6%2BsDdl5zjHUqE7dZnQrB859VBtqE/5LhYDj9d/Ep2rGa%2B1xcKMlvdufaOUvEfvKSBo28RRUsDXrdSjhTR1j/pEWVWuu%2Bv9gaIc9ue/W4RYXVD1Lq2u88viKsqpw5f6xKCtxQBCsbrwiu6LD9%2BuYHjEGC3A/E4AU%2BteJaAyPom16lCryXzHGEzEootyhpycSl4V6zKNOUooviBSscM76ggGD9lp8MNU5fvz7e0JtJX4e3cxtvD97ixbed8/7BQs3XfruLf4M7B4NZ8dTi/TPPn1uUCYL1ivXEaxM1njmAVATFGc5%2Bn4glBeWPHqouIzOH/kdK0NXhgAnhX9vvEInQdm4r0219jzC5eB/OJQ0HDH5JhPH1XevqTBeyJkQXUu1BV/eGkrLPF3FVnPUBHlIq3A/wC/1HI57jcnKWVS0GvXUvCZqYKKb8%2BIoCw0aCIWt4gobSUlMpQqmFngT54gxooJjjXn5l2JAYDStsqutbIQuedb/wBgj2Y3hvURYvbV5N3465icK/p76%2B8pB8K2PXmXUjY9fl3UpPH/AMv1zEUIba59eIILFltP09eI31pceP8AYLd826PjUZICvr1cdMS0bOnsh0WymF7gryXXo9agHCl8LFvruCYDDBXjExwyrRox79ymyn3/AMll0Dh11MAi3ChW3MQ8OyZp6%2BIlWXzDedzZd%2B025KsHeX8S5QairmrzKN1mAoYjdNbjyWtioYLjmVev595hQzcf3BVLe71xFHN2Zb8HiXKyin67a%2B9RoFiyNA9n2TzMCEPRFQVnfJXT55ZRKVTP7ON5YZkFaXl8xezrp3eZWj8HjUvcD7nxHZko9U1OkXXOe5SooC739IQBx98%2BqirbRzXNe%2BmXtZZCgpa/rBfAq4rTRmFun0nkVX4qasWkoAUyXDQ2SbRc9etQObmNrtMbMkVUXTENBxGha1A2rr9MTdXFtycQvYMzMJ3B/IUjzLCLYvh78kvjb2P1%2BZqRlj9/MvIFvL2S5UeY194stdbvZvPzzBxsE3/XvB7ZLwPEugSvpjxcGt8vOb1xFA4Vu%2Buo3NAvt413FQzy5H9/Ms0NN4MY%2BGEAr4MyirLVw5x68QvSFUd%2BuY7qBVRosuUuG6gYLzBQqnyiyIAySuZYMJhmZEnmOAvr3iWGsynLSWD7PRFfZ9/iW23qtwFqMvXrccpdUQpL2uKUriUVXMUmXEfRqYrzAWuP1LPxD2CZ%2B%2BICVFvTLLoeHrllljVPuRQCuHoggeBr3PEsaqnrPn/sQxFufx6/MA7LvxdzMQPJ/wAYACilXY/ZPMC88tte%2BoEp17dRIUFVjwx1bJX3iHHtUPpcqyww8QUrWpihZzUZSmWUObshd76SlrGfpEyU1Nc49o4c4qJdXbX4muXcwjWIvJeOI1u7lm9v75iVQmH3gpambnBEpFtqFJQQ0c4lrdPMG8GHmMVtT%2BoPz5gF1yHJ0nUNVYuzXGfvHUXQ11d0fiFpvLnr6RTdZ34vqbTKJvXqtYlnoP1UUazOW8waY0WdJGIAoVX6I5xHXftj3mA2l3VJnNn4gKqs2efVznI9f1OTDNf8gWuX8SAnlUAiC/EtpcLSXjH06lUSLkjB944WjK52c39p8EsHGNwEx7pQ5Nw11mo7a1LHTA6Hn0SyBbYBkuYq8zzeZVFMFipKBaWT6EAZrhn9n4iuMe/TvcDXQSJXBpxGRzOG9/H6gIwfTiCgwee/6iUBa16JVzYeeNMsl6PD6BFTtW6OuYVWhG8Zv4iFCdsH292Hxoe/cA919nzGGlf1lIYUP9glGKH3MP4hRTFMLzBxWIBam4HLVhsjgHYltLzRviIsXm/xMZRSvvA7OV/3NqVK9bl1tNzDh1nMNDNQvLeJah1AM5hVq9TKeCY2wrdmJl7x2xbbr6H7mrZYOK51iWxsGNu31qVi2vcZUUWwrn3mK66tz6YgRVo%2Bv9ijTFe1wEAFh6qLIoz9fPxAXecp7J%2BHmA2Ywfq4w1%2BYYPPx4mdpWlf1AAtl71BKw/UwfWCXm60%2BOPtNB0G6iXVcSrdNQKFvFxALM1LEFflLLRz6zCUMmmWlUsZKKXr3jjbTALXeZhG9EsPQltt89TDHcW3MTNWjdtSgF1E83iLGcFwzQKHBO68Yo7l1CB1v6/MBSmmv2whJza39fMoW/s1%2BpvHh%2Bv8AyDwr5Ik2E1vgr8xqoqu/W7mqq7dXd%2B39xejU756%2BkeKFcfi44xzd3365mRU8ZSQbtznrmOuDIv17wlOTv0zFlcKobKTs4YCIZk5e0ARX7mKKBwePNyj3AllAxiBolNTCBeHMwZFnr9SlBbFyhdbq/X9xAXjLOB9oFX7TJlrqYsDzENtQDUB8iViyMeIK006%2BkGTBXL6qaglFOfERtVesWYmyCnvZ1CtOF%2BI7Iz00X6ucmRzcGR%2BiaAlTP07YiIo9PpEG3j/cxEqzivrFlq5vqWoGl/7%2BYFTXFnruAKaXD%2BnxzAHF6OYAe77cQDWgn33XeOYiH25%2BPbwx%2BA3y/qAdsWPatnu3F0py9XFg3lhurMwaKOf1FA17V/UpcHR8TKuL1UaUJxAfOCZpOSLnwsvIjPJEWeZaophq2cRpXkwWI2ImS%2Bl7g6n1V94CAYXM0ThZ7%2BWMyFCyX9opTFevVTc8X1Upb1SteuZbSrl3647lBsjovx%2BooC0OX19I3oe445zDoWvzAjIYPMHdFZH/AGXVMpvHPco5Eu3vLx%2B5ewf%2BS4d20%2B3F8eSIgMLernTo4hbgZUxx/wBZgpQXegMff4liD7%2BA/VuIYxUCrbsm8W8oFywoUlmahSXcxVOv73FeDmKMtMHFGyIpFzCoso/L7SjkPlioMZr1cE54hlqmZdQv7GjJ5xxGaW8qfb39dStgqK435im7WaqOkMht5vWpa%2BL1f3%2BGVl0s08x4/Hrn%2BpULrwtuLaQrt19O4zeQVZUuXGjWOrIE2Xn18ylMuEwn394BgdVzMwti7lRKodcxCALdc36YEAK3xqNBdp23DKgJbWPrAQFUxjESJBC7puPdS3xObSmotb3GzBFSbDf6jxtQeDEEYPq4gBZ4/wBluQ3ESuRgYm1Elhorj1cDYy1tjWq9dzMVzcFCgaZlMmMSqrt/hxD%2BFr114e%2BoqEmnXz61LqdD/wBJSxeCXCi81%2BYq0ZyHq%2B4GTWKp8/8AY6I8XZz/AFE0FffftUpsab0Dnx7xaBpx1WoFW1y6D7Q0QP46/EKVtun0r18xYDf/AH4mQGr9f9i%2BkfaWNhA%2BfmJZZsr3iIKuiq/rv9Sl5Qdu3gM8RUcBTYXn2ta8YiE1xKH0eZitlcfdiGTuAg0WMp6FrmALBkfSupyFY7/MpivhCyMergSw%2BkyqgevaBZWKmQc3KFVlGJRFq7YrEkCwG95Q%2BxWYbuKY9nT7a1xMgSW17/aDTZVp8Y%2B%2Bo4XmY%2BGWdlX98zIxhPPPXr4hwo4xu/zFCLhPn4iUwoVsykzV0aK/uVHuyVKIUHnv2X8Qq7217hMy7VBsKGdnu%2BfaZ0EG9SjGvo%2BrDUBvrj5%2BIlLEBlXR%2B4kEQPAbPJ2IAEOEYA2i0QycD%2Bdy1AMsxRdtF7j87iKMFRgzlQGldxocLjOsVKL3h/MMo%2BhxMhXh/cZwzjiW3bu9RXYxiBLERNVMwoa%2BHocXt%2BZWs3w/D9G4j3CNwd/63Hh3YO3usQ4GvSd%2Bu4A7ZHnHrxKt2wBrzDFR8PXUYS5A/wAltoBuKgYLjgeesRVLWM8%2BvP3ldisN65gLtRqvXEbYgGVXG/H0xKAwiBwca8uX6ShKbgYAuJV0tRQOI1Z9PeIBVFmoKzx38S%2BNW3LD4XC5vdY94doK%2B8vajyeGMBpMQAIU3LHLDPxCw5KYAUXd95gK1bBAwX69YmyVX6/qEmJuHJTiKuXCtJ66O71KBxv3%2BP8AY41pnZhjuB6cw6O2t8cHvmUIODfuQVFLbz5PNQNenPXv9Yhh54%2BfHiJoDa88xc0QOOUgCnI9feHZXvNAwr8ykv8ABg53yrQCxGLadnh48D5uXVnLc4HEFMPEsqu4BrOveW26PEqDd9PrLgIVmsFX9YFUOT69oJTsvE//2gAIAQMDAT8QsiBCjRBL5lVHBlIZWplF4/fibhzBrFM517/7EgV8RbAgIti8OiczmBQTBFYBUXCrpAtC/q9e0bSfzN8Pf/v5gSML9/rHVH0RWiv%2BSy0ajQNUKghwCRaYWTA5Mf8AIijZ5lLByhixMGIocGNTeW0AaDH9zsIS/h8zSypKIIl1/n6gqgeOJlTa7llMWUvcXIrU36PX0mHI%2B04C8mHAtb%2B8GrcrnBtK7XUWQ3LdiKFVqlqkcQEtf9%2BviUgEdApeCgx%2BoCCi8iueqzGCmUVk08%2BmaqzVgY7de05SeXBDjF5794KBvLp8GfJBi1e5%2BjXfMDsUtZ4fAmIhUXAjYnvwvFy2bIgw5KnsN%2BvXUS9mCso%2B0rZsR/LAoa4Y0aUJfFv06lrbSJle4KgXKC1AAJzNQiJbKu5ltmpxVfaVpY5sX5lt53NytJXDqGSMyWZ4lOKv/WJh3BdoK9%2B1%2BOZYANPv9H8%2BYikXPGh%2BeJdVW3TeHWhb5ow6xM0Te%2B/HmHCzP1P17zOVlFXV/EsZVXQ59uoiWAXY0XLAY55uIoveHFni9/DBLg5wZvlH9ajVpLtv8j0yiNJm%2BHorh9ICooefPmpQII36qIMGAoodGULhYUWoXsYwTBxH7EQobmtLdRFl2ywc9xCg/gDeHWY40oEeoOzWvROOLYBz3OVbiWrSUI2scOX3xfzweYVug5A0P2wxV7Y%2BbwJlO638vtULax0/PB6qCpPfcRgo0UY%2B0u17nNPpuAJn2dHYu4lkGHG8axfMomK3nuvm9xSzC6zmIRR8%2Brl0YKW/EKGpXrD7%2BveBQAD351VeuJWooMiod2BeNYcwAkFnmmsZX7XlYLImb1iArnEICA5NpkpamFQxPLSLhAL38wLs4IcofeZW7yhdg/2WFHp8xFeYA0MIttSgBjkHP4l5myKr%2BEyUa9yimN3LNaaWOgNrgYgBY0VyPbW/YjOttd1%2BEahsf91AVqs49dRtoq4jVjY3xKJXRvNb6rzBLYfd/qF2Mmrv/OIWLG/Ocb/yDyw2u7/f65l/s1XD69VGsUSxoseW/f6RJQDxzWfm5Raweb3nvxFDkxrs9fMS8BzX1cdGGqxh48X5Db%2BYG0f9PXMtUCv6TIJlX9SzkFXUzx6/iZocsDL6zPRx6%2B0dn2gXSytZr/JbaOBqBtMI2yxNjaymSczNgZJw29sqxQMQN0NJ6faBqNuYozANcAeFeiY%2BU1y%2B3xzEJwU%2BfPiYPTcHjDeE/wCcnmJUAVt9YIECC2mozSk98/ncF6Nirb58D/2WRfgf5AEErkw12y5UtOTPvX9QFoWnir/fxEG62zfxziUX08vb2gNQtt59oLldaMOmBQrd8%2BORz1zBCF3Qn1p4fMKByUMKyF7s5vnUMSojft5jdUf6QLQ6hSmk7YCSFKzMbS/%2BY1Y8MyUjZ1KB7P7gJjl9XLeUROjP4/7DCz0%2B0pqqghhrm3EbGef36uCBrf2vn/IWhEWFqrF9nn4lEQBwfk8MWmuznn4gEaX8F/8AJheDp069VAoQtb68HUAUMnMyP3e3mDABV5w/niCyFDspx4xxCF8t8KeuJQ12pr55hTdM69vVzkXauLPtcHAvrd/P%2BRNUFPtn/n1mvy5xePb8z2J4bPc4YKvKFnEq0XLWr6fMQKBoWJwnrOo%2BAPEVb%2B8b87jcv08QFn0I2ClkCTDHIbqFOLMpuhVPNYi1UrP33OmZz2QKk4TAar36uXjY/wDfzFhFB84lVm03161AiuX7dRQEXwKt8rwd59oydVY2C/Pe5mVEt8%2B8BLK/W5RYMe31qUyrJ69fqYMADfPr%2BoCZxyJ6%2BviChnb2ijWb1zruVHK8G6x3CnN19b/qDEq0az/UUAW4tr9RzDjn1orqDKC/Z6/uK3xHjXz6MRAtQxR6q5lXdW/73HPpQ2e37IKqnxtOPpr9Shmy3r2zDdbTks3Yy6WyoKfOvbfSSzrZ%2BZu3z9oq0ZESgo%2Bl9zTeq/qEeRChG4OaWqIU0tYrEGFiMNzAVcnEsHBr9RlE13EOTj7%2BuIRgUy7hIMFB0GvTefEtP75VKWAZjYID7n%2BjqXUlaIlDtNfuCwosRrMAfjd1KcyY4/fiIsUx8%2B/H%2BRG0P4lxo05fXj4g0Shvt/EUBcvvXtDicMtZ/FP2lmZr3%2Ba6/EMnCb8%2BeH4jts4D3roz7xIzOfn4mRrbQFQA60jh8PXceho6E0ePrEADZnuoOjvTOLQtDmv2wzV/j1cytfp/kKSzTx7T2PtDDRu3MQqNMLWGpeC3ExWRmGBc4ewfMyrI9auIpTyjVKFWytDuNAShttro6/MSBWIGNFa8w6dlwcV9Jsu3x9orBlTn2gttUrz8eZcqo65/xlVTPzgqceOK%2BvrMMJpZhwDnz%2BzqPzQ1m69eZdiJV4C/P0ihkN49Z8y%2BoodY42ygWjHj6Qz0ueb%2B0FtoS0OD9koAovDY/wBfBGFLWfNBz7feL9LmUEWxmvX4jtbgxXJ%2B663KDDUeZZTdXZ45PkgkVHEcX0/FU%2BYFCOqib6%2BtQM03wXBSrL9v3OKuCKdkAapgiqEXIXfZBooy7g7Gs69e0crKUUGvRAYhV59oNE6DoMGfWJetM39YLabKweviIirRee%2B98RnKbdc%2BJT27SjcfEOc7v6weDL9ZtfTUMChGH9fSecHqokqnz4gi%2BTH6iTqscnj/ALAaA3%2BD/sBaDH4moP6f5KQQuq1Hgjjisf145lplkUhZ9NkEZx5Mn7pZl5a7sbx1n7wNpThTLVKrb/kbfVar5pyPUS4rreR%2BunHyS6AyRVGkvJFAqtRLs8S3V4nI4r43iNHYL1%2B43RTPc2pv9P8AsWy8ZmANm9S6p4Zr%2BpczeDq8afn9zZhu78xlWZe0VVBd%2BvEyAun16uXYSholhctzQSjv9zKGn3jRsyp9GKbaHMonFXXtLbvmwc0/TfiJ1ZT4g5ciOq1Hr0QjKjEQU36yQ95CdkGzbp5CKBsPNSm6yHX4%2BY7Co88b/IbjvSh7cv8Az8SvvGra0A/SNXYH15%2BRlOUyBVRVKsrMCXRXrMcFUNW1qByYXf8AUXBwn7Jmim69f8lt3w4l0hKqC084hKtnGNuvs0xxWG3BkWv318RkFk648/5FWlxh6faYgJ4mNmdZE0ytu32gKFWmXgxANO/v7VFTNH6Tbmu39wZk9%2B831AUl4gBWtzMU4ZgJr9zz9pztfhFQipEoleJVyZaD8fMoM46jrOZj576iINsO6z%2BoCxrZ36/EIo779cyl9HH1/qO6Kglun3%2BI01VCnw8Sw9Q41viBaEsXtgBlzEfkRV4Xn68SpI0xc41U8t89SxI5liGyuvd9qnXi5%2BsAbYZda/7FNKtn146%2BYqgdaz56jCVy9fES7C30%2BkbrZ78cRqHo%2B1ylUVu%2B97JbqyL%2BDcVhg7%2BsVUzb5gKBu/NRQBZcAsVWohgO2JRFdb/5HoCiUY7MWwPTUoUhUv8A2Bjtz7VD1mw0%2B%2BM/Mra7a6Jch6P0%2BEnkj%2BiB11vPVrmvpKK5zBvjeZaJeHhuYpvUU8m5dRrEDZWPzFNNH7iLJ0WWB5PMRUGil4NX9ptfomVpxeawJO4AqXTayuv894YEZqXyXLxuCopt%2BftBFZ4qJNpSowqh/USFmvvBlbQeY3BvuWqwNO/Ht6uVFON9fwAivJ%2BeISz/AFCzduZRAccmoTVSu/H6YtWLN%2B3UdsdN%2BP7SVLcnfr9wnDqJpHCvPrUq0yD8m/7hTbUF2GbhSrtlVkTM0ELiJbv1xLAyPPvE1WAPX0nS/MC1s07lG0luNfqXFShB8FNHs7%2B0spz555htkZmBFBkAwePzH4X7/cJp/wBt16qMY5EJS1j5yYge3cbl0G%2B%2B4rW9795dnFIxAMNJS155mALKJSwO/wBSjVZrctZKM7g/uUAM7MTM0t/5AC%2Ba%2BnqoRpT9wmbQzu8l8wrBkdd3xCWSeghDZtLgVgXAUVh1UVGHMWlkbtvcKFp4iVVqKChqZRVi%2B3EaFDUyhpjHP79oi7BfH09EuAZd71AxjAkD3h6%2BYBDQe8ByL6x7eu4WgaGPgRTKLcd%2BJZRn1/UT3bf9gFn/ALCwOniNgBReYjKXmCBQXFxDTE2/BqXpJfmU3iA2UsL9/wCvaNrAfMNBm8/PUpFZa8RWAbweu4yhdHoy1B5O8YfWZqJSvm8/aC4CyW0W6l4us/wYS3Pr9RxQywVg1LUrzFPgiBDYM/EtDp1BBEykK1zh5zRFS3czvdSgpY483UUFCmsevrMVNcK8vlldp7/mKztv1qc3Axsj/wA8wHI3/cMNyQQsk79vXUwsLviW5sotlYvao7UwPzF9Nz6zFg2SA2HA/mIFcY57gSi09XBUMLxW5da4V9u5WLIXddf1mKbNXPr1UqcB6%2B3iYm3TWOGs/WLZrUVduP1AKvNfmVRU4cxYXEL0OI0rOaltpncE5A/MwqohAFG/XxCT73OjnHvrMKeaV%2BMaYaKM8f7GIVf5StwTmxhhlM19vp8QUjT7TfVePX9yxRqq3%2BoQpChZ/kNai1zj2g1mgCrJhZfGO/pGMUtMxNgWnXmXod1EB4CWWygZP%2BQs5Cn67gQRFluLek3qWY1Hz9MXHVbwL58fXmUCxPL6%2Bu5vGPrxEQiJ9oGWBXHXxAQIPy8n1lMk2LDKVaB5FfwaYUYAjxfxAB3RpZy/cVK0riVaqGAq0v8AUowVtxKClbKX3a7fGa8yzgit%2B9VcAJrNB%2B/aCrg/CKxoF59tSkq1Si/6yX8zfjCrL7dRCsIWUaT3f6lLtn2n48Qu1Md5xrHp8zJK6Wp8JTmAIi01Gouw%2BqAE2tf3ELCrH49e8SzshpwWxbn4jttMmq%2BoYrncBp7Zy%2B7KJTkbNJUqM5Hk9FeCYZQ0to4v99Stdwib9XqBEAkfXGYTyj8Yf9lPfm%2Bs7%2BMR6O4usuv7nisQNjVTVr2/gwjzEvGalIwK/h4lgVRlW5uuZyW5v1mGVPTc33n1uIGAF5uuL0cf9hkuc%2Bn5/McB5%2BcejMLozSfnn6SvpvL1t%2BsJdMP61UBcFPX3hF6hqJFRWr4/5CY4a8wt5jV74SOxa1neMfMIK3Z%2BYCrVY9PmYnyPmEM0tiNRhbzFJCG/z8xgaAcrm/H3mHC7quD1uIAbFUd6ggM/b1iKbLhl8%2BvWYSrNRghrP21jHHUZ5aAzbhq9Vb8QMaMHE6wYXXMduv0k/syeJx1LOMkAcYJlYauBsQzAPC%2Bn0iYOQzBQ4YFEu8%2BmbAi77L5UsPODeB5WGqdV9Zo4btwDRbEdQH4/0/EWdZNeu4Vrhr/ZdCafev8AYuCvOeMSkJTV5Hs7H7QKhEYoZPn64X2Zs4cuab4NbjjbLjPtnU4C6fMfZS9X%2BPeIFX%2B2YKFh3mUGWav27iaHSCqB2qBQrDTdY9vxMlFEcIVYnl/Go62%2B7EproOX3V9pdgLxVnn56%2BkRco1x9aiNAaz68wUGnN91DVyGtH9Sui53jGK%2BzbNYQW7cce%2BdSnlWXGi/7ywtVGKi01WZmscyhS59M5WVAFpQxVUcfSHlxzKqpiA1jbMYXgihf9w8SrPe437cQCXVNmq4PmWhYQqvvn1UuEqt69dREed/ZidQpm1wY/EuOjkLQ5b0aeeJXFJPA8Gt3uGlPC6DO7KsMYNkquNC7nms9aW66mmmTZfF9G/JKqXIPZ69riblLz3%2BokVn13DWUVf3lhdCob2MVNs2wOd4HWOY2TJMBgDgrfiNmTSkZN6wrVe1N8W4FGZGsuEPHcVcDjOrGA0NGINehMFFrd0PvK1FJGompeDSGu%2B/P9xwgJ0tqt/JzCCH39J%2ByW828Pgafj%2BHsydncUw7Zx3qWsqalvJMdPebb/wAmV7Q4%2BuZfSP2f8SGXUPGeR0n7l95r%2B1/XD94WBF0ej8y6Fj58yo0y48esQEvpp8eIaoqhtPX3jwUDSrzi8RcvRgbPfVyzQ3GQPdX8YjdA6sZbGty%2BkDYVb5W46D1G4a1ESANeiUJoCXGwfPr3hUTxsNtYvfdc%2B8uVYkqK81dVwxk0R2HvVma7vEx99lSYznedVKf6lqiqw8b6OYwkoc41s0fJAVUpv%2BpRGIhbo/bollrKje7BRfjxAuofVIbcwOSt2%2B7mKIUbvITcoKd21G0yxMHCm%2B4AtaRwCcS85pJbqsfeeznn2gLzGV28J9T4WYDp7%2BOpgLtl69ceJewv7PiZug%2Bgh1Li/eNSM6x/UTU5rPFtc1MTKgK/dv2izirN%2B8NLgvcpLYa%2BZhWFrxzMwU9PzBNNv/aJ7z99RLCWnZ64lEUNn0%2BfVRFGMEdBpgDPySxJwuqrfcqg%2BzH0gHQH5iUtYlMqUfuFbLqP%2BQgRve5cTSd05Fe0bqzkPVfxhpVzBNDFZiFVLdizUAM1c0ehEstb9M%2BipkRic2X6/MPzstaVWPuhUWpoFvTUwVPOfo/aVNT0XUyN460%2BIZdXffFmYvm7f8lRbK4hwO%2Bu4lTRK/2ZkS3x8QLXLJRyS4it3ATyBUaqtOfeGyxEa%2B0F0deZRhY4iQVpPm5ZOGWf8jN0%2BGIillOJxDpuUoGr9L%2BvEK6YP1KWbWsy1qbRC/f9y90VRyLR3hC%2B8Gg0cn7g07zKUHH%2BQ0gYYC3GI2qmwmb7GKtOMH1JlqjKTCoc69fMEcm7nPD%2Bpkui58Bv7LFaKIe4BX0YWFptV5%2BnmVAaL8wFC3fz/wAllqhFqwzX1IpZmeqmsq00%2B/rccwDjbsdsPCdwgE6iL3vrK2X2%2Bf8AIDFg/fUKqG/xFDJ/SA2PniWMw/pGoqn6ef8AssAILxEuWT18xwXg/SFQF3nnEqt3PUAgyC%2BIYHav6aIgl1UXaNBfuBmLKauBwEQt9soFOSNguSomPKZBor%2BYuiEG%2BPhHRnJKAK9e8QDO7/yOtwYzkjuZBAvvCm8aj10XJX25lYMUT5GUNsb8/ELIA4evaEhrd4TrmWg8VaO/McAggFNo%2B3qoD8BicipfxBUytA0D8JQ1cwLa53AVNC8TdxvHr5jkeD6uUFeb5x9JQzHHMvwQ38ygaWY/2UwrC/j3/UHMzX5gKF/nXjmIrErOgCr%2BPvLRV54YCdYz7tzYCq5ntOephvhfxBbLdkdMXBFYsxDZrEJZWJXZeYlVWQxxWC/vB1b66lQVvD7AeC33V8RICu63Wda7haDsQCVj744iOhruOG9469EsZw3C2waYouS611KWWXr9Y6nfzjHn/ZmFr/mCgocJj2tvVwKLjkriJEUMxAOYow39ZgBEX9xE%2BB/UaB5fXxDuDAjEHr/I3K3KVcADjXNwSSvIOHsNjqXxFAOR1G9MLq7tonAvULrJmWdErDnITCEzmFImoq2vz9Y7sKJm3dVKR3cKbu9RyqwZTrJ4DCHDpOE3TsAfqtiDwn5gijjTMgc5sfnUKzeXedQTDqChb2648yuz08ykox9a94k8Hjr3/wAmEtQk8GALkN%2B0Faqo%2Be4sYCzq5QjKyev9lqFnx63AGBfdqC4ZC1mZtsu9ZiGCst7i7ruIRAp36%2BspnglvML5hs4upaG5nK/gOgxLAecxq7CVef%2BJYjnEaK27I1ZowxK8TKxXGPXvCyAYQoqKP2m8OmN2lPUGcr5hVagXxmWVGP1FDeZLovIXxUQQiHPvCwDxUE259zDDV%2BpQFs8/SUS1VLiGi5zGc7VzwPmJrfkAqxkKZk2FHDb5cP2lIzHB9qir7Ezd4jvrwBk8X15hnAFG41tUtmVcT6v2m3NXd3WWBNi8uPoxpdsMfWEYrLUZyt%2BehfrEI6K/ll1QFe0M2K5JcLUtUyxXzCtSxjSukKhTeUwAq4s5E78TOQhirf9hpzKYWnfmYyBj8QBvuvXzGhyZj31IT6zdsvUIHT69zgsRKBt5/X0iMUJuNQBamffr7Zlxpev1KQtANWnvj11Btu6cVANZB9B15gMCXzMJunPd/qfYEx5mlX%2BTdFwvphajL6zOEW2oJk1T79Qub1X9x6OGebqq%2B8DDFNTFCwLy3uohLEq5nQG8S6HCswUMLGnTZLBDFnA4iqDXcDk9kt8QBUzKDS4%2B870OpZqEwUwVBYU/Ru5TarC/7zAeXMcIt5%2BTP3I4Jde8Ndn7SlacS266DuXpojiyi9R9CZRdKS5eDRXx9P7gvnC8fdloWsMF6wqKbXZ95YoFPxFtXjU5BLfvGKUX%2BpjLMPmIKAmHkhlyUQHOvVQbCF9Zv0SgQm3cWfBlm7cywuoyrIczSt5PTMLjUbVbdQTOYYur1MvBKHzHJCp5Vdn5lDDA79bZdRyb6YJB7QXkGa/qXm4/qolBSiQvY4UevMUHOH0ruWUA5FSus%2BqmDLat8QwAgGi/pCRsyclfWIWMd1BazPx6JcaCeeYLpfJ1/fU1fhniLDyK37fuYkWYhmkfsFnvEMXCgDdVDZW9x4eLzNFGo8FmBsFetS15/EVABioJRjPM2e8LLt9XMq8MRwAOZzdUyizPEp8wAPeCwxQEfcvzLMCGa3Y0e8KyA%2BvrEQsKdc%2BfWIdFlDL3FOwFm787nsKshb7bleoONFwJyE0Eg4wzlY4cmPxDFMPB85/EElKvO195wzMl4x%2Bsxi1Hu9tHEANBYNO4Zz9MQLAcIFnRv0xV3FTNtU9VtXzOarUFbUx6qFbZdosxtDBATX4TKhVA2H1/UtR9vvAao13OFhZZdwFauq/MpR8Jd4mMZjsriFhnmOOcVOWJf39pY%2Bs3zdeIwFl67MTIbl366gHJfriNC2s%2BvWo6Sz7236xUSpnBu/lX6i7gHL6%2BuoukFLwuL8XWqgi9DjBYchtgabF0tdtfhMYSWxNQF2Hnvv6TuOms146YrEzcVyB%2BsIFUK%2BkuUp6/uBlHQeXR1cy7um17XKvzFrAZhyMPxM1uWWB8Tal3iFmXEymHPpltVuYpziY%2BhTr7fqJSrow4xiJK3b1iGMDFRulcyi75l46TnW4XzPYiB6c%2Br7SkVvC%2B4ix8uu4tC6ZZYXBigp8u4ClUhqsuPf1mL2Dij2/5mHWKEed9%2B1VAX6q61S8ecZY2KGjgcqS8dZs%2B%2BPMf3PJGGJfGPzEA278RrA1cUk53CYthDvgfMrIuNLdTZp5mLMrj8ThtHQFA8RShxXxDQbJaF5uItB/2FVjU2MPUS2lhXMvBZVU3/AMlmh1KpzOfExiolV/CIUCPSTJDBPkNPG0Urx/mGEf5iDo6PeWS1/wCfqNQZr/kvMw89SqP0X9Y0joXl1Pc7hZYcr3d/EKDhGNTLuIhdZzmMsex46qOG07KMTQI%2Bs/eAvCVWt6x68Qyc73Eq8qRc56Ta5xzHK6KlmMYuXgxZqZN6YiW0nG%2BZlZRG4ZNYgih7TdWYgEaQp4dsZSfMvi47zOZnHURo5LX1jP1mQKtp9c6IQBdfnMECnSYAxf29oEBCnH%2BxaN4Y4j5C8DvEox4b33mbCj8RooYP7h9tx7wRbLKi6xQaPPr%2B5YRX%2BI5TP89RTQX9trEssOVhdC5iZLyf1GrM2R0TFwvxmawRboY7iZq2oFssNUm6W9y6KMVL5qLpN3LAirugM/wVgr7fxeK/jNcXPEC%2B%2BXftFNH1D6%2B0Pg3mYfVB5%2Bz%2BCaEB%2Bf1GOQc9EEJUfWNBUtvp1cwBS8QNkkwbQgDwLHFOX6MEYD6uCUK/t/yWQr4/U7CODygZItuzKyoxe4FES6PvMGkxf/J2acEKDaiWXVVEU4zFfB%2B5QtGIHKQwVqF5SLfDPPMwVZmWdSsYKf5DFOZS8F/Ky2KZG74/uUwUU09QIuiteZRW1iYg/qe3rUrNhYIk03fnUHD3IKJyLHOvj194kSIP7iRYiKDp%2Bk2AViOCRD2%2Bsz0HfXcxEmb65%2B9zihzC34Qasan9yxcmrl5Y4mRBaJyRVDvUtp9fMtW9mVoaYjYfKXzavMFrcF87jXOphRll/MUtUvNRMGcywXFH5iH8lwRDw%2BK9ZmlHcqlPRf7fWoKtV49n%2BoVl583%2Bv3AoEr8wMClu%2BpU6a%2BPXmUMgAlKEY98/Msylt/5xD0TcYkVsxFbyOKjsQJWPEZlBt1jQl989yglIqfK/pMl5ZatUczLhwLMIXxAqJqoEsrEAsXnNERyaEEPn8wwDxc5SjCYgYqYs6nBWZXWKgUDcK7IzhvMirPt94R4rU3Tg985mK6vWPrK0FpMy2BxCpE/4hVyfh8kCZYQSkRjPBDVAH6zJEoabqZXT5viCtp8/bLAWMLmAUO/tACWf3EDFb%2B8BUyN%2BWZKt3zeCKLnDoid6ZghjNn3vjx1BLptllK6/EQWfCC49x8etSzYaiXofVymhcTSXeNzBRpPvKoMQUjiDDhGwTmHTzLT6w7XDZDYaf3LQG0UOCoeHs8wzHiGTs5xqZDRupXgjWfRKoLRePXGYsC22PeJac2/vf0i5yp186lBzfvZ/sMsXt7wW0vTevWoFaZIU2FPHUuETBYBZ9fiUMml/8jgl7YCxxf5/cXXb7ezwyo358JbLvPZvX4JRw8xxs5lcuP8AsQBjLuLRTKgLkFtRuUFupZaHCfSltVxCrzC6KJmkTME848wCnPfiYXXMO0vCSlQ7aiEKn53E4VafiFCGektLPsgVLYIBAeH6ygsn5JcNmD17kSulZ8S8j7/MV7G%2BGCLbRxqDZVXBsBTvUrDpMde17mWmTuMqdj1mXXnAVFTRrOPMsHlxqUcjh4Cygz7tGoHKyojDA0GtxRQsQYbIAD4gbL3TKZheKhtKzExwuGU2%2B8AcreGUUjWeYKuQeICWK8Y5/uUyBzdKU%2BKlmqA65vOIXhD75lxBzxLTBLnkDz%2BotBGXr/ssNuhv4iusNHq4haGvzz9ZlSYuT17ysMmMwQ49OYqdri0zTeYo2KvcFPqI8m5BBQ2T4lFhSn43BI1%2B/wDkAChb1%2B5VtcAbTZfVuPMwB0QKxBYeJYFdPxqLIXmW8AjDy/uWBZfq43w5jVAhpW4Bh6R19HzFacDMaqpr1cavG%2BoNNLP6iiQyxGWEN1uNn3P1CwsemZcnKWEtaq/eJRLsQ1KZJZeJZWNP48xKA7mfvj1iZOR1mI1Ke/XUaDMfeJyA%2B0GHDXtCrgu/aXWwESHYkvyDn3hXQos/zwwm9H7P%2B%2BI5IOpb0ey9uPI9SnkpR4AHGAvzbA4ueRggHcW7aqAlKeTjEdM4lBVWMBSaJg21XMNILe5pt7gJDhAWvrEqtEap8/aWLZbOo6w/PMQBwi1asDOLccqpz3ANtQdgrIjEs2Y67mdiIth19pYsxjq4Zhm32mM5fsyh19uW8QAD7woKkuvaGIF91wdtagW8Fx8RAs2sSgX%2BDLIrB1Dk2eOIResNe3wS90ZaB330HdSyZTieFfe6OCubhyDPE4%2B6ZMwYbfmF0NCMwGyDgYgQ1vMCdgIARwrnUQMJgNhMCWt1EIDT9Qcsc%2BIYHBTEQp55i1RTU0Dp6ljlvcrVop3tvJ1nXBEqrjHxONySgQ9LnAeX34jUUZa81LFjh6itGnjPiGzt9oFPzx%2BJSDLlEtdsJFWfP9xOAD9n1jllW3E5ehjQM5TNOc4A%2BL2vW5kBhtFnw0ex8wJY0v0lWWajee4J1LFUMSwFZEKHlG2%2BCIoG4LVrMAtV/wAbJ3BFZNXBAHDh8yisNaiO2LOc7jl4fmFgpmJqEccP195TikBcId25TiuIYjdXeeopSDi5mp0wK0E9Zha4gkKqx8y9bQgJvAD1fiXSdn06l2FbK6MRILi69evaDFVAUS1MVRWamQYFZbj9i1A98Okd9c9mDRwl23OTllWW4cwaUr1%2BoKHkQYZ5SosdjFyJnxLDDI3uf//Z'

In [76]:
print(base)

...


In [73]:
base == base2

False

In [70]:
print('here')
img = np.array(Image.open(BytesIO(base64.b64decode(base))))
print('here1')
label, value = model_prediction(cv2.cvtColor(img,cv2.COLOR_BGR2RGB),model_dir,0,0.0)
print('here2')

here
here1
(80, 80, 3)
torch.Size([3, 80, 80])
(80, 80, 3)
torch.Size([3, 80, 80])
here2


In [47]:
value

0.9999997615814209